In [126]:
pip install geopandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [127]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [128]:
!pip install snowflake-connector-python[pandas]

Defaulting to user installation because normal site-packages is not writeable


In [129]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [130]:
from datetime import datetime

# Generate today's date in the format YYYY.MM.DD
today_str = datetime.today().strftime('%Y.%m.%d')

In [131]:
import os
print("Current working directory:", os.getcwd())
print("Files in current directory:", os.listdir('.'))

Current working directory: /home/ubuntu/Phoenix
Files in current directory: ['JTM_Sectors_Likelihood_Based_2025.08.05.xlsx', 'phoenix_sub_ba_jtm_orig_2025.07.30_update.xlsx', 'phoenix_ba_jtm_weighted_wage_2025.07.17.xlsx', 'phoenix_sub_ba_jtm_weighted_wage_2025.08.07_update.xlsx', 'Phoenix_JTM_Enhanced_Deliverable_2025.08.07.xlsx', 'JTMs_Below_Living_Wage_2025.07.31.xlsx', 'OccupationalProjections.xlsx', 'phoenix_sub_ba_jtm_weighted_wage_2025.07.16_update.xlsx', 'phoenix_ba_jtm_weighted_wage_2025.08.07.xlsx', 'phoenix_ba_jtm_orig_2025.08.07.xlsx', 'phoenix_fast_facts_sub_ba_jtm_weighted_wage_2025.07.25_update.xlsx', 'JTM_Sectors_Likelihood_Based_2025.07.31.xlsx', 'phoenix_fast_facts_sub_ba_jtm_orig_2025.08.05_update.xlsx', 'education.xlsx', 'JTM_Below_Living_Wage_2025.07.30.xlsx', 'phoenix_fast_facts_sub_ba_jtm_weighted_wage_2025.07.28_update.xlsx', 'phoenix_sub_ba_jtm_orig_2025.07.16_update.xlsx', 'phoenix_ba_jtm_orig_2025.07.28.xlsx', 'phoenix_sub_ba_jtm_weighted_wage_2025.08.06_upda

In [132]:
import importlib.util

# Direct file import 
spec = importlib.util.spec_from_file_location("config", "/home/ubuntu/Phoenix/config.py")
config = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config)

In [133]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import snowflake.connector as snow
import os
from pandas.api.types import is_numeric_dtype
import re
from datetime import datetime
from collections import Counter
from collections import Counter
from snowflake.connector import connect, DictCursor
import gzip
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows


In [134]:
# The Snowflake Connector library.
import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas

In [135]:
# Set up the Snowflake connection
user = config.credentials['USERNAME']
password = config.credentials['PASSWORD']
account = 'PCA67849'
warehouse = config.credentials['WAREHOUSE']

conn = snow.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse, 
    database='',       
    schema='' 
)

In [136]:
# SQL Query function

def query_table_sf(conn, database, schema, table, query=None, batch_size=10000):
    cursor = conn.cursor(DictCursor)
    try:
        cursor.execute("ALTER SESSION SET STATEMENT_TIMEOUT_IN_SECONDS = 600")
        sql = query or f"SELECT * FROM {database}.{schema}.{table}"
        cursor.execute(sql)
        
        cols, dfs, total = [c[0] for c in cursor.description], [], 0
        while True:
            batch = cursor.fetchmany(batch_size)
            if not batch:
                break
            df_batch = pd.DataFrame(batch, columns=cols)
            dfs.append(df_batch)
            total += len(df_batch)
            print(f"  • fetched {total} rows so far")
        
        cursor.close()
        if dfs:
            df = pd.concat(dfs, ignore_index=True)
            print(f"Successfully retrieved {len(df)} rows.")
            return df
        else:
            print("Query returned no rows.")
            return pd.DataFrame(columns=cols)
    except:
        cursor.close()
        raise

# Simple mode() like R’s
def mode(x):
    cnt = Counter(x.dropna())
    return cnt.most_common(1)[0][0] if cnt else None

In [137]:
# Parameters & Date window

data_end_year  = 2024
year_end_month = 9

min_date = f"'{data_end_year-2}-{year_end_month+1:02d}-01'"  # Added :02d for zero padding
max_date = f"'{data_end_year}-{year_end_month:02d}-30'"      # Added :02d for zero padding

print(f"min_date: {min_date}")
print(f"max_date: {max_date}")


min_date: '2022-10-01'
max_date: '2024-09-30'


In [138]:
# Phoenix-Mesa-Chandler MSA Definitions

# MAPPING 1: For Snowflake posting data (format: "County Name, AZ")

phoenix_msa_definitions_snowflake = {
    "Phoenix–Mesa–Chandler": ["Maricopa, AZ", "Pinal, AZ"]
}

county_to_msa_snowflake = {}
for msa, counties in phoenix_msa_definitions_snowflake.items():
    for c in counties:
        county_to_msa_snowflake[c] = msa

# MAPPING 2: For ACS data (format: "County Name" without ", AZ")
# Can also use MET2023 and use Phoenix MSA Code

phoenix_msa_definitions_acs = {
    "Phoenix–Mesa–Chandler": ["Maricopa", "Pinal"]
}

county_to_msa_acs = {}
for msa, counties in phoenix_msa_definitions_acs.items():
    for c in counties:
        county_to_msa_acs[c] = msa

In [139]:
# BLS education benchmark

bls_benchmark = query_table_sf(
    conn, 'outside_data', 'bls', 'occ_deg_require',
    query=f"""
      SELECT DISTINCT
        "soc_2018_6_code",
        "soc_2018_6_name",
        "education"
      FROM outside_data.bls.occ_deg_require
    """, batch_size=500
)
# map text → numeric
education_map = {
    "No formal educational credential":0,
    "High school diploma or equivalent":0,
    "Postsecondary nondegree award":0,
    "Some college, no degree":0,
    "Associate's degree":1,
    "Bachelor's degree":2,
    "Master's degree":3,
    "Doctoral or professional degree":4
}
bls_benchmark['edulevel'] = bls_benchmark['education'].map(education_map)


  • fetched 500 rows so far
  • fetched 847 rows so far
Successfully retrieved 847 rows.


In [140]:
bls_benchmark

,soc_2018_6_code,soc_2018_6_name,education,edulevel
0,53-7071,Gas Compressor and Gas Pumping Station Operators,High school diploma or equivalent,0
1,53-7063,Machine Feeders and Offbearers,No formal educational credential,0
2,53-6099,"Transportation Workers, All Other",High school diploma or equivalent,0
3,53-4022,"Railroad Brake, Signal, and Switch Operators a...",High school diploma or equivalent,0
4,53-3099,"Motor Vehicle Operators, All Other",No formal educational credential,0
...,...,...,...,...
842,15-1243,Database Architects,Bachelor's degree,2
843,13-2082,Tax Preparers,High school diploma or equivalent,0
844,13-1021,"Buyers and Purchasing Agents, Farm Products",Bachelor's degree,2
845,11-9072,"Entertainment and Recreation Managers, Except ...",Bachelor's degree,2


In [141]:
# BLS SOC crosswalk & aggregation to 2021 codes

bls_lct_soc_cw = query_table_sf(
    conn, 'crosswalks', 'custom', 'soc_2019_soc_2021_crosswalk',
    batch_size=500
)
# lowercase columns to avoid case mismatches
bls_benchmark.columns   = bls_benchmark.columns.str.lower()
bls_lct_soc_cw.columns  = bls_lct_soc_cw.columns.str.lower()

bls_benchmark = (
    bls_benchmark
    .merge(bls_lct_soc_cw,
           left_on='soc_2018_6_code',
           right_on='soc_2019_5',
           how='left')
    .groupby('soc_2021_5', as_index=False)
    .agg(edulevel=('edulevel', lambda s: np.floor(s.mean())))
)
# back to human names
edulevel_name_map = {
    0: "High school or GED",
    1: "Associate degree",
    2: "Bachelor's degree",
    3: "Master's degree",
    4: "Ph.D. or professional degree"
}
bls_benchmark['edulevel_name'] = bls_benchmark['edulevel'].map(edulevel_name_map)


  • fetched 500 rows so far
  • fetched 866 rows so far
Successfully retrieved 866 rows.


In [142]:
# after you’ve got bls_benchmark with columns ['soc_2021_5', 'edulevel', 'edulevel_name']
edulevel_map      = dict(zip(bls_benchmark['soc_2021_5'], bls_benchmark['edulevel']))
edulevel_name_map = dict(zip(bls_benchmark['soc_2021_5'], bls_benchmark['edulevel_name']))


In [143]:
bls_benchmark

,soc_2021_5,edulevel,edulevel_name
0,11-1011,2.0,Bachelor's degree
1,11-1021,2.0,Bachelor's degree
2,11-2011,2.0,Bachelor's degree
3,11-2021,2.0,Bachelor's degree
4,11-2022,2.0,Bachelor's degree
...,...,...,...
790,53-7072,0.0,High school or GED
791,53-7073,0.0,High school or GED
792,53-7081,0.0,High school or GED
793,53-7121,0.0,High school or GED


In [144]:
# Fixed SQL query that ensures education names match education codes
def query_sf(sql: str) -> pd.DataFrame:
    cur = conn.cursor()
    cur.execute(sql)
    df = cur.fetch_pandas_all()
    cur.close()
    return df

sql = f"""
WITH bls AS (
  SELECT
    cw.SOC_2021_5,
    FLOOR(AVG(
      CASE occ."education"
        WHEN 'No formal educational credential' THEN 0
        WHEN 'High school diploma or equivalent' THEN 0
        WHEN 'Postsecondary nondegree award' THEN 0
        WHEN 'Some college, no degree' THEN 0
        WHEN 'Associate''s degree' THEN 1
        WHEN 'Bachelor''s degree' THEN 2
        WHEN 'Master''s degree' THEN 3
        WHEN 'Doctoral or professional degree' THEN 4
      END
    )) AS edulevel,
    CASE FLOOR(AVG(
      CASE occ."education"
        WHEN 'No formal educational credential' THEN 0
        WHEN 'High school diploma or equivalent' THEN 0
        WHEN 'Postsecondary nondegree award' THEN 0
        WHEN 'Some college, no degree' THEN 0
        WHEN 'Associate''s degree' THEN 1
        WHEN 'Bachelor''s degree' THEN 2
        WHEN 'Master''s degree' THEN 3
        WHEN 'Doctoral or professional degree' THEN 4
      END
    ))
      WHEN 0 THEN 'High school or GED'
      WHEN 1 THEN 'Associate degree'
      WHEN 2 THEN 'Bachelor''s degree'
      WHEN 3 THEN 'Master''s degree'
      WHEN 4 THEN 'Ph.D. or professional degree'
      ELSE 'High school or GED'
    END AS edulevel_name
  FROM OUTSIDE_DATA.BLS.OCC_DEG_REQUIRE occ
  JOIN CROSSWALKS.CUSTOM.SOC_2019_SOC_2021_CROSSWALK cw
    ON occ."soc_2018_6_code" = cw.SOC_2019_5
  GROUP BY cw.SOC_2021_5
),
postings_with_edu AS (
  SELECT
    p.ID,
    p.SOC_5,
    p.SOC_5_NAME,
    p.SOC_2021_5,
    p.SALARY,
    p.POSTED,
    p.MIN_EDULEVELS,
    p.MIN_EDULEVELS_NAME,
    CASE
      WHEN MONTH(p.POSTED) <= {year_end_month} THEN YEAR(p.POSTED)
      ELSE YEAR(p.POSTED) + 1
    END AS year,
    -- Use BLS education code when posting data is missing or invalid
    COALESCE(NULLIF(p.MIN_EDULEVELS, 99), b.edulevel) AS final_education_code,
    -- ALWAYS derive the education name from the final code to ensure consistency
    CASE COALESCE(NULLIF(p.MIN_EDULEVELS, 99), b.edulevel)
      WHEN 0 THEN 'High school or GED'
      WHEN 1 THEN 'Associate degree'
      WHEN 2 THEN 'Bachelor''s degree'
      WHEN 3 THEN 'Master''s degree'
      WHEN 4 THEN 'Ph.D. or professional degree'
      ELSE 'High school or GED'
    END AS final_education_name
  FROM EMSI_BURNING_GLASS_INSTITUTE.US.POSTINGS p
  LEFT JOIN bls b
    ON p.SOC_2021_5 = b.SOC_2021_5
  WHERE p.POSTED >= '2022-10-01' AND p.POSTED <= '2024-09-30'
    AND p.SOC_5 != '99-9999'
),
national_by_year AS (
  SELECT
    SOC_5,
    SOC_5_NAME,
    year,
    MEDIAN(SALARY) AS median_salary_national,
    COUNT(DISTINCT ID) AS posting_obs_national,
    MODE(final_education_name) AS education_level,
    MODE(final_education_code) AS education_level_code
  FROM postings_with_edu
  GROUP BY SOC_5, SOC_5_NAME, year
),
yearly_totals AS (
  SELECT
    year,
    SUM(posting_obs_national) AS total_postings_national
  FROM national_by_year
  GROUP BY year
),
national_with_shares AS (
  SELECT
    n.*,
    t.total_postings_national,
    CAST(n.posting_obs_national AS FLOAT) / t.total_postings_national AS job_share_national
  FROM national_by_year n
  JOIN yearly_totals t ON n.year = t.year
),
growth AS (
  SELECT
    *,
    LAG(job_share_national) OVER (PARTITION BY SOC_5 ORDER BY year) AS prev_share
  FROM national_with_shares
)
SELECT
  SOC_5,
  SOC_5_NAME,
  median_salary_national,
  posting_obs_national,
  education_level,
  education_level_code,
  CASE
    WHEN prev_share IS NULL OR prev_share = 0 THEN NULL
    ELSE ROUND(100.0 * (job_share_national - prev_share) / prev_share, 2)
  END AS obs_growth_national
FROM growth
WHERE year = {data_end_year}
ORDER BY SOC_5
"""

# Execute the query 
posting_data_national = query_sf(sql)
print("Returned rows:", len(posting_data_national))
print("Education level distribution:")
print(posting_data_national['EDUCATION_LEVEL'].value_counts())

# Check surgical technologists
surgical_check = posting_data_national[posting_data_national['SOC_5'].isin(['29-2055', '29-9093'])]
print("\nSurgical technologists check:")
print(surgical_check[['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL', 'EDUCATION_LEVEL_CODE']])

Returned rows: 766
Education level distribution:
EDUCATION_LEVEL
High school or GED              475
Bachelor's degree               180
Associate degree                 43
Ph.D. or professional degree     38
Master's degree                  30
Name: count, dtype: int64

Surgical technologists check:
       SOC_5              SOC_5_NAME     EDUCATION_LEVEL EDUCATION_LEVEL_CODE
316  29-2055  Surgical Technologists  High school or GED                    0
328  29-9093     Surgical Assistants  High school or GED                    0


In [145]:
# Read Employment Projections CSV file
# Read the CSV file
employment_projections = pd.read_csv('Employment Projections.csv')

# Select only the columns we need
columns_needed = [
    'Occupation Title', 
    'Occupation Code', 
    'Employment 2023', 
    'Employment 2033', 
    'Employment Change, 2023-2033', 
    'Employment Percent Change, 2023-2033'
]

employment_data = employment_projections[columns_needed].copy()

# Clean up the SOC codes - remove Excel formatting (="XX-XXXX")
employment_data['SOC_5_clean'] = employment_data['Occupation Code'].astype(str)
employment_data['SOC_5_clean'] = employment_data['SOC_5_clean'].str.replace('="', '', regex=False)
employment_data['SOC_5_clean'] = employment_data['SOC_5_clean'].str.replace('"', '', regex=False)
employment_data['SOC_5_clean'] = employment_data['SOC_5_clean'].str.strip()

# Rename columns for easier handling
employment_data = employment_data.rename(columns={
    'SOC_5_clean': 'SOC_5',
    'Employment Percent Change, 2023-2033': 'employment_percent_growth'
})

# Merge with posting_data_national
postings_with_projections = posting_data_national.merge(
    employment_data[['SOC_5', 'employment_percent_growth']],
    on='SOC_5',
    how='left'
)

# Replace OBS_GROWTH_NATIONAL with the CSV employment percent change
postings_with_projections['OBS_GROWTH_NATIONAL'] = postings_with_projections['employment_percent_growth']

# Drop the temporary column and update the original dataframe
posting_data_national = postings_with_projections.drop('employment_percent_growth', axis=1)


In [146]:
posting_data_national

,SOC_5,SOC_5_NAME,MEDIAN_SALARY_NATIONAL,POSTING_OBS_NATIONAL,EDUCATION_LEVEL,EDUCATION_LEVEL_CODE,OBS_GROWTH_NATIONAL
0,11-1011,Chief Executives,98000.0,64067,Bachelor's degree,2,5.5
1,11-1021,General and Operations Managers,80000.0,370084,Bachelor's degree,2,5.8
2,11-2011,Advertising and Promotions Managers,104582.0,3794,Bachelor's degree,2,-2.6
3,11-2021,Marketing Managers,137500.0,210604,Bachelor's degree,2,8.2
4,11-2022,Sales Managers,100352.0,290916,Bachelor's degree,2,5.9
...,...,...,...,...,...,...,...
761,53-7073,Wellhead Pumpers,55120.0,72,High school or GED,0,-2.0
762,53-7081,Refuse and Recyclable Material Collectors,41600.0,455,High school or GED,0,2.3
763,53-7121,"Tank Car, Truck, and Ship Loaders",40404.0,10644,High school or GED,0,3.9
764,53-7199,"Material Moving Workers, All Other",34840.0,682,High school or GED,0,4.4


In [147]:
def check_posting_data_education(df, dataset_name="Dataset"):
    """
    Check education mismatches in posting data (before merging)
    """
    print(f"=== {dataset_name.upper()} EDUCATION CHECK ===")
    
    # Define expected mapping
    expected_mapping = {
        0: 'High school or GED',
        1: 'Associate degree', 
        2: 'Bachelor\'s degree',
        3: 'Master\'s degree',
        4: 'Ph.D. or professional degree'
    }
    
    print(f"\nEducation level distribution:")
    print(df['EDUCATION_LEVEL'].value_counts())
    
    # Check for mismatches
    total_mismatches = 0
    for code, expected_name in expected_mapping.items():
        mismatched_count = len(df[(df['EDUCATION_LEVEL_CODE'] == code) & 
                                 (df['EDUCATION_LEVEL'] != expected_name)])
        total_mismatches += mismatched_count
    
    print(f"Education mismatches: {total_mismatches} out of {len(df)} occupations ({total_mismatches/len(df)*100:.1f}%)")
    
    if total_mismatches == 0:
        print("All education data is consistent!")
    else:
        print("Education mismatches found - consider fixing")
    
    return total_mismatches == 0

In [148]:
# Check national posting data education consistency
check_posting_data_education(posting_data_national, "National Posting Data")

=== NATIONAL POSTING DATA EDUCATION CHECK ===

Education level distribution:
EDUCATION_LEVEL
High school or GED              475
Bachelor's degree               180
Associate degree                 43
Ph.D. or professional degree     38
Master's degree                  30
Name: count, dtype: int64
Education mismatches: 0 out of 766 occupations (0.0%)
All education data is consistent!


True

In [149]:
posting_data_national

,SOC_5,SOC_5_NAME,MEDIAN_SALARY_NATIONAL,POSTING_OBS_NATIONAL,EDUCATION_LEVEL,EDUCATION_LEVEL_CODE,OBS_GROWTH_NATIONAL
0,11-1011,Chief Executives,98000.0,64067,Bachelor's degree,2,5.5
1,11-1021,General and Operations Managers,80000.0,370084,Bachelor's degree,2,5.8
2,11-2011,Advertising and Promotions Managers,104582.0,3794,Bachelor's degree,2,-2.6
3,11-2021,Marketing Managers,137500.0,210604,Bachelor's degree,2,8.2
4,11-2022,Sales Managers,100352.0,290916,Bachelor's degree,2,5.9
...,...,...,...,...,...,...,...
761,53-7073,Wellhead Pumpers,55120.0,72,High school or GED,0,-2.0
762,53-7081,Refuse and Recyclable Material Collectors,41600.0,455,High school or GED,0,2.3
763,53-7121,"Tank Car, Truck, and Ship Loaders",40404.0,10644,High school or GED,0,3.9
764,53-7199,"Material Moving Workers, All Other",34840.0,682,High school or GED,0,4.4


In [150]:
# Local (Phoenix MSA counties) postings
def query_sf(sql: str) -> pd.DataFrame:
    cur = conn.cursor()
    cur.execute(sql)
    df = cur.fetch_pandas_all()
    cur.close()
    return df

# SQL for pulling Phoenix MSA Postings
sql = f"""
WITH phoenix_postings AS (
    SELECT
        SALARY,
        ID,
        SOC_5,
        SOC_5_NAME,
        COUNTY_NAME,
        POSTED,
        CASE
            WHEN MONTH(POSTED) <= {year_end_month} THEN YEAR(POSTED)
            ELSE YEAR(POSTED) + 1
        END AS YEAR 
    FROM EMSI_BURNING_GLASS_INSTITUTE.US.POSTINGS
    WHERE POSTED BETWEEN {min_date} AND {max_date}
      AND SOC_5 != '99-9999'
      AND COUNTY_NAME IN ('Maricopa, AZ', 'Pinal, AZ')
)
SELECT
    YEAR,
    COUNTY_NAME,
    SOC_5,
    SOC_5_NAME,
    SALARY,  -- Keep individual salaries, don't aggregate yet
    ID
FROM phoenix_postings
ORDER BY COUNTY_NAME, SOC_5, YEAR
"""

# Get the raw data
posting_data_local = query_sf(sql)

# Apply your county mapping 
posting_data_local['phoenix_msa'] = posting_data_local['COUNTY_NAME'].map(county_to_msa_snowflake)

# NOW aggregate
posting_data_local = posting_data_local.groupby(
    ['YEAR', 'phoenix_msa', 'SOC_5', 'SOC_5_NAME']
).agg({
    'SALARY': 'median',  # True median across all individual salaries
    'ID': 'nunique'      # Count distinct job postings
}).reset_index()

# Rename columns to match your expected output
posting_data_local = posting_data_local.rename(columns={
    'SALARY': 'MEDIAN_SALARY_LOCAL',
    'ID': 'POSTINGS_OBS_LOCAL'
})

# Calculate total postings by year and MSA for share calculation
total_postings_by_year = posting_data_local.groupby(['YEAR', 'phoenix_msa'])['POSTINGS_OBS_LOCAL'].sum().reset_index()
total_postings_by_year = total_postings_by_year.rename(columns={'POSTINGS_OBS_LOCAL': 'TOTAL_POSTINGS'})

# Merge to get total postings for each year/MSA
posting_data_local = posting_data_local.merge(
    total_postings_by_year, 
    on=['YEAR', 'phoenix_msa'], 
    how='left'
)

# Calculate share of jobs for each SOC5
posting_data_local['job_share'] = posting_data_local['POSTINGS_OBS_LOCAL'] / posting_data_local['TOTAL_POSTINGS']

# Sort and calculate growth rate of job shares (keeping variable name as obs_growth_local)
posting_data_local = posting_data_local.sort_values(['phoenix_msa', 'SOC_5', 'YEAR'])
posting_data_local['obs_growth_local'] = (
    posting_data_local.groupby(['phoenix_msa', 'SOC_5'])['job_share']
    .pct_change() * 100
)

# Keep only the final year and clean up
posting_data_local = posting_data_local[
    posting_data_local['YEAR'] == data_end_year
].drop(columns=['YEAR', 'TOTAL_POSTINGS', 'job_share']).reset_index(drop=True)


In [151]:
# Read the Arizona 10 year employment projections Excel file and process employment projections
df = pd.read_excel('OccupationalProjections.xlsx')

# Select needed columns and filter for employment measures
columns_needed = ['Occupation Group', 'SOC Code', 'Occupation Title', 'Measure Names', 'Measure Values']
employment_measures = ['Based Employment', 'Projected Employment']

df_employment = df[columns_needed][df['Measure Names'].isin(employment_measures)].copy()

# Pivot to get employment values as separate columns
df_pivot = df_employment.pivot_table(
    index=['Occupation Group', 'SOC Code', 'Occupation Title'],
    columns='Measure Names',
    values='Measure Values',
    aggfunc='first'
).reset_index()

df_pivot.columns.name = None

# Calculate local growth rate and clean up columns
df_pivot['obs_growth_local'] = np.where(
    (df_pivot['Based Employment'].notna()) & (df_pivot['Based Employment'] != 0),
    ((df_pivot['Projected Employment'] - df_pivot['Based Employment']) / df_pivot['Based Employment']) * 100,
    np.nan
)

local_projections = df_pivot.rename(columns={'SOC Code': 'SOC_5'})

# Merge the Excel projections with posting_data_local and replace growth rates
posting_data_local = posting_data_local.merge(
    local_projections[['SOC_5', 'obs_growth_local']],
    on='SOC_5',
    how='left',
    suffixes=('', '_excel')
)

# Replace the original obs_growth_local with the Excel-based one
posting_data_local['obs_growth_local'] = posting_data_local['obs_growth_local_excel']

# Drop the temporary column
posting_data_local = posting_data_local.drop('obs_growth_local_excel', axis=1)

In [152]:
posting_data_local

,phoenix_msa,SOC_5,SOC_5_NAME,MEDIAN_SALARY_LOCAL,POSTINGS_OBS_LOCAL,obs_growth_local
0,Phoenix–Mesa–Chandler,11-1011,Chief Executives,95000.0,819,11.248861
1,Phoenix–Mesa–Chandler,11-1021,General and Operations Managers,80000.0,6273,16.085043
2,Phoenix–Mesa–Chandler,11-2011,Advertising and Promotions Managers,77500.0,53,5.154639
3,Phoenix–Mesa–Chandler,11-2021,Marketing Managers,127500.0,3125,14.080354
4,Phoenix–Mesa–Chandler,11-2022,Sales Managers,100000.0,5410,13.165900
...,...,...,...,...,...,...
711,Phoenix–Mesa–Chandler,53-7072,"Pump Operators, Except Wellhead Pumpers",57200.0,78,NaN
712,Phoenix–Mesa–Chandler,53-7081,Refuse and Recyclable Material Collectors,52000.0,3,14.882122
713,Phoenix–Mesa–Chandler,53-7121,"Tank Car, Truck, and Ship Loaders",40300.0,245,15.068493
714,Phoenix–Mesa–Chandler,53-7199,"Material Moving Workers, All Other",NaN,3,16.037736


In [153]:
# Distribution analysis of growth rates
growth_stats = {
    'count': len(posting_data_local),
    'mean': posting_data_local['obs_growth_local'].mean(),
    'median': posting_data_local['obs_growth_local'].median(),
    'std': posting_data_local['obs_growth_local'].std(),
    'min': posting_data_local['obs_growth_local'].min(),
    'max': posting_data_local['obs_growth_local'].max(),
    'negative_pct': (posting_data_local['obs_growth_local'] < 0).mean() * 100
}

print("Growth Rate Distribution:")
for stat, value in growth_stats.items():
    print(f"{stat}: {value}")

# Create histogram or bins of growth rates
print("\nGrowth Rate Buckets:")
buckets = [-float('inf'), -50, -25, -10, 0, 10, 25, 50, float('inf')]
bucket_labels = ['< -50%', '-50% to -25%', '-25% to -10%', '-10% to 0%', 
                 '0% to 10%', '10% to 25%', '25% to 50%', '> 50%']
growth_bins = pd.cut(posting_data_local['obs_growth_local'], buckets, labels=bucket_labels)
bin_counts = growth_bins.value_counts().sort_index()

for label, count in bin_counts.items():
    print(f"{label}: {count} occupations ({count/len(posting_data_local)*100:.1f}%)")

Growth Rate Distribution:
count: 716
mean: 12.718620858283119
median: 12.965636731157955
std: 10.745187858054635
min: -30.76923076923077
max: 70.46028627561044
negative_pct: 8.798882681564246

Growth Rate Buckets:
< -50%: 0 occupations (0.0%)
-50% to -25%: 1 occupations (0.1%)
-25% to -10%: 13 occupations (1.8%)
-10% to 0%: 50 occupations (7.0%)
0% to 10%: 178 occupations (24.9%)
10% to 25%: 343 occupations (47.9%)
25% to 50%: 59 occupations (8.2%)
> 50%: 3 occupations (0.4%)


In [154]:
# Test query to examine column naming behavior
test_sql = """
SELECT 
    "SOC_2021_5",
    "a_median"
FROM OUTSIDE_DATA.OEWS.NATIONAL_DETAILED
LIMIT 5
"""

test_result = query_sf(test_sql)
print("Column names in direct select:")
print(test_result.columns.tolist())

# Now test with an aggregate function
test_agg_sql = """
SELECT 
    "SOC_2021_5",
    SUM("a_median") AS "a_median_sum"
FROM OUTSIDE_DATA.OEWS.NATIONAL_DETAILED
GROUP BY "SOC_2021_5"
LIMIT 5
"""

test_agg_result = query_sf(test_agg_sql)
print("Column names in aggregate query:")
print(test_agg_result.columns.tolist())

# Final attempt with our specific query structure
final_test_sql = f"""
SELECT 
    "SOC_2021_5" AS "SOC_5", 
    "year",
    SUM("a_median" * "tot_emp") / SUM("tot_emp") AS "weighted_median"
FROM OUTSIDE_DATA.OEWS.NATIONAL_DETAILED
WHERE "year" BETWEEN '{data_end_year-1}' AND '{data_end_year}'
GROUP BY "SOC_2021_5", "year"
LIMIT 5
"""

final_test_result = query_sf(final_test_sql)
print("Column names in our target query:")
print(final_test_result.columns.tolist())

Column names in direct select:
['SOC_2021_5', 'a_median']
Column names in aggregate query:
['SOC_2021_5', 'a_median_sum']
Column names in our target query:
['SOC_5', 'year', 'weighted_median']


In [155]:
# National wages from OEWS with a clear column naming
national_wages_sql = f"""
SELECT 
    "SOC_2021_5" AS "SOC_5", 
    "year",
    SUM("a_median" * "tot_emp") / SUM("tot_emp") AS "weighted_median",
    SUM("tot_emp") AS "total_employment"
FROM OUTSIDE_DATA.OEWS.NATIONAL_DETAILED
WHERE "year" BETWEEN '{data_end_year-1}' AND '{data_end_year}'
GROUP BY "SOC_2021_5", "year"
ORDER BY "SOC_2021_5", "year"
"""

national_wages = query_sf(national_wages_sql)

In [156]:
national_wages

,SOC_5,year,weighted_median,total_employment
0,11-1011,2023,206680.0,211230.0
1,11-1011,2024,206420.0,211850.0
2,11-1021,2023,101280.0,3507810.0
3,11-1021,2024,102950.0,3584420.0
4,11-1031,2023,47290.0,32460.0
...,...,...,...,...
1585,53-7081,2024,48350.0,139180.0
1586,53-7121,2023,58620.0,11400.0
1587,53-7121,2024,58070.0,10920.0
1588,53-7199,2023,40310.0,23970.0


In [157]:
# Calculate wage growth with the verified column names
national_wages = national_wages.sort_values(['SOC_5', 'year'])
national_wages['wage_growth_national'] = (
    national_wages.groupby('SOC_5')['weighted_median']
    .pct_change() * 100
)


/tmp/ipykernel_54907/3957238183.py:5: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change() * 100


In [158]:
national_wages

,SOC_5,year,weighted_median,total_employment,wage_growth_national
0,11-1011,2023,206680.0,211230.0,NaN
1,11-1011,2024,206420.0,211850.0,-0.125798
2,11-1021,2023,101280.0,3507810.0,NaN
3,11-1021,2024,102950.0,3584420.0,1.648894
4,11-1031,2023,47290.0,32460.0,NaN
...,...,...,...,...,...
1585,53-7081,2024,48350.0,139180.0,5.659965
1586,53-7121,2023,58620.0,11400.0,NaN
1587,53-7121,2024,58070.0,10920.0,-0.938246
1588,53-7199,2023,40310.0,23970.0,NaN


In [159]:
national_wages['year'] = national_wages['year'].astype(int)


In [160]:
# Filter to most recent year
national_wages = national_wages[
    national_wages['year'] == data_end_year
].drop(columns=['year']).reset_index(drop=True)

In [161]:
national_wages

,SOC_5,weighted_median,total_employment,wage_growth_national
0,11-1011,206420.0,211850.0,-0.125798
1,11-1021,102950.0,3584420.0,1.648894
2,11-1031,44810.0,26510.0,-5.244238
3,11-2011,126960.0,21100.0,-3.723364
4,11-2021,161030.0,384980.0,2.163431
...,...,...,...,...
790,53-7072,60020.0,12600.0,9.745840
791,53-7073,70010.0,17350.0,-2.533760
792,53-7081,48350.0,139180.0,5.659965
793,53-7121,58070.0,10920.0,-0.938246


In [162]:
# Rename columns to match what the R code would have produced
national_wages = national_wages.rename(columns={
    'weighted_median': 'a_median',
    'total_employment': 'tot_emp'
})

print("\nFinal national wages data:")
print(f"Shape: {national_wages.shape}")
print(national_wages.head())


Final national wages data:
Shape: (795, 4)
     SOC_5  a_median    tot_emp  wage_growth_national
0  11-1011  206420.0   211850.0             -0.125798
1  11-1021  102950.0  3584420.0              1.648894
2  11-1031   44810.0    26510.0             -5.244238
3  11-2011  126960.0    21100.0             -3.723364
4  11-2021  161030.0   384980.0              2.163431


In [163]:
national_wages

,SOC_5,a_median,tot_emp,wage_growth_national
0,11-1011,206420.0,211850.0,-0.125798
1,11-1021,102950.0,3584420.0,1.648894
2,11-1031,44810.0,26510.0,-5.244238
3,11-2011,126960.0,21100.0,-3.723364
4,11-2021,161030.0,384980.0,2.163431
...,...,...,...,...
790,53-7072,60020.0,12600.0,9.745840
791,53-7073,70010.0,17350.0,-2.533760
792,53-7081,48350.0,139180.0,5.659965
793,53-7121,58070.0,10920.0,-0.938246


In [164]:
# --------------------------------------------------------------
# National wage growth (ECI) — 5 year average annual growth rates
# --------------------------------------------------------------
eci_sql = f"""
SELECT 
    ECI_SOC5_ESTIMATED, 
    SOC_2019_5, 
    YEAR
FROM temporary_data.fsteemers.eci_soc5_estimated
WHERE YEAR BETWEEN 2018 AND 2022
ORDER BY SOC_2019_5, YEAR
"""

eci_raw_data = query_table_sf(
    conn,
    'temporary_data',
    'fsteemers',
    'eci_soc5_estimated',
    eci_sql,
    batch_size=10000
)

# 5 year Average of annual growth rates
def calculate_avg_growth_rate(group):
    if len(group) < 2:
        return pd.Series({'ECI_SOC5_ESTIMATED': None})
    
    # Calculate year-over-year growth rates
    growth_rates = group['ECI_SOC5_ESTIMATED'].pct_change().dropna()
    
    if len(growth_rates) == 0:
        return pd.Series({'ECI_SOC5_ESTIMATED': None})
    
    # Return average of annual growth rates
    return pd.Series({'ECI_SOC5_ESTIMATED': growth_rates.mean()})

# Final dataframe named eci_data with 2022 as the year
eci_data = (eci_raw_data
            .groupby('SOC_2019_5')
            .apply(calculate_avg_growth_rate)
            .reset_index()
            .assign(YEAR=2022))  # Add YEAR=2022 even though it's 5-year average

print(f"ECI 5-year average growth rate calculated for {len(eci_data)} occupations")

  • fetched 4135 rows so far
Successfully retrieved 4135 rows.
ECI 5-year average growth rate calculated for 827 occupations


/tmp/ipykernel_54907/1022600311.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_avg_growth_rate)


In [165]:
# Step 1: Merge ECI with SOC crosswalk
full_soc_list = bls_lct_soc_cw[['soc_2019_5', 'soc_2021_5', 'soc_2021_5_name']].drop_duplicates()

eci_full = full_soc_list.merge(
    eci_data,
    left_on='soc_2019_5',
    right_on='SOC_2019_5',
    how='left'
)


In [166]:
# Aggregate to SOC_2021_5 and impute SOC_2-level averages

eci_grouped = (
    eci_full
    .groupby(['YEAR', 'soc_2021_5', 'soc_2021_5_name'], as_index=False)
    .agg(ECI_SOC5_ESTIMATED=('ECI_SOC5_ESTIMATED', 'mean'))
)

eci_grouped['SOC_2021_2'] = eci_grouped['soc_2021_5'].str[:2]

eci_soc2 = (
    eci_grouped
    .groupby(['YEAR', 'SOC_2021_2'], as_index=False)
    .agg(ECI_SOC2_ESTIMATED=('ECI_SOC5_ESTIMATED', 'mean'))
)

eci_final = eci_grouped.merge(eci_soc2, on=['YEAR', 'SOC_2021_2'], how='left')
eci_final['ECI_SOC5_ESTIMATED'] = eci_final['ECI_SOC5_ESTIMATED'].fillna(eci_final['ECI_SOC2_ESTIMATED'])

# Final cleanup
eci_final = eci_final.rename(columns={
    'soc_2021_5': 'SOC_2021_5',
    'soc_2021_5_name': 'SOC_2021_5_NAME'
}).drop(columns=['SOC_2021_2', 'ECI_SOC2_ESTIMATED'])

national_wage_growth = eci_final.copy()

In [167]:
# Frank's ECI data base is missing some soc_2021_5 occupations, so manually add back in
# 4. Manually patch missing SOC_2021_5 codes from SOC-2 groups 11 and 19

soc2_avg = (
    national_wage_growth
    .assign(SOC_2021_2=lambda d: d['SOC_2021_5'].str[:2])
    .query("SOC_2021_2 in ['11', '19']")
    .groupby('SOC_2021_2', as_index=False)
    .agg(ECI_SOC5_ESTIMATED=('ECI_SOC5_ESTIMATED', 'mean'))
)

manual_patch = pd.DataFrame({
    'SOC_2021_5': [
        '11-2011', '11-2021', '11-2022', '11-2032', '11-2033',
        '19-5011', '19-5012'
    ],
    'SOC_2021_5_NAME': [
        'Advertising and Promotions Managers', 'Marketing Managers', 'Sales Managers',
        'Public Relations Managers', 'Fundraising Managers',
        'Occupational Health and Safety Specialists', 'Occupational Health and Safety Technicians'
    ]
})
manual_patch['SOC_2021_2'] = manual_patch['SOC_2021_5'].str[:2]
manual_patch = manual_patch.merge(soc2_avg, on='SOC_2021_2', how='left')
manual_patch['YEAR'] = 2022
manual_patch = manual_patch[['YEAR', 'SOC_2021_5', 'SOC_2021_5_NAME', 'ECI_SOC5_ESTIMATED']]

# Append to main table
national_wage_growth = pd.concat([national_wage_growth, manual_patch], ignore_index=True)


In [168]:
# Check if any SOC_2021_5 codes are still missing, SOC codes 45 and 55 are ok to be missing
all_soc_2021 = set(bls_lct_soc_cw['soc_2021_5'].dropna().unique())
retained_soc_2021 = set(national_wage_growth['SOC_2021_5'].dropna().unique())
dropped_soc_2021 = sorted(all_soc_2021 - retained_soc_2021)

if dropped_soc_2021:
    dropped_df = pd.DataFrame({'SOC_2021_5': dropped_soc_2021})
    dropped_df = dropped_df.merge(
        bls_lct_soc_cw[['soc_2021_5', 'soc_2021_5_name']].drop_duplicates(),
        left_on='SOC_2021_5',
        right_on='soc_2021_5',
        how='left'
    ).drop(columns='soc_2021_5')

    print("SOC_2021_5 codes dropped (not found in crosswalk or ECI data):")
    print(dropped_df.sort_values('SOC_2021_5').reset_index(drop=True))
else:
    print("All SOC_2021_5 codes retained after merge and imputation")

SOC_2021_5 codes dropped (not found in crosswalk or ECI data):
   SOC_2021_5                                    soc_2021_5_name
0     45-1011  First-Line Supervisors of Farming, Fishing, an...
1     45-2011                            Agricultural Inspectors
2     45-2021                                    Animal Breeders
3     45-2041         Graders and Sorters, Agricultural Products
4     45-2091                   Agricultural Equipment Operators
5     45-2092  Farmworkers and Laborers, Crop, Nursery, and G...
6     45-2093  Farmworkers, Farm, Ranch, and Aquacultural Ani...
7     45-2099                    Agricultural Workers, All Other
8     45-3031                        Fishing and Hunting Workers
9     45-4011                    Forest and Conservation Workers
10    45-4021                                            Fallers
11    45-4022                        Logging Equipment Operators
12    45-4023                            Log Graders and Scalers
13    45-4029              

In [169]:
national_wage_growth

,YEAR,SOC_2021_5,SOC_2021_5_NAME,ECI_SOC5_ESTIMATED
0,2022.0,11-1011,Chief Executives,0.102432
1,2022.0,11-1021,General and Operations Managers,0.100404
2,2022.0,11-3012,Administrative Services Managers,0.129413
3,2022.0,11-3013,Facilities Managers,0.129633
4,2022.0,11-3021,Computer and Information Systems Managers,0.129793
...,...,...,...,...
776,2022.0,11-2022,Sales Managers,0.167704
777,2022.0,11-2032,Public Relations Managers,0.167704
778,2022.0,11-2033,Fundraising Managers,0.167704
779,2022.0,19-5011,Occupational Health and Safety Specialists,0.259971


In [170]:
# USE ACS 5 year wage data to calculate wages for Phoenix

# File path to your gzipped ACS Phoenix data
file_path = "usa_00053.csv.gz"

# Load 
acs_data = pd.read_csv(file_path, compression='gzip')

# (Optional) quick check
print(acs_data.shape)
print(acs_data.columns.tolist())


(15912393, 21)
['YEAR', 'MULTYEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'STATEFIP', 'COUNTYICP', 'STRATA', 'GQ', 'PERNUM', 'PERWT', 'EMPSTAT', 'EMPSTATD', 'OCC', 'OCC2010', 'OCCSOC', 'WKSWORK1', 'UHRSWORK', 'INCWAGE']


In [171]:
# Filter for Phoenix only
acs_az = acs_data[acs_data['STATEFIP'] == 4].copy()
print(f"Arizona records: {len(acs_az)}")

# Filter for employed persons
acs_employed = acs_az[acs_az['EMPSTAT'] == 1].copy()
print(f"Employed persons: {len(acs_employed)}")

# Filter for valid wages
acs_wages = acs_employed[
    (acs_employed['INCWAGE'] > 0) & 
    (acs_employed['INCWAGE'] < 999999)
].copy()
print(f"With valid wages: {len(acs_wages)}")

# Filter for valid work time
acs_work_time = acs_wages[
    (acs_wages['WKSWORK1'] > 0) & 
    (acs_wages['UHRSWORK'] > 0)
].copy()
print(f"With valid work time: {len(acs_work_time)}")

# Filter for valid person weights
acs_final = acs_work_time[acs_work_time['PERWT'] > 0].copy()
print(f"With valid weights: {len(acs_final)}")

Arizona records: 347046
Employed persons: 149565
With valid wages: 141663
With valid work time: 141663
With valid weights: 141663


In [172]:
# Variable Creation

# Calculate annual hours worked
acs_final['annual_hours'] = acs_final['WKSWORK1'] * acs_final['UHRSWORK']
print(f"Annual hours calculated for {len(acs_final)} records")

# Calculate hourly wage
acs_final['hourly_wage'] = acs_final['INCWAGE'] / acs_final['annual_hours']
print(f"Hourly wages calculated")

# Check hourly wage distribution
print(f"Hourly wage stats:")
print(f"  Min: ${acs_final['hourly_wage'].min():.2f}")
print(f"  Max: ${acs_final['hourly_wage'].max():.2f}")
print(f"  Median: ${acs_final['hourly_wage'].median():.2f}")
print(f"  Mean: ${acs_final['hourly_wage'].mean():.2f}")


Annual hours calculated for 141663 records
Hourly wages calculated
Hourly wage stats:
  Min: $0.01
  Max: $22222.22
  Median: $23.08
  Mean: $36.55


In [173]:
# Convert SOC code to string if needed
acs_final['OCCSOC'] = acs_final['OCCSOC'].astype(str).str.zfill(6)

# Filter for SOC 15-1221 (Computer and Information Research Scientists)
research_sci = acs_final[acs_final['OCCSOC'].str.startswith('151221')].copy()
print(f"Records for SOC 15-1221: {len(research_sci)}")

# Check hourly wage stats for this occupation
if not research_sci.empty:
    print("Wage stats for Computer and Information Research Scientists (15-1221):")
    print(f"  Min: ${research_sci['hourly_wage'].min():.2f}")
    print(f"  Max: ${research_sci['hourly_wage'].max():.2f}")
    print(f"  Median: ${research_sci['hourly_wage'].median():.2f}")
    print(f"  Mean: ${research_sci['hourly_wage'].mean():.2f}")
else:
    print("No records found for SOC 15-1221.")


Records for SOC 15-1221: 14
Wage stats for Computer and Information Research Scientists (15-1221):
  Min: $11.72
  Max: $198.24
  Median: $53.42
  Mean: $56.83


In [174]:
# Define weighted median function
def weighted_median(values, weights):
    mask = ~(np.isnan(values) | np.isnan(weights))
    if not mask.any():
        return np.nan
    values, weights = values[mask], weights[mask]
    sorted_idx = np.argsort(values)
    values_sorted = values[sorted_idx]
    weights_sorted = weights[sorted_idx]
    cumsum = np.cumsum(weights_sorted)
    cutoff = cumsum[-1] / 2.0
    return values_sorted[np.searchsorted(cumsum, cutoff)]

# Make sure OCCSOC and PHOENIX_MSA columns exist
acs_final['OCCSOC'] = acs_final['OCCSOC'].astype(str).str.zfill(6)
acs_final['PHOENIX_MSA'] = 'Phoenix MSA'  # assign a constant label for grouping

# Clean dataframe to use
acs_clean = acs_final[['OCCSOC', 'PHOENIX_MSA', 'INCWAGE', 'hourly_wage', 'PERWT']].copy()

# Group by MSA and OCCSOC and calculate weighted stats
phoenix_wages_final = (
    acs_clean
    .groupby(['PHOENIX_MSA', 'OCCSOC'])
    .apply(lambda x: pd.Series({
        'a_median_local': weighted_median(x['INCWAGE'].values, x['PERWT'].values),
        'h_median_local': weighted_median(x['hourly_wage'].values, x['PERWT'].values),
        'tot_emp_local': x['PERWT'].sum(),
        'n_observations': len(x)
    }))
    .reset_index()
    .fillna({'tot_emp_local': 15})  # optional fallback for emp count
)

# Display summary
print(f"Final dataset shape: {phoenix_wages_final.shape}")
print(f"Unique MSAs: {phoenix_wages_final['PHOENIX_MSA'].nunique()}")
print(f"Unique occupations: {phoenix_wages_final['OCCSOC'].nunique()}")

# Example: display 15-1221 wages
print("\n15-1221 – Computer and Information Research Scientists:")
print(phoenix_wages_final[phoenix_wages_final['OCCSOC'] == '151221'])


Final dataset shape: (529, 6)
Unique MSAs: 1
Unique occupations: 529

15-1221 – Computer and Information Research Scientists:
    PHOENIX_MSA  OCCSOC  a_median_local  h_median_local  tot_emp_local  \
60  Phoenix MSA  151221         98919.0       47.557212          295.0   

    n_observations  
60            14.0  


/tmp/ipykernel_54907/67910198.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [175]:
acs_final

,YEAR,MULTYEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,COUNTYICP,STRATA,...,EMPSTATD,OCC,OCC2010,OCCSOC,WKSWORK1,UHRSWORK,INCWAGE,annual_hours,hourly_wage,PHOENIX_MSA
275839,2023,2019,202303,128242,2019010001356,18.0,2023001282423,4,130,10904,...,10,4055,4050,353023,44,10,2360,440,5.363636,Phoenix MSA
275842,2023,2019,202303,128245,2019010001402,2.0,2023001282453,4,250,250304,...,10,9645,5620,537065,52,22,2714,1144,2.372378,Phoenix MSA
275849,2023,2019,202303,128252,2019010001793,4.0,2023001282523,4,0,190404,...,10,2400,2400,254010,30,20,8495,600,14.158333,Phoenix MSA
275851,2023,2019,202303,128254,2019010001878,15.0,2023001282543,4,250,250204,...,10,4055,4050,353023,4,20,94,80,1.175000,Phoenix MSA
275855,2023,2019,202303,128258,2019010002117,20.0,2023001282583,4,130,10904,...,10,4640,4640,399041,10,15,8377,150,55.846667,Phoenix MSA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622851,2023,2023,202303,281575,2023001459754,11.0,2023002815753,4,130,12004,...,10,300,300,119041,52,50,170000,2600,65.384615,Phoenix MSA
622852,2023,2023,202303,281575,2023001459754,11.0,2023002815753,4,130,12004,...,10,530,530,131023,52,40,70000,2080,33.653846,Phoenix MSA
622858,2023,2023,202303,281577,2023001459815,18.0,2023002815773,4,0,190204,...,10,4010,4010,351012,52,30,32000,1560,20.512821,Phoenix MSA
622859,2023,2023,202303,281578,2023001459918,13.0,2023002815783,4,130,11004,...,10,2600,2600,271010,52,50,5000,2600,1.923077,Phoenix MSA


In [176]:
# Create county mapping

# Arizona counties mapping (from ACS Countyicp code)
arizona_counties = {
    130: "Maricopa", 
    210: "Pinal"
    # Add other Arizona counties here if needed:
    # 10: "Apache", 30: "Cochise", 50: "Coconino", 70: "Gila", 90: "Graham",
    # 110: "Greenlee", 150: "Mohave", 170: "Navajo", 190: "Pima", 230: "Santa Cruz",
    # 250: "Yavapai", 270: "Yuma", 290: "La Paz"
}

# Map county codes to county names
if 'COUNTYICP' in acs_final.columns:
    acs_final['COUNTY'] = acs_final['COUNTYICP'].map(arizona_counties)
    print(f"Using COUNTYICP for county mapping")
else:
    print("Warning: Could not find COUNTYICP column")

# Map counties to MSA using the ACS mapping
acs_final['PHOENIX_MSA'] = acs_final['COUNTY'].map(county_to_msa_acs)

# Check mapping success
print(f"Records with valid county mapping: {acs_final['COUNTY'].notna().sum()}")
print(f"Records with valid MSA mapping: {acs_final['PHOENIX_MSA'].notna().sum()}")

# Remove records that couldn't be mapped to MSA
acs_clean = acs_final.dropna(subset=['PHOENIX_MSA']).copy()


Using COUNTYICP for county mapping
Records with valid county mapping: 92013
Records with valid MSA mapping: 92013


In [177]:
# Calculate Phoenix MSA wages by OCCSOC
# --------------------------------------------------------------

def weighted_median(values, weights):
    """Calculate weighted median"""
    mask = ~(np.isnan(values) | np.isnan(weights))
    if not mask.any():
        return np.nan
    values, weights = values[mask], weights[mask]
    sorted_idx = np.argsort(values)
    cumsum = np.cumsum(weights[sorted_idx])
    return values[sorted_idx[np.searchsorted(cumsum, cumsum[-1] / 2)]]

print("Calculating ACS wages...")

# Group and calculate weighted statistics
phoenix_wages_final = (acs_clean
    .groupby(['PHOENIX_MSA', 'OCCSOC'])
    .apply(lambda x: pd.Series({
        'a_median_local': weighted_median(x['INCWAGE'].values, x['PERWT'].values),
        'h_median_local': weighted_median(x['hourly_wage'].values, x['PERWT'].values),
        'tot_emp_local': x['PERWT'].sum(),
        'n_observations': len(x)
    }))
    .reset_index()
    .fillna({'tot_emp_local': 15}))



Calculating ACS wages...


/tmp/ipykernel_54907/2038584186.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [178]:
phoenix_wages_final


,PHOENIX_MSA,OCCSOC,a_median_local,h_median_local,tot_emp_local,n_observations
0,Phoenix–Mesa–Chandler,111021,80000.0,35.396538,18409.0,840.0
1,Phoenix–Mesa–Chandler,1110XX,154194.0,67.638462,19816.0,1032.0
2,Phoenix–Mesa–Chandler,112011,76692.0,40.048077,521.0,33.0
3,Phoenix–Mesa–Chandler,112021,83300.0,39.511607,8314.0,401.0
4,Phoenix–Mesa–Chandler,112022,114538.0,48.076923,10273.0,493.0
...,...,...,...,...,...,...
520,Phoenix–Mesa–Chandler,5371XX,38936.0,18.719231,570.0,23.0
521,Phoenix–Mesa–Chandler,551010,100000.0,32.051282,210.0,6.0
522,Phoenix–Mesa–Chandler,552010,54024.0,22.893773,264.0,14.0
523,Phoenix–Mesa–Chandler,553010,32000.0,17.307692,330.0,23.0


In [179]:
# Ensure OCCSOC is string and properly formatted
phoenix_wages_final['OCCSOC'] = phoenix_wages_final['OCCSOC'].astype(str).str.zfill(6)

# Filter for Computer and Information Research Scientists
research_sci_row = phoenix_wages_final[phoenix_wages_final['OCCSOC'] == '151221']

# Display it
print(research_sci_row)


              PHOENIX_MSA  OCCSOC  a_median_local  h_median_local  \
60  Phoenix–Mesa–Chandler  151221          7315.0       11.722756   

    tot_emp_local  n_observations  
60          113.0             5.0  


In [180]:
# Load ACS OCCSOC to SOC5 Crosswalk and Aggregate to SOC-level

# Pull full crosswalk with valid mappings
crosswalk_sql = """
SELECT 
    *
FROM CROSSWALKS.CUSTOM.ONET_2019_ACS_SOC_CROSSWALK
WHERE ONET_2019 IS NOT NULL 
  AND SOC_2019_5_ACS IS NOT NULL
"""
acs_soc_xw_raw = query_sf(crosswalk_sql)

# Standardize column names
acs_soc_xw_raw.columns = acs_soc_xw_raw.columns.str.lower()

# Extract SOC 5-digit code from ONET
acs_soc_xw_raw['soc_2019_5'] = acs_soc_xw_raw['onet_2019'].str[:7]

# Keep all valid OCCSOC → SOC_5 mappings
acs_soc_xw = acs_soc_xw_raw[['soc_2019_5_acs', 'soc_2019_5']].drop_duplicates()

print("Merging Phoenix wages with crosswalk...")

# Ensure OCCSOC is padded
phoenix_wages_final['OCCSOC'] = phoenix_wages_final['OCCSOC'].astype(str).str.zfill(6)

# Merge in full crosswalk (many-to-one)
phoenix_wages_final_soc = phoenix_wages_final.merge(
    acs_soc_xw.rename(columns={'soc_2019_5': 'SOC_5'}),
    left_on='OCCSOC',
    right_on='soc_2019_5_acs',
    how='left'
).dropna(subset=['SOC_5'])




Merging Phoenix wages with crosswalk...


In [181]:
phoenix_wages_final_soc= (
    phoenix_wages_final_soc
    .groupby(['PHOENIX_MSA', 'SOC_5'])
    .apply(lambda x: pd.Series({
        'a_median_local': weighted_median(x['a_median_local'].values, x['tot_emp_local'].values),
        'h_median_local': weighted_median(x['h_median_local'].values, x['tot_emp_local'].values),
        'tot_emp_local': x['tot_emp_local'].sum(),
        'n_observations': x['n_observations'].sum()
    }))
    .reset_index()
)


/tmp/ipykernel_54907/4265600360.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [182]:
print(phoenix_wages_final_soc.columns)


Index(['PHOENIX_MSA', 'SOC_5', 'a_median_local', 'h_median_local',
       'tot_emp_local', 'n_observations'],
      dtype='object')


In [183]:
# Six-year wage estimates (Lightcast ONET mobility)


wage_mob_query = """
SELECT 
    "first_ONET"   AS SOC_5, 
    "first_ONET_NAME" AS SOC_5_NAME, 
    "mean_tot_earn"
FROM temporary_data.amartin.six_year_earnings_by_onet
WHERE REGEXP_LIKE(SUBSTRING("first_ONET", 8, 3), '\\.00')
"""
wage_mob = query_table_sf(
    conn,
    'temporary_data',
    'amartin',
    'six_year_earnings_by_onet',
    wage_mob_query,
    batch_size=10000
)
print("Successfully loaded six-year wage estimates")

# Truncate the ONET code to the first 7 characters
wage_mob['SOC_5'] = wage_mob['SOC_5'].str[:7]

  • fetched 805 rows so far
Successfully retrieved 805 rows.
Successfully loaded six-year wage estimates


In [184]:
# MIT Living Wage Data 

mit_file_path = "MIT_living_wage_clean_phoenix.xlsx"
mit_regional_wage = pd.read_excel(mit_file_path)

# Clean column names
mit_regional_wage.columns = mit_regional_wage.columns.str.lower().str.replace(' ', '_').str.replace('.', '_')

# Convert FY25 salary to numeric (remove $ and commas)
mit_regional_wage['fy25_annual_salary'] = pd.to_numeric(
    mit_regional_wage['fy25_annual_salary'].astype(str).str.replace('[$,]', '', regex=True), 
    errors='coerce'
)

# Group by MSA and calculate living wage metric (using FY25.Annual.Salary)
mit_msa_aggregated = (
    mit_regional_wage
    .groupby('msa', as_index=False)  # Group by MSA (lowercase after column cleaning)
    .agg(living_wage_metric=('fy25_annual_salary', lambda x: x.mean(skipna=True)))
    .rename(columns={'msa': 'MSA'})  # Rename to match your merge key
)

print("Living wage by MSA:")
print(mit_msa_aggregated.head())

Living wage by MSA:
                     MSA  living_wage_metric
0  Phoenix-Mesa-Chandler             53435.2


In [185]:
# MIT data is already at MSA level - no county mapping needed
# The MIT data already has MSA-level data, so we just need to clean it up
# and calculate the living wage metric
mit_msa_aggregated = mit_regional_wage.copy()

# Calculate living wage metric (using fy25_annual_salary directly)
mit_msa_aggregated['living_wage_metric'] = mit_msa_aggregated['fy25_annual_salary']

# Rename MSA column to match merge expectations and standardize the name
mit_msa_aggregated = mit_msa_aggregated.rename(columns={'msa': 'MSA'})

# Standardize the MSA name to match your other data
mit_msa_aggregated['MSA'] = mit_msa_aggregated['MSA'].str.replace('Phoenix-Mesa-Chandler', 'Phoenix–Mesa–Chandler')

# Keep only the columns we need
mit_msa_aggregated = mit_msa_aggregated[['MSA', 'living_wage_metric']]

print("\nFinal MIT data by MSA:")
print(mit_msa_aggregated)
print(f"\nLiving wage metric for Phoenix MSA: ${mit_msa_aggregated['living_wage_metric'].iloc[0]:,.2f}")


Final MIT data by MSA:
                     MSA  living_wage_metric
0  Phoenix–Mesa–Chandler             53435.2

Living wage metric for Phoenix MSA: $53,435.20


In [186]:
posting_data_national

,SOC_5,SOC_5_NAME,MEDIAN_SALARY_NATIONAL,POSTING_OBS_NATIONAL,EDUCATION_LEVEL,EDUCATION_LEVEL_CODE,OBS_GROWTH_NATIONAL
0,11-1011,Chief Executives,98000.0,64067,Bachelor's degree,2,5.5
1,11-1021,General and Operations Managers,80000.0,370084,Bachelor's degree,2,5.8
2,11-2011,Advertising and Promotions Managers,104582.0,3794,Bachelor's degree,2,-2.6
3,11-2021,Marketing Managers,137500.0,210604,Bachelor's degree,2,8.2
4,11-2022,Sales Managers,100352.0,290916,Bachelor's degree,2,5.9
...,...,...,...,...,...,...,...
761,53-7073,Wellhead Pumpers,55120.0,72,High school or GED,0,-2.0
762,53-7081,Refuse and Recyclable Material Collectors,41600.0,455,High school or GED,0,2.3
763,53-7121,"Tank Car, Truck, and Ship Loaders",40404.0,10644,High school or GED,0,3.9
764,53-7199,"Material Moving Workers, All Other",34840.0,682,High school or GED,0,4.4


In [187]:
# Calculate overall median growth as a single value (matching R code)
median_growth_overall = posting_data_national['OBS_GROWTH_NATIONAL'].median(skipna=True)

# Calculate local median growth by MSA (matching R code)
local_postings_medians = posting_data_local.groupby('phoenix_msa').agg(  
    local_median_growth=('obs_growth_local', 'median')
).reset_index()

print(f"Overall median growth: {median_growth_overall}")
print("Local postings medians sample:")
print(local_postings_medians.head())

Overall median growth: 3.8
Local postings medians sample:
             phoenix_msa  local_median_growth
0  Phoenix–Mesa–Chandler            12.965637


In [188]:
local_postings_medians

,phoenix_msa,local_median_growth
0,Phoenix–Mesa–Chandler,12.965637


In [189]:
# Merge datasets together

# Start with posting_data_local and merge step by step like R code
phoenix_lmi = posting_data_local.merge(
    local_postings_medians, 
    on='phoenix_msa',
    how='left'
)

# Merge with national posting data
phoenix_lmi = phoenix_lmi.merge(
    posting_data_national, 
    on=['SOC_5', 'SOC_5_NAME'], 
    how='left'
)

# Merge with ACS Phoenix wages 
phoenix_lmi = phoenix_lmi.merge(
    phoenix_wages_final_soc.drop(columns=['PHOENIX_MSA']), 
    on='SOC_5',
    how='left'
)


# Merge with six-year wage estimates
phoenix_lmi = phoenix_lmi.merge(
    wage_mob, 
    on=['SOC_5', 'SOC_5_NAME'], 
    how='left'
)

# Drop pre-existing ECI columns to avoid *x/*y issues
phoenix_lmi = phoenix_lmi.drop(columns=[col for col in phoenix_lmi.columns if col.startswith('ECI_SOC5_ESTIMATED')], errors='ignore')

# Merge with national wage growth (ECI)
phoenix_lmi = phoenix_lmi.merge(
    national_wage_growth.drop(columns=['YEAR']).rename(columns={
        'SOC_2021_5': 'SOC_5', 
        'SOC_2021_5_NAME': 'SOC_5_NAME'
    }), 
    on=['SOC_5', 'SOC_5_NAME'], 
    how='left'
)

# Merge with national wages data
phoenix_lmi = phoenix_lmi.merge(
    national_wages, 
    on='SOC_5', 
    how='left'
)

# Merge with MIT living wage data (MSA-level merge)
phoenix_lmi = phoenix_lmi.merge(
    mit_msa_aggregated, 
    left_on='phoenix_msa',
    right_on='MSA',
    how='left'
)

# Sort by MSA and local growth/postings (matching R arrange)
phoenix_lmi = phoenix_lmi.sort_values(['phoenix_msa', 'obs_growth_local', 'POSTINGS_OBS_LOCAL'], 
                                    ascending=[True, False, False])

# Create calculated columns (matching R mutate operations)
print("Creating calculated columns...")
# Convert growth columns to float to avoid decimal/float type issues
phoenix_lmi['OBS_GROWTH_NATIONAL'] = pd.to_numeric(phoenix_lmi['OBS_GROWTH_NATIONAL'], errors='coerce')
phoenix_lmi['obs_growth_local'] = pd.to_numeric(phoenix_lmi['obs_growth_local'], errors='coerce')
phoenix_lmi['local_median_growth'] = pd.to_numeric(phoenix_lmi['local_median_growth'], errors='coerce')

# Also convert median_growth_overall to float
median_growth_overall = float(median_growth_overall)

# # Salary hierarchy (coalesce equivalent) - using correct column names
# phoenix_lmi['salary_use'] = phoenix_lmi['a_median_local'].fillna(
#     phoenix_lmi['a_median'].fillna(
#         phoenix_lmi['MEDIAN_SALARY_NATIONAL'].fillna(
#             phoenix_lmi['MEDIAN_SALARY_LOCAL']
#         )
#     )
# )

# New Salary hierarchy as suggest by Alex Martin prioritizing local salary (uncomment out if needed)
phoenix_lmi['salary_use'] = phoenix_lmi['a_median_local'].fillna(
    phoenix_lmi['MEDIAN_SALARY_LOCAL'].fillna(
        phoenix_lmi['a_median'].fillna(
            phoenix_lmi['MEDIAN_SALARY_NATIONAL']
        )
    )
)

# Mean total earnings with fallback
phoenix_lmi['mean_tot_earn'] = phoenix_lmi['mean_tot_earn'].fillna(6 * phoenix_lmi['salary_use'])

# Percentage wage compared to local/MSA median (renamed from pct_wage_msa)
phoenix_lmi['pct_wage_msa'] = 100 * phoenix_lmi['salary_use'] / phoenix_lmi['MEDIAN_SALARY_LOCAL']

# # Normalized growth calculations (using the scalar median_growth_overall)
# phoenix_lmi['growth_normalized'] = 100 * (
#     (100 + phoenix_lmi['OBS_GROWTH_NATIONAL']) / (100 + median_growth_overall) - 1
# )
# phoenix_lmi['local_growth_normalized'] = 100 * (
#     (100 + phoenix_lmi['obs_growth_local']) / (100 + phoenix_lmi['local_median_growth']) - 1
# )



Creating calculated columns...


In [190]:
# Add BLS education data to phoenix_lmi 
phoenix_lmi = phoenix_lmi.merge(
    bls_benchmark[['soc_2021_5', 'edulevel_name']],
    left_on='SOC_5',
    right_on='soc_2021_5',
    how='left'
).drop('soc_2021_5', axis=1)

In [191]:
# Get LLM exposure data
ai_exposure_sql = """
SELECT 
   SOC6,
   LLM_EXPOSURE
FROM OUTSIDE_DATA.FELTEN_AI.OCCUPATION_EXPOSURE
WHERE SOC6 IS NOT NULL
"""
ai_exposure = query_sf(ai_exposure_sql)

# Merge LLM exposure with phoenix_lmi
phoenix_lmi = phoenix_lmi.merge(
    ai_exposure[['SOC6', 'LLM_EXPOSURE']], 
    left_on='SOC_5',
    right_on='SOC6', 
    how='left'
).drop(columns=['SOC6'])

In [192]:
acs_55 = pd.read_csv("Phoenix_ACS.csv", dtype={'soc_2019_5': str})

In [193]:
# Prepare ACS 55+ data for merging
acs_55_clean = acs_55.rename(columns={'soc_2019_5': 'SOC_5'}).copy()

# Merge ACS 55+ data with phoenix_lmi
phoenix_lmi = phoenix_lmi.merge(
    acs_55_clean[['SOC_5', 'tot_emp_55_plus', 'share_55_plus', 'few_obs_flag']], 
    left_on='SOC_5', 
    right_on='SOC_5', 
    how='left'
)

In [194]:
# Import BLS Shares of education within occupations
bls_edu_shares = pd.read_excel('education.xlsx', 
                   sheet_name='Table 5.3', 
                   skiprows=1)  # Skip the first row


In [195]:
bls_edu_shares

,2023 National Employment Matrix title,2023 National Employment Matrix code,Less than high school diploma,High school diploma or equivalent,"Some college, no degree",Associate's degree,Bachelor's degree,Master's degree,Doctoral or professional degree
0,"Total, all occupations",00-0000,7.4,22.6,18.9,9.5,25.1,11.7,4.8
1,Chief executives[1],11-1011,1.6,7.9,13.7,5.2,40.0,24.7,6.9
2,General and operations managers,11-1021,3.2,17.5,23.5,9.9,32.7,11.5,1.6
3,Legislators[1],11-1031,1.6,7.9,13.7,5.2,40.0,24.7,6.9
4,Advertising and promotions managers,11-2011,1.0,4.2,7.8,5.2,65.2,15.2,1.4
...,...,...,...,...,...,...,...,...,...
832,"Material moving workers, all other[1]",53-7199,14.4,51.5,19.8,6.4,6.6,1.0,0.4
833,Footnotes:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
834,[1] This occupation is not an exact match with...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
835,Data Source: 2021 and 2022 American Community ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
# Remove title column and any footer rows from BLS data
bls_edu_shares_clean = bls_edu_shares.copy()

# Remove any footer/footnote rows (usually the last few rows with NaN values)
bls_edu_shares_clean = bls_edu_shares_clean.dropna(subset=['2023 National Employment Matrix code'])

# Drop the title column since we're merging on SOC code
bls_edu_shares_no_title = bls_edu_shares_clean.drop(columns=['2023 National Employment Matrix title'])

# Merge BLS education shares with phoenix_lmi 
phoenix_lmi = phoenix_lmi.merge(
    bls_edu_shares_no_title,
    left_on='SOC_5',
    right_on='2023 National Employment Matrix code',
    how='left'
).drop(columns=['2023 National Employment Matrix code'])


In [197]:
# Group education columns in phoenix_lmi 

phoenix_lmi['Less_than_HS'] = phoenix_lmi['Less than high school diploma']
phoenix_lmi['HS_Some_College'] = (
    phoenix_lmi['High school diploma or equivalent'] + 
    phoenix_lmi['Some college, no degree'] + 
    phoenix_lmi["Associate's degree"]
)
phoenix_lmi['BA'] = phoenix_lmi["Bachelor's degree"]
phoenix_lmi['BA_Plus'] = (
    phoenix_lmi["Master's degree"] + 
    phoenix_lmi['Doctoral or professional degree']
)

# Remove original BLS education columns from phoenix_lmi
original_education_columns = [
   'Less than high school diploma',
   'High school diploma or equivalent', 
   'Some college, no degree',
   "Associate's degree",
   "Bachelor's degree",
   "Master's degree",
   'Doctoral or professional degree'
]
phoenix_lmi = phoenix_lmi.drop(columns=original_education_columns)


In [250]:
phoenix_lmi

,phoenix_msa,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,POSTINGS_OBS_LOCAL,...,tot_emp_55_plus,share_55_plus,few_obs_flag,Less_than_HS,HS_Some_College,BA,BA_Plus,OBS_GROWTH_NATIONAL_rank,obs_growth_local_rank,edulevel_name_y
0,Phoenix–Mesa–Chandler,29-1171,Nurse Practitioners,111425.0,a_median_local,111425.0,130114.5,129210.0,135000.0,2790,...,829.0,0.217016,NaN,0.4,2.0,8.4,89.2,3.0,1.0,Master's degree
1,Phoenix–Mesa–Chandler,47-2231,Solar Photovoltaic Installers,48000.0,a_median_local,48000.0,61360.0,51860.0,56614.0,212,...,NaN,NaN,NaN,10.6,74.0,12.7,2.7,2.0,2.0,High school or GED
2,Phoenix–Mesa–Chandler,29-1071,Physician Assistants,117988.0,a_median_local,117988.0,125103.0,133260.0,135200.0,759,...,312.0,0.110403,NaN,0.8,6.2,19.1,73.8,6.0,3.0,Master's degree
3,Phoenix–Mesa–Chandler,11-9111,Medical and Health Services Managers,80176.0,a_median_local,80176.0,80000.0,117960.0,90000.0,8282,...,2780.0,0.251061,NaN,1.2,33.5,33.8,31.6,6.0,4.0,Bachelor's degree
4,Phoenix–Mesa–Chandler,31-2021,Physical Therapist Assistants,25000.0,a_median_local,25000.0,43867.0,65510.0,65000.0,985,...,168.0,0.097788,NaN,0.5,62.7,30.9,5.9,9.0,5.0,Associate degree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
660,Phoenix–Mesa–Chandler,47-2043,Floor Sanders and Finishers,40000.0,a_median_local,40000.0,41600.0,49150.0,41600.0,1,...,697.0,0.191431,NaN,35.7,58.8,4.7,0.8,390.0,NaN,High school or GED
661,Phoenix–Mesa–Chandler,47-2072,Pile Driver Operators,49555.0,a_median_local,49555.0,NaN,70510.0,57200.0,1,...,907.0,0.200486,NaN,19.8,76.8,2.8,0.6,271.0,NaN,High school or GED
662,Phoenix–Mesa–Chandler,49-9061,Camera and Photographic Equipment Repairers,61354.0,a_median_local,61354.0,34834.0,49300.0,50440.0,1,...,325.0,0.494673,NaN,3.4,74.2,17.9,4.5,643.0,NaN,High school or GED
663,Phoenix–Mesa–Chandler,51-7032,"Patternmakers, Wood",28000.0,a_median_local,28000.0,85000.0,52520.0,57200.0,1,...,7.0,0.034483,Low obs count,13.7,66.7,16.5,3.1,632.0,NaN,High school or GED


In [198]:
# # Filter out specific occupation (matching R filter)
# phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Obstetricians and Gynecologists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Dancer"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Accountants and Auditors"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Mining and Geological Engineers, Including Mining Safety Engineers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Loan Officers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Rotary Drill Operators, Oil and Gas"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Helpers--Extraction Workers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Funeral Home Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Property, Real Estate, and Community Association Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Funeral Attendants"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Pharmacists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Education Administrators, Postsecondary"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Chief Executives"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Human Resources Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Personal Financial Advisors"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Managers, All Other"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Human Resources Specialists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Compensation and Benefits Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Training and Development Specialists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Training and Development Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Fundraising Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Compensation, Benefits, and Job Analysis Specialists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Fundraisers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Probation Officers and Correctional Treatment Specialists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "First-Line Supervisors of Entertainment and Recreation Workers, Except Gambling Services"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Securities, Commodities, and Financial Services Sales Agents"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Insurance Sales Agents"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "First-Line Supervisors of Security Workers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Firefighters"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Financial Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Sales Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Marketing Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Management Analysts"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Financial and Investment Analysts"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Producers and Directors"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Public Relations Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Detectives and Criminal Investigators"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Public Relations Specialists"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Entertainment and Recreation Managers, Except Gambling"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Writers and Authors"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Tax Examiners and Collectors, and Revenue Agents"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Set and Exhibit Designers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Credit Counselors"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Sales Representatives, Wholesale and Manufacturing, Except Technical and Scientific Products"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Actuaries"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Compliance Officers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Budget Analysts"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Lodging Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Police and Sheriff's Patrol Officers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "First-Line Supervisors of Police and Detectives"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Gambling Managers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "First-Line Supervisors of Gambling Services Workers"]
phoenix_lmi = phoenix_lmi[phoenix_lmi['SOC_5_NAME'] != "Gambling Service Workers, All Other"]

In [199]:
target_soc = "15-1221"

row = phoenix_lmi[phoenix_lmi['SOC_5'] == target_soc]
row

,phoenix_msa,SOC_5,SOC_5_NAME,MEDIAN_SALARY_LOCAL,POSTINGS_OBS_LOCAL,obs_growth_local,local_median_growth,MEDIAN_SALARY_NATIONAL,POSTING_OBS_NATIONAL,EDUCATION_LEVEL,...,pct_wage_msa,edulevel_name,LLM_EXPOSURE,tot_emp_55_plus,share_55_plus,few_obs_flag,Less_than_HS,HS_Some_College,BA,BA_Plus
17,Phoenix–Mesa–Chandler,15-1221,Computer and Information Research Scientists,146700.0,673,32.831325,12.965637,169300.0,47436,Bachelor's degree,...,4.986367,Master's degree,0.968235,NaN,NaN,NaN,0.3,4.4,40.3,55.0


In [200]:
def add_salary_source(df):
    """Add salary_use_source column to track which wage source was used"""
    # Ensure numeric types
    salary_cols = ['a_median_local', 'MEDIAN_SALARY_LOCAL', 'a_median', 'MEDIAN_SALARY_NATIONAL']
    for col in salary_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Initialize salary_use_source column
    df['salary_use_source'] = np.nan
    
    # Determine source based on Alex Martin salary hierarchy
    # a_median_local → MEDIAN_SALARY_LOCAL → a_median → MEDIAN_SALARY_NATIONAL
    
    # Check a_median_local first
    mask = df['a_median_local'].notna() & (df['salary_use'] == df['a_median_local'])
    df.loc[mask, 'salary_use_source'] = 'a_median_local'
    
    # Check MEDIAN_SALARY_LOCAL second  
    mask = (df['salary_use_source'].isna() & 
            df['MEDIAN_SALARY_LOCAL'].notna() & 
            (df['salary_use'] == df['MEDIAN_SALARY_LOCAL']))
    df.loc[mask, 'salary_use_source'] = 'MEDIAN_SALARY_LOCAL'
    
    # Check a_median third
    mask = (df['salary_use_source'].isna() & 
            df['a_median'].notna() & 
            (df['salary_use'] == df['a_median']))
    df.loc[mask, 'salary_use_source'] = 'a_median'
    
    # Check MEDIAN_SALARY_NATIONAL last
    mask = (df['salary_use_source'].isna() & 
            df['MEDIAN_SALARY_NATIONAL'].notna() & 
            (df['salary_use'] == df['MEDIAN_SALARY_NATIONAL']))
    df.loc[mask, 'salary_use_source'] = 'MEDIAN_SALARY_NATIONAL'
    
    return df

# Apply to add salary source tracking without changing salary_use
phoenix_lmi = add_salary_source(phoenix_lmi)

# Reorder salary columns to follow SOC_5_NAME
salary_columns = [
    'salary_use', 'salary_use_source', 
    'a_median_local', 'MEDIAN_SALARY_LOCAL', 
    'a_median', 'MEDIAN_SALARY_NATIONAL'
]

# Reorder columns in phoenix_lmi
cols = phoenix_lmi.columns.tolist()
if 'SOC_5_NAME' in cols:
    idx = cols.index('SOC_5_NAME') + 1
    for col in salary_columns:
        if col in cols:
            cols.remove(col)
    for i, col in enumerate(salary_columns):
        if col in phoenix_lmi.columns:
            cols.insert(idx + i, col)
    phoenix_lmi = phoenix_lmi[cols]

print("Added salary_use_source column without changing salary_use values")
print("Salary source distribution:")
print(phoenix_lmi['salary_use_source'].value_counts())

Added salary_use_source column without changing salary_use values
Salary source distribution:
salary_use_source
a_median_local         642
MEDIAN_SALARY_LOCAL     22
a_median                 1
Name: count, dtype: int64


/tmp/ipykernel_54907/3154095480.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'a_median_local' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'salary_use_source'] = 'a_median_local'


In [201]:
phoenix_lmi

,phoenix_msa,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,POSTINGS_OBS_LOCAL,...,pct_wage_msa,edulevel_name,LLM_EXPOSURE,tot_emp_55_plus,share_55_plus,few_obs_flag,Less_than_HS,HS_Some_College,BA,BA_Plus
0,Phoenix–Mesa–Chandler,29-1171,Nurse Practitioners,111425.0,a_median_local,111425.0,130114.5,129210.0,135000.0,2790,...,85.636113,Master's degree,0.301660,829.0,0.217016,NaN,0.4,2.0,8.4,89.2
1,Phoenix–Mesa–Chandler,47-2231,Solar Photovoltaic Installers,48000.0,a_median_local,48000.0,61360.0,51860.0,56614.0,212,...,78.226858,High school or GED,-1.036982,NaN,NaN,NaN,10.6,74.0,12.7,2.7
2,Phoenix–Mesa–Chandler,29-1071,Physician Assistants,117988.0,a_median_local,117988.0,125103.0,133260.0,135200.0,759,...,94.312686,Master's degree,0.040241,312.0,0.110403,NaN,0.8,6.2,19.1,73.8
3,Phoenix–Mesa–Chandler,11-9111,Medical and Health Services Managers,80176.0,a_median_local,80176.0,80000.0,117960.0,90000.0,8282,...,100.220000,Bachelor's degree,1.314992,2780.0,0.251061,NaN,1.2,33.5,33.8,31.6
4,Phoenix–Mesa–Chandler,31-2021,Physical Therapist Assistants,25000.0,a_median_local,25000.0,43867.0,65510.0,65000.0,985,...,56.990448,Associate degree,-0.247278,168.0,0.097788,NaN,0.5,62.7,30.9,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,Phoenix–Mesa–Chandler,47-2043,Floor Sanders and Finishers,40000.0,a_median_local,40000.0,41600.0,49150.0,41600.0,1,...,96.153846,High school or GED,-1.574358,697.0,0.191431,NaN,35.7,58.8,4.7,0.8
712,Phoenix–Mesa–Chandler,47-2072,Pile Driver Operators,49555.0,a_median_local,49555.0,NaN,70510.0,57200.0,1,...,NaN,High school or GED,-1.454892,907.0,0.200486,NaN,19.8,76.8,2.8,0.6
713,Phoenix–Mesa–Chandler,49-9061,Camera and Photographic Equipment Repairers,61354.0,a_median_local,61354.0,34834.0,49300.0,50440.0,1,...,176.132514,High school or GED,-0.359770,325.0,0.494673,NaN,3.4,74.2,17.9,4.5
714,Phoenix–Mesa–Chandler,51-7032,"Patternmakers, Wood",28000.0,a_median_local,28000.0,85000.0,52520.0,57200.0,1,...,32.941176,High school or GED,-1.208153,7.0,0.034483,Low obs count,13.7,66.7,16.5,3.1


In [202]:
print(f"Final merged dataset shape: {phoenix_lmi.shape}")
print(f"Columns: {phoenix_lmi.columns.tolist()}")

# Save the compiled dataset 
print("Saving compiled dataset...")

# Compose the filename
filename = f'occ-data-compiled-{today_str}-update.xlsx'

# Save to Excel
phoenix_lmi.to_excel(filename, index=False)
print(f"Dataset saved successfully as: {filename}")

print("Dataset saved successfully!")

Final merged dataset shape: (665, 35)
Columns: ['phoenix_msa', 'SOC_5', 'SOC_5_NAME', 'salary_use', 'salary_use_source', 'a_median_local', 'MEDIAN_SALARY_LOCAL', 'a_median', 'MEDIAN_SALARY_NATIONAL', 'POSTINGS_OBS_LOCAL', 'obs_growth_local', 'local_median_growth', 'POSTING_OBS_NATIONAL', 'EDUCATION_LEVEL', 'EDUCATION_LEVEL_CODE', 'OBS_GROWTH_NATIONAL', 'h_median_local', 'tot_emp_local', 'n_observations', 'mean_tot_earn', 'ECI_SOC5_ESTIMATED', 'tot_emp', 'wage_growth_national', 'MSA', 'living_wage_metric', 'pct_wage_msa', 'edulevel_name', 'LLM_EXPOSURE', 'tot_emp_55_plus', 'share_55_plus', 'few_obs_flag', 'Less_than_HS', 'HS_Some_College', 'BA', 'BA_Plus']
Saving compiled dataset...
Dataset saved successfully as: occ-data-compiled-2025.08.07-update.xlsx
Dataset saved successfully!


In [203]:
phoenix_lmi

,phoenix_msa,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,POSTINGS_OBS_LOCAL,...,pct_wage_msa,edulevel_name,LLM_EXPOSURE,tot_emp_55_plus,share_55_plus,few_obs_flag,Less_than_HS,HS_Some_College,BA,BA_Plus
0,Phoenix–Mesa–Chandler,29-1171,Nurse Practitioners,111425.0,a_median_local,111425.0,130114.5,129210.0,135000.0,2790,...,85.636113,Master's degree,0.301660,829.0,0.217016,NaN,0.4,2.0,8.4,89.2
1,Phoenix–Mesa–Chandler,47-2231,Solar Photovoltaic Installers,48000.0,a_median_local,48000.0,61360.0,51860.0,56614.0,212,...,78.226858,High school or GED,-1.036982,NaN,NaN,NaN,10.6,74.0,12.7,2.7
2,Phoenix–Mesa–Chandler,29-1071,Physician Assistants,117988.0,a_median_local,117988.0,125103.0,133260.0,135200.0,759,...,94.312686,Master's degree,0.040241,312.0,0.110403,NaN,0.8,6.2,19.1,73.8
3,Phoenix–Mesa–Chandler,11-9111,Medical and Health Services Managers,80176.0,a_median_local,80176.0,80000.0,117960.0,90000.0,8282,...,100.220000,Bachelor's degree,1.314992,2780.0,0.251061,NaN,1.2,33.5,33.8,31.6
4,Phoenix–Mesa–Chandler,31-2021,Physical Therapist Assistants,25000.0,a_median_local,25000.0,43867.0,65510.0,65000.0,985,...,56.990448,Associate degree,-0.247278,168.0,0.097788,NaN,0.5,62.7,30.9,5.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,Phoenix–Mesa–Chandler,47-2043,Floor Sanders and Finishers,40000.0,a_median_local,40000.0,41600.0,49150.0,41600.0,1,...,96.153846,High school or GED,-1.574358,697.0,0.191431,NaN,35.7,58.8,4.7,0.8
712,Phoenix–Mesa–Chandler,47-2072,Pile Driver Operators,49555.0,a_median_local,49555.0,NaN,70510.0,57200.0,1,...,NaN,High school or GED,-1.454892,907.0,0.200486,NaN,19.8,76.8,2.8,0.6
713,Phoenix–Mesa–Chandler,49-9061,Camera and Photographic Equipment Repairers,61354.0,a_median_local,61354.0,34834.0,49300.0,50440.0,1,...,176.132514,High school or GED,-0.359770,325.0,0.494673,NaN,3.4,74.2,17.9,4.5
714,Phoenix–Mesa–Chandler,51-7032,"Patternmakers, Wood",28000.0,a_median_local,28000.0,85000.0,52520.0,57200.0,1,...,32.941176,High school or GED,-1.208153,7.0,0.034483,Low obs count,13.7,66.7,16.5,3.1


In [204]:
# Create ranking columns for raw growth variables
phoenix_lmi['OBS_GROWTH_NATIONAL_rank'] = phoenix_lmi.groupby('phoenix_msa')['OBS_GROWTH_NATIONAL'].rank(method='min', ascending=False)
phoenix_lmi['obs_growth_local_rank'] = phoenix_lmi.groupby('phoenix_msa')['obs_growth_local'].rank(method='min', ascending=False)

In [205]:
# Define JTM identification function 
def identify_jtm(df, educ, metrics, cols):
    """Identify jobs that matter based on criteria"""
    filtered_df = df[
        (df['EDUCATION_LEVEL'].isin(educ)) &  
        (df['POSTINGS_OBS_LOCAL'] >= 30) &
        (df['POSTING_OBS_NATIONAL'] >= 30) &  
        (df['OBS_GROWTH_NATIONAL'] > 0) &  # 10 year growth just needs to be positive
        (df['obs_growth_local'] > 0) &  # 10 year Arizona state employment projections needs to be positive
        (df['ECI_SOC5_ESTIMATED'] > 0) &  # Changed from -5 to 0 (positive wage growth required)
        (df['salary_use'] >= df['living_wage_metric'])
    ].copy()
    
    # Create rank columns for each metric (global ranking like R)
    for metric in metrics:
        if metric in filtered_df.columns:
            filtered_df[f'{metric}_rank'] = filtered_df[metric].rank(ascending=False, method='min')
    
    return filtered_df

# Define sub-BA education levels
sub_ba = ['High school or GED', 'Associate degree', 'No Education Listed']

# Updated metrics to match your actual column names
metrics = ['POSTING_OBS_NATIONAL', 'growth_normalized', 'POSTINGS_OBS_LOCAL',  # Fixed: uppercase for posting columns
           'local_growth_normalized', 'ECI_SOC5_ESTIMATED', 'salary_use', 'mean_tot_earn']

occ_cols = ['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL']  # Fixed: uppercase

# Identify sub-BA JTMs
sub_ba_jtm = identify_jtm(phoenix_lmi, sub_ba, metrics, occ_cols)

# # Filter out specific occupations (matching R code exactly)
# sub_ba_jtm = sub_ba_jtm[sub_ba_jtm['SOC_5'] != "29-1071"]  # PA exclusion only



In [206]:
sub_ba_jtm

,phoenix_msa,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,POSTINGS_OBS_LOCAL,...,HS_Some_College,BA,BA_Plus,OBS_GROWTH_NATIONAL_rank,obs_growth_local_rank,POSTING_OBS_NATIONAL_rank,POSTINGS_OBS_LOCAL_rank,ECI_SOC5_ESTIMATED_rank,salary_use_rank,mean_tot_earn_rank
10,Phoenix–Mesa–Chandler,29-2032,Diagnostic Medical Sonographers,75000.0,a_median_local,75000.0,109200.0,89340.0,114400.0,481,...,48.9,39.0,11.0,32.0,11.0,12.0,23.0,31.0,8.0,5.0
14,Phoenix–Mesa–Chandler,49-9062,Medical Equipment Repairers,61354.0,a_median_local,61354.0,60000.0,62630.0,62920.0,143,...,74.2,17.9,4.5,19.0,15.0,31.0,32.0,18.0,20.0,15.0
23,Phoenix–Mesa–Chandler,49-9041,Industrial Machinery Mechanics,60000.0,a_median_local,60000.0,55120.0,63760.0,58853.5,431,...,83.4,6.6,0.9,24.0,24.0,24.0,25.0,20.0,22.0,31.0
24,Phoenix–Mesa–Chandler,29-2035,Magnetic Resonance Imaging Technologists,93713.0,a_median_local,93713.0,122772.0,88180.0,122200.0,551,...,48.3,38.0,12.5,104.0,25.0,7.0,22.0,24.0,4.0,2.0
27,Phoenix–Mesa–Chandler,29-1126,Respiratory Therapists,66074.0,a_median_local,66074.0,81733.0,80450.0,93048.0,569,...,63.2,30.8,5.6,46.0,28.0,11.0,21.0,16.0,10.0,7.0
87,Phoenix–Mesa–Chandler,29-2034,Radiologic Technologists and Technicians,63714.0,a_median_local,63714.0,103405.0,77660.0,109564.0,3032,...,70.2,24.2,5.1,190.0,86.0,2.0,3.0,32.0,15.0,27.0
92,Phoenix–Mesa–Chandler,47-1011,First-Line Supervisors of Construction Trades ...,58994.0,a_median_local,58994.0,74880.0,78690.0,70200.0,862,...,72.7,8.8,1.9,183.0,91.0,20.0,15.0,40.0,26.0,17.0
109,Phoenix–Mesa–Chandler,47-5023,"Earth Drillers, Except Oil and Gas",56634.0,a_median_local,56634.0,62400.0,59600.0,55487.5,65,...,76.5,3.7,0.2,311.0,107.0,41.0,39.0,2.0,33.0,34.0
110,Phoenix–Mesa–Chandler,29-1124,Radiation Therapists,106190.0,a_median_local,106190.0,103605.0,101990.0,119600.0,93,...,47.4,40.6,11.5,364.0,108.0,32.0,34.0,30.0,3.0,6.0
116,Phoenix–Mesa–Chandler,29-2031,Cardiovascular Technologists and Technicians,60000.0,a_median_local,60000.0,98280.0,67260.0,119652.0,1149,...,52.6,31.2,13.2,296.0,113.0,6.0,10.0,36.0,22.0,19.0


In [207]:
# Create JTM Rankings (Weighted Wage Version)

# Create the weighted version by copying sub_ba_jtm
sub_ba_jtm_ww = sub_ba_jtm.copy()

# Calculate JTM rank using raw growth variables instead of normalized
sub_ba_jtm_ww['composite_score'] = (
    sub_ba_jtm_ww['POSTING_OBS_NATIONAL_rank'] +
    sub_ba_jtm_ww['OBS_GROWTH_NATIONAL_rank'] +  # Use raw national growth rank
    sub_ba_jtm_ww['POSTINGS_OBS_LOCAL_rank'] +
    sub_ba_jtm_ww['obs_growth_local_rank'] +     # Use raw local growth rank
    3 * sub_ba_jtm_ww['salary_use_rank'] +       # 3x weight on salary
    sub_ba_jtm_ww['ECI_SOC5_ESTIMATED_rank'] +
    sub_ba_jtm_ww['mean_tot_earn_rank']
)

# Step 2: Rank the composite score within each MSA (lower composite score = better rank)
sub_ba_jtm_ww['jtm_rank'] = sub_ba_jtm_ww.groupby('phoenix_msa')['composite_score'].rank(method='min', ascending=True)

# Sort by MSA and rank
sub_ba_jtm_ww = sub_ba_jtm_ww.sort_values(['phoenix_msa', 'jtm_rank'])

# Reorder columns to put jtm_rank second
cols = sub_ba_jtm_ww.columns.tolist()
cols.remove('jtm_rank')
cols.insert(1, 'jtm_rank')  # Insert at position 1 (second column)
sub_ba_jtm_ww = sub_ba_jtm_ww[cols]

# Add positive demand growth indicator
sub_ba_jtm_ww['positive_demand_growth'] = sub_ba_jtm_ww['OBS_GROWTH_NATIONAL'] > 0

for msa_group in sub_ba_jtm_ww.groupby('phoenix_msa'):
    msa = msa_group[0]
    data = msa_group[1]
    
    # 1. LOCAL DEMAND terciles (based on POSTINGS_OBS_LOCAL)
    sub_ba_jtm_ww.loc[data.index, 'lp_tercile'] = pd.qcut(data['POSTINGS_OBS_LOCAL'], 3, labels=[1, 2, 3])
    sub_ba_jtm_ww.loc[data.index, 'lp_tercile_label'] = sub_ba_jtm_ww.loc[data.index, 'lp_tercile'].map({1: 'Low', 2: 'Medium', 3: 'High'})
    
    # 2. NATIONAL DEMAND terciles (based on POSTING_OBS_NATIONAL)
    sub_ba_jtm_ww.loc[data.index, 'np_tercile'] = pd.qcut(data['POSTING_OBS_NATIONAL'], 3, labels=[1, 2, 3])
    sub_ba_jtm_ww.loc[data.index, 'np_tercile_label'] = sub_ba_jtm_ww.loc[data.index, 'np_tercile'].map({1: 'Low', 2: 'Medium', 3: 'High'})
    
    # 3. NATIONAL GROWTH terciles (based on OBS_GROWTH_NATIONAL - ALL occupations)
    sub_ba_jtm_ww.loc[data.index, 'growth_tercile'] = pd.qcut(data['OBS_GROWTH_NATIONAL'], 3, labels=[1, 2, 3])
    sub_ba_jtm_ww.loc[data.index, 'growth_tercile_label'] = sub_ba_jtm_ww.loc[data.index, 'growth_tercile'].map({1: 'Low Growth', 2: 'Medium Growth', 3: 'High Growth'})
    
    # 4. LOCAL GROWTH terciles (based on obs_growth_local)
    sub_ba_jtm_ww.loc[data.index, 'local_growth_tercile'] = pd.qcut(data['obs_growth_local'], 3, labels=[1, 2, 3])
    sub_ba_jtm_ww.loc[data.index, 'local_growth_tercile_label'] = sub_ba_jtm_ww.loc[data.index, 'local_growth_tercile'].map({1: 'Low Local Growth', 2: 'Medium Local Growth', 3: 'High Local Growth'})

# Add salary growth estimate
sub_ba_jtm_ww['estimate_salary_growth'] = (sub_ba_jtm_ww['ECI_SOC5_ESTIMATED'] * 100).round(2).astype(str) + " %"

In [208]:
# Debug why a_median_local is blank
print("=== DEBUGGING a_median_local ===")

# 1. Check if phoenix_wages_final_soc exists and has data
print("1. Check phoenix_wages_final_soc:")
if 'phoenix_wages_final_soc' in globals():
    print(f"   Shape: {phoenix_wages_final_soc.shape}")
    print(f"   Columns: {phoenix_wages_final_soc.columns.tolist()}")
    print(f"   Non-null a_median_local: {phoenix_wages_final_soc['a_median_local'].notna().sum()}")
    print(f"   Sample data:")
    print(phoenix_wages_final_soc[['SOC_5', 'PHOENIX_MSA', 'a_median_local']].head())
else:
    print("   phoenix_wages_final_soc not found!")

# 2. Check the merge step in phoenix_lmi
print("\n2. Check merge with ACS data:")
if 'phoenix_lmi' in globals():
    print(f"   phoenix_lmi shape: {phoenix_lmi.shape}")
    print(f"   a_median_local non-null: {phoenix_lmi['a_median_local'].notna().sum()}")
    print(f"   Sample phoenix_lmi data:")
    print(phoenix_lmi[['SOC_5', 'phoenix_msa', 'a_median_local']].head())
    
    # Check if merge keys match
    print(f"\n   Unique SOC_5 codes in phoenix_lmi: {phoenix_lmi['SOC_5'].nunique()}")
    print(f"   Unique phoenix_msa values: {phoenix_lmi['phoenix_msa'].unique()}")
else:
    print("   phoenix_lmi not found!")

# 3. Check the original ACS wage calculation
print("\n3. Check original ACS wage calculation:")
if 'phoenix_wages_final' in globals():
    print(f"   phoenix_wages_final shape: {phoenix_wages_final.shape}")
    print(f"   Columns: {phoenix_wages_final.columns.tolist()}")
    print(f"   Non-null a_median_local: {phoenix_wages_final['a_median_local'].notna().sum()}")
    print(f"   Sample data:")
    print(phoenix_wages_final[['OCCSOC', 'PHOENIX_MSA', 'a_median_local']].head())
else:
    print("   phoenix_wages_final not found!")

# 4. Check the crosswalk step
print("\n4. Check crosswalk step:")
if 'acs_soc_xw' in globals():
    print(f"   Crosswalk shape: {acs_soc_xw.shape}")
    print(f"   Sample crosswalk:")
    print(acs_soc_xw[['soc_2019_5_acs', 'soc_2019_5']].head())
else:
    print("   acs_soc_xw not found!")

# 5. Check if the issue is in the merge logic
print("\n5. Debug merge logic:")
if 'phoenix_wages_final_soc' in globals() and 'phoenix_lmi' in globals():
    # Check merge keys alignment
    lmi_keys = set(phoenix_lmi[['SOC_5', 'phoenix_msa']].drop_duplicates().apply(tuple, axis=1))
    acs_keys = set(phoenix_wages_final_soc[['SOC_5', 'PHOENIX_MSA']].drop_duplicates().apply(tuple, axis=1))
    
    print(f"   LMI unique SOC_5 + MSA combinations: {len(lmi_keys)}")
    print(f"   ACS unique SOC_5 + MSA combinations: {len(acs_keys)}")
    
    # Check for matches
    common_keys = lmi_keys.intersection(acs_keys)
    print(f"   Common merge keys: {len(common_keys)}")
    
    if len(common_keys) == 0:
        print("   NO COMMON MERGE KEYS - This is the problem!")
        print(f"   Sample LMI keys: {list(lmi_keys)[:5]}")
        print(f"   Sample ACS keys: {list(acs_keys)[:5]}")

=== DEBUGGING a_median_local ===
1. Check phoenix_wages_final_soc:
   Shape: (845, 6)
   Columns: ['PHOENIX_MSA', 'SOC_5', 'a_median_local', 'h_median_local', 'tot_emp_local', 'n_observations']
   Non-null a_median_local: 845
   Sample data:
     SOC_5            PHOENIX_MSA  a_median_local
0  11-1011  Phoenix–Mesa–Chandler        154194.0
1  11-1021  Phoenix–Mesa–Chandler         80000.0
2  11-1031  Phoenix–Mesa–Chandler        154194.0
3  11-2011  Phoenix–Mesa–Chandler         76692.0
4  11-2021  Phoenix–Mesa–Chandler         83300.0

2. Check merge with ACS data:
   phoenix_lmi shape: (665, 37)
   a_median_local non-null: 642
   Sample phoenix_lmi data:
     SOC_5            phoenix_msa  a_median_local
0  29-1171  Phoenix–Mesa–Chandler        111425.0
1  47-2231  Phoenix–Mesa–Chandler         48000.0
2  29-1071  Phoenix–Mesa–Chandler        117988.0
3  11-9111  Phoenix–Mesa–Chandler         80176.0
4  31-2021  Phoenix–Mesa–Chandler         25000.0

   Unique SOC_5 codes in phoenix_l

In [209]:
sub_ba_jtm_ww

,phoenix_msa,jtm_rank,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,...,positive_demand_growth,lp_tercile,lp_tercile_label,np_tercile,np_tercile_label,growth_tercile,growth_tercile_label,local_growth_tercile,local_growth_tercile_label,estimate_salary_growth
10,Phoenix–Mesa–Chandler,1.0,29-2032,Diagnostic Medical Sonographers,75000.0,a_median_local,75000.0,109200.0,89340.0,114400.0,...,True,2,Medium,3,High,3,High Growth,3,High Local Growth,17.96 %
27,Phoenix–Mesa–Chandler,2.0,29-1126,Respiratory Therapists,66074.0,a_median_local,66074.0,81733.0,80450.0,93048.0,...,True,2,Medium,3,High,3,High Growth,3,High Local Growth,21.46 %
14,Phoenix–Mesa–Chandler,3.0,49-9062,Medical Equipment Repairers,61354.0,a_median_local,61354.0,60000.0,62630.0,62920.0,...,True,1,Low,1,Low,3,High Growth,3,High Local Growth,20.71 %
24,Phoenix–Mesa–Chandler,4.0,29-2035,Magnetic Resonance Imaging Technologists,93713.0,a_median_local,93713.0,122772.0,88180.0,122200.0,...,True,2,Medium,3,High,3,High Growth,3,High Local Growth,19.89 %
23,Phoenix–Mesa–Chandler,5.0,49-9041,Industrial Machinery Mechanics,60000.0,a_median_local,60000.0,55120.0,63760.0,58853.5,...,True,2,Medium,2,Medium,3,High Growth,3,High Local Growth,20.6 %
128,Phoenix–Mesa–Chandler,6.0,11-3071,"Transportation, Storage, and Distribution Mana...",59351.0,a_median_local,59351.0,73101.0,102010.0,75000.0,...,True,3,High,3,High,3,High Growth,3,High Local Growth,10.79 %
87,Phoenix–Mesa–Chandler,7.0,29-2034,Radiologic Technologists and Technicians,63714.0,a_median_local,63714.0,103405.0,77660.0,109564.0,...,True,3,High,3,High,2,Medium Growth,3,High Local Growth,17.82 %
142,Phoenix–Mesa–Chandler,8.0,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",56275.0,a_median_local,56275.0,63960.0,63980.0,61290.0,...,True,3,High,3,High,3,High Growth,3,High Local Growth,19.64 %
92,Phoenix–Mesa–Chandler,9.0,47-1011,First-Line Supervisors of Construction Trades ...,58994.0,a_median_local,58994.0,74880.0,78690.0,70200.0,...,True,3,High,2,Medium,3,High Growth,3,High Local Growth,10.3 %
215,Phoenix–Mesa–Chandler,10.0,19-5012,Occupational Health and Safety Technicians,56000.0,a_median_local,56000.0,72800.0,58440.0,67600.0,...,True,3,High,3,High,3,High Growth,2,Medium Local Growth,26.0 %


In [210]:
manual_additions_by_msa = {
    # "Phoenix–Mesa–Chandler": [
    #     # Your manual sector override occupations
    #     "15-1211",  # Computer Systems Analysts -> Technology
    #     "11-9111",  # Medical and Health Services Managers -> Healthcare
    #     "49-9062",  # Medical Equipment Repairers -> Healthcare
    #     "41-9031",  # Sales Engineers -> Tech-Adjacent
    #     "13-1161",  # Market Research Analysts and Marketing Specialists -> Tech-Adjacent
    #     "41-4011",  # Sales Representatives, Wholesale and Manufacturing, Technical and Scientific Products -> Tech-Adjacent
    #     "41-3091",  # Sales Representatives of Services, Except Advertising, Insurance, Financial Services, and Travel -> Tech-Adjacent
    #     "41-1012",  # First-Line Supervisors of Non-Retail Sales Workers -> Tech-Adjacent
    #     "29-1299",  # Healthcare Diagnosing or Treating Practitioners -> Healthcare
    #     "29-2061",  # Licensed Practical and Licensed Vocational Nurses -> Healthcare
    #     "29-2081",  # Opticians, Dispensing -> Healthcare
    #     "29-2099",  # Health Technologists and Technicians -> Healthcare
    #     "29-2092",  # Hearing Aid Specialists -> Healthcare
    #     "31-2012",  # Occupational Therapy Aides -> Healthcare
    #     "31-2011",  # Occupational Therapy Assistants -> Healthcare
    #     "29-2072",  # Medical Records Specialists -> Healthcare
    #     "31-9095",  # Pharmacy Aides -> Healthcare
    #     "21-1094",  # Community Health Workers -> Healthcare
    # ]
}

# Keep your manual sector overrides dictionary for sector assignment
manual_sector_overrides = {
    # '15-1211': 'Technology',
    # '11-9111': 'Healthcare',
    # '49-9062': 'Healthcare',
    # '41-9031': 'Tech-Adjacent',
    # '13-1161': 'Tech-Adjacent',
    # '41-4011': 'Tech-Adjacent',
    # '41-3091': 'Tech-Adjacent',
    # '41-1012': 'Tech-Adjacent',
    # '29-1299': 'Healthcare',
    # '29-2061': 'Healthcare',
    # '29-2081': 'Healthcare',
    # '29-2099': 'Healthcare',
    # '29-2092': 'Healthcare',
    # '31-2012': 'Healthcare',
    # '31-2011': 'Healthcare',
    # '29-2072': 'Healthcare',
    # '31-9095': 'Healthcare',
    # '21-1094': 'Healthcare',
}

# MSA-specific SOC codes to REMOVE
manual_removals_by_msa = {
    # "Phoenix–Mesa–Chandler": [
    #     "43-5051",  # Postal Service Clerks
    #     "21-2011",  # Clergy
    #     "27-3041",  # Editors
    #     "27-3023",  # News Analysts, Reporters, Journalists
    #     "19-3091",  # Anthropologists and Archeologists
    #     "27-4031",  # Camera Operators, Television, Video, and Film
    # ]
}

In [211]:
# --------------------------------------------------------------
# Manual Additions & Removals – Sub-BA JTM
# --------------------------------------------------------------
print("Applying manual additions and removals to sub-BA JTMs...")

# Remove specified SOCs by MSA
for msa, soc_list in manual_removals_by_msa.items():
    sub_ba_jtm = sub_ba_jtm[~(
        (sub_ba_jtm['phoenix_msa'] == msa) &
        (sub_ba_jtm['SOC_5'].isin(soc_list))
    )]

# Add specified SOCs by MSA
manual_add_rows_subba = []
for msa, soc_list in manual_additions_by_msa.items():
    for soc_code in soc_list:
        match = phoenix_lmi[
            (phoenix_lmi['phoenix_msa'] == msa) &
            (phoenix_lmi['SOC_5'] == soc_code)
        ]
        if not match.empty:
            temp = match.copy()
            temp['manual_addition'] = 1
            manual_add_rows_subba.append(temp)

if manual_add_rows_subba:
    sub_ba_jtm = pd.concat([sub_ba_jtm, pd.concat(manual_add_rows_subba)], ignore_index=True)

Applying manual additions and removals to sub-BA JTMs...


In [212]:
# Metrics for JTM rank
metrics = [
    'POSTING_OBS_NATIONAL', 'growth_normalized', 'POSTINGS_OBS_LOCAL',
    'local_growth_normalized', 'ECI_SOC5_ESTIMATED', 'salary_use', 'mean_tot_earn'
]

# Recalculate rank columns
for metric in metrics:
    if metric in sub_ba_jtm.columns:
        sub_ba_jtm[f'{metric}_rank'] = sub_ba_jtm[metric].rank(ascending=False, method='min')


In [213]:
# sub_ba_jtm_ww_output is just sub_ba_jtm_ww combined with sub_ba_jtm_manual (manually adding jtm's back in the list) and dropping duplicates, for Louiiana we do not add anything manually back in
sub_ba_jtm_ww_output = sub_ba_jtm_ww


In [214]:
unique_occs = sub_ba_jtm_ww_output[['SOC_5', 'SOC_5_NAME']].drop_duplicates()


In [215]:
# Save unique occupations for classification

# Create output dataset with selected columns (now with growth terciles)
sub_ba_jtm_ww_output = sub_ba_jtm_ww[['phoenix_msa', 'jtm_rank', 'SOC_5', 'SOC_5_NAME', 
                                       'lp_tercile_label', 'EDUCATION_LEVEL', 'salary_use', 
                                       'np_tercile_label', 'estimate_salary_growth', 'growth_tercile_label']].copy()

# Rename columns to match your labeling (10 columns - back to original structure)
sub_ba_jtm_ww_output.columns = ["Phoenix MSA", "JTM Ranking", "Occupation Code", "Occupation", "Local Demand",
                                "Education Level Needed", "Estimated Salary", "National Demand", 
                                "Salary Growth Estimate", "National Demand Growth"]

# Create unique occupations from the output data
unique_occs = sub_ba_jtm_ww_output[['Occupation Code', 'Occupation']].drop_duplicates()

# Add leading apostrophe to force text format
unique_occs_fixed = unique_occs.copy()
unique_occs_fixed['Occupation Code'] = "'" + unique_occs_fixed['Occupation Code'].astype(str)
unique_occs_fixed.to_csv('unique_occs_phoenix.csv', index=False)
print(f"Saved {len(unique_occs_fixed)} unique occupations with text formatting")


# Save Output Files


# Define salary columns to reorder
salary_columns = [
    'salary_use', 'salary_use_source', 'a_median_local', 'MEDIAN_SALARY_LOCAL',
    'a_median', 'MEDIAN_SALARY_NATIONAL'
]

# Function to move salary columns after SOC_5_NAME
def reorder_salary_columns(df, salary_cols):
    cols = df.columns.tolist()
    if 'SOC_5_NAME' in cols:
        idx = cols.index('SOC_5_NAME') + 1
        # Remove any of the salary columns if already in the list
        for col in salary_cols:
            if col in cols:
                cols.remove(col)
        # Insert salary columns in specified order
        for i, col in enumerate(salary_cols):
            if col in df.columns:
                cols.insert(idx + i, col)
        return df[cols]
    else:
        return df

# Apply to sub-BA JTM DataFrames
sub_ba_jtm_ww = reorder_salary_columns(sub_ba_jtm_ww, salary_columns)
# sub_ba_jtm_ww_fast_facts = reorder_salary_columns(sub_ba_jtm_ww_fast_facts, salary_columns)

# Compose filenames
main_filename = f'phoenix_sub_ba_jtm_weighted_wage_{today_str}_update.xlsx'
# facts_filename = f'phoenix_fast_facts_sub_ba_jtm_weighted_wage_{today_str}_update.xlsx'

# Save files
sub_ba_jtm_ww.to_excel(main_filename, index=False)
# sub_ba_jtm_ww_fast_facts.to_excel(facts_filename, index=False)


Saved 44 unique occupations with text formatting


In [216]:
sub_ba_jtm_ww

,phoenix_msa,jtm_rank,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,...,positive_demand_growth,lp_tercile,lp_tercile_label,np_tercile,np_tercile_label,growth_tercile,growth_tercile_label,local_growth_tercile,local_growth_tercile_label,estimate_salary_growth
10,Phoenix–Mesa–Chandler,1.0,29-2032,Diagnostic Medical Sonographers,75000.0,a_median_local,75000.0,109200.0,89340.0,114400.0,...,True,2,Medium,3,High,3,High Growth,3,High Local Growth,17.96 %
27,Phoenix–Mesa–Chandler,2.0,29-1126,Respiratory Therapists,66074.0,a_median_local,66074.0,81733.0,80450.0,93048.0,...,True,2,Medium,3,High,3,High Growth,3,High Local Growth,21.46 %
14,Phoenix–Mesa–Chandler,3.0,49-9062,Medical Equipment Repairers,61354.0,a_median_local,61354.0,60000.0,62630.0,62920.0,...,True,1,Low,1,Low,3,High Growth,3,High Local Growth,20.71 %
24,Phoenix–Mesa–Chandler,4.0,29-2035,Magnetic Resonance Imaging Technologists,93713.0,a_median_local,93713.0,122772.0,88180.0,122200.0,...,True,2,Medium,3,High,3,High Growth,3,High Local Growth,19.89 %
23,Phoenix–Mesa–Chandler,5.0,49-9041,Industrial Machinery Mechanics,60000.0,a_median_local,60000.0,55120.0,63760.0,58853.5,...,True,2,Medium,2,Medium,3,High Growth,3,High Local Growth,20.6 %
128,Phoenix–Mesa–Chandler,6.0,11-3071,"Transportation, Storage, and Distribution Mana...",59351.0,a_median_local,59351.0,73101.0,102010.0,75000.0,...,True,3,High,3,High,3,High Growth,3,High Local Growth,10.79 %
87,Phoenix–Mesa–Chandler,7.0,29-2034,Radiologic Technologists and Technicians,63714.0,a_median_local,63714.0,103405.0,77660.0,109564.0,...,True,3,High,3,High,2,Medium Growth,3,High Local Growth,17.82 %
142,Phoenix–Mesa–Chandler,8.0,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",56275.0,a_median_local,56275.0,63960.0,63980.0,61290.0,...,True,3,High,3,High,3,High Growth,3,High Local Growth,19.64 %
92,Phoenix–Mesa–Chandler,9.0,47-1011,First-Line Supervisors of Construction Trades ...,58994.0,a_median_local,58994.0,74880.0,78690.0,70200.0,...,True,3,High,2,Medium,3,High Growth,3,High Local Growth,10.3 %
215,Phoenix–Mesa–Chandler,10.0,19-5012,Occupational Health and Safety Technicians,56000.0,a_median_local,56000.0,72800.0,58440.0,67600.0,...,True,3,High,3,High,3,High Growth,2,Medium Local Growth,26.0 %


In [217]:
# Create Original JTM Rankings (Original Version) 

# Create the original version by copying sub_ba_jtm
sub_ba_jtm_og = sub_ba_jtm.copy()

# FIXED: Calculate original JTM rank using raw growth variables instead of normalized
# Step 1: Calculate composite score first
sub_ba_jtm_og['composite_score'] = (
    sub_ba_jtm_og['POSTING_OBS_NATIONAL_rank'] +
    sub_ba_jtm_og['OBS_GROWTH_NATIONAL_rank'] +        # Use raw national growth rank
    sub_ba_jtm_og['POSTINGS_OBS_LOCAL_rank'] +
    sub_ba_jtm_og['obs_growth_local_rank'] +           # Use raw local growth rank  
    sub_ba_jtm_og['ECI_SOC5_ESTIMATED_rank'] +
    sub_ba_jtm_og['salary_use_rank'] +                 # No 3x multiplier in original
    sub_ba_jtm_og['mean_tot_earn_rank']
)

# Step 2: Rank the composite score within each MSA (lower composite score = better rank)
sub_ba_jtm_og['jtm_rank'] = sub_ba_jtm_og.groupby('phoenix_msa')['composite_score'].rank(method='min', ascending=True)

# Sort by MSA and rank
sub_ba_jtm_og = sub_ba_jtm_og.sort_values(['phoenix_msa', 'jtm_rank'])

# Reorder columns to put jtm_rank second
cols = sub_ba_jtm_og.columns.tolist()
cols.remove('jtm_rank')
cols.insert(1, 'jtm_rank')
sub_ba_jtm_og = sub_ba_jtm_og[cols]

# --------------------------------------------------------------
# Calculate Original Fast Facts Summary (Top 20 only)
# --------------------------------------------------------------
# Filter to top 20 JTMs per MSA for fast facts (matching R code)
sub_ba_jtm_og_top20 = sub_ba_jtm_og[sub_ba_jtm_og['jtm_rank'] <= 20]

sub_ba_jtm_og_fast_facts = sub_ba_jtm_og_top20.groupby('phoenix_msa').agg({
    'POSTINGS_OBS_LOCAL': 'mean',
    'salary_use': 'mean', 
    'mean_tot_earn': 'mean',
    'EDUCATION_LEVEL_CODE': ['mean', lambda x: np.average(x, weights=sub_ba_jtm_og_top20.loc[x.index, 'POSTINGS_OBS_LOCAL'])]
}).round(2)

# Flatten column names
sub_ba_jtm_og_fast_facts.columns = ['avg_nmbr_local_postings', 'avg_local_salary', 'avg_total_earnings_6yrs', 'avg_educ', 'wtd_avg_educ']

# Keep education as numbers for original version (matching R code - no percentage conversion)
sub_ba_jtm_og_fast_facts = sub_ba_jtm_og_fast_facts.reset_index()

# Save Original Version Files
# Apply salary column reordering
sub_ba_jtm_og = reorder_salary_columns(sub_ba_jtm_og, salary_columns)
sub_ba_jtm_og_fast_facts = reorder_salary_columns(sub_ba_jtm_og_fast_facts, salary_columns)

# Compose dynamic filenames
og_main_filename = f'phoenix_sub_ba_jtm_orig_{today_str}_update.xlsx'
og_facts_filename = f'phoenix_fast_facts_sub_ba_jtm_orig_{today_str}_update.xlsx'

# Save files
sub_ba_jtm_og.to_excel(og_main_filename, index=False)
sub_ba_jtm_og_fast_facts.to_excel(og_facts_filename, index=False)

print(f"Files saved: {og_main_filename}, {og_facts_filename}")

Files saved: phoenix_sub_ba_jtm_orig_2025.08.07_update.xlsx, phoenix_fast_facts_sub_ba_jtm_orig_2025.08.07_update.xlsx


In [218]:
# BA (Bachelor's degree) JTM Analysis

# BA analysis - matching R code exactly
ba = ['Bachelor\'s degree']

# # Apply the >=50 filter BEFORE calling identify_jtm (like R code)
# ba_jtm_filtered = la_lmi[la_lmi['POSTINGS_OBS_LOCAL'] >= 50].copy()

# Change this in your BA analysis:
ba_jtm_filtered = phoenix_lmi[phoenix_lmi['POSTINGS_OBS_LOCAL'] >= 30].copy()  # Changed from 50 to 30

In [219]:
# Manual Additions & Removals – BA JTM
# Start from ba_jtm_filtered (pre-ranking version)

# Remove specified SOCs by MSA
for msa, soc_list in manual_removals_by_msa.items():
    ba_jtm_filtered = ba_jtm_filtered[~(
        (ba_jtm_filtered['phoenix_msa'] == msa) &
        (ba_jtm_filtered['SOC_5'].isin(soc_list))
    )]

# Add specified SOCs by MSA
manual_add_rows_ba = []
for msa, soc_list in manual_additions_by_msa.items():
    for soc_code in soc_list:
        match = phoenix_lmi[
            (phoenix_lmi['phoenix_msa'] == msa) &
            (phoenix_lmi['SOC_5'] == soc_code)
        ]
        if not match.empty:
            temp = match.copy()
            temp['manual_addition'] = 1
            manual_add_rows_ba.append(temp)

# Add to ba_jtm_filtered
if manual_add_rows_ba:
    ba_jtm_filtered = pd.concat([ba_jtm_filtered, pd.concat(manual_add_rows_ba)], ignore_index=True)

In [220]:
# Recalculate Ranks After Manual Edits – BA JTM

# Define education level and metrics again if needed
ba = ["Bachelor's degree"]
# Updated metrics to use raw growth variables instead of normalized
metrics = ['POSTING_OBS_NATIONAL', 'OBS_GROWTH_NATIONAL', 'POSTINGS_OBS_LOCAL',
           'obs_growth_local', 'ECI_SOC5_ESTIMATED', 'salary_use', 'mean_tot_earn']
occ_cols = ['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL']

# Re-run identify_jtm to restore rank columns after manual edits
ba_jtm_filtered = identify_jtm(ba_jtm_filtered, ba, metrics, occ_cols)

# Create Original BA JTM Rankings - 

ba_jtm_og = ba_jtm_filtered.copy()

# FIXED: Calculate original JTM rank using raw growth variables instead of normalized
# Step 1: Calculate composite score first (no 3x salary multiplier, includes local growth)
ba_jtm_og['composite_score'] = (
    ba_jtm_og['POSTING_OBS_NATIONAL_rank'] +
    ba_jtm_og['OBS_GROWTH_NATIONAL_rank'] +        # Use raw national growth rank
    ba_jtm_og['POSTINGS_OBS_LOCAL_rank'] +
    ba_jtm_og['obs_growth_local_rank'] +           # Use raw local growth rank
    ba_jtm_og['ECI_SOC5_ESTIMATED_rank'] +
    ba_jtm_og['salary_use_rank'] +
    ba_jtm_og['mean_tot_earn_rank']
)

# Step 2: Rank the composite score within each MSA (lower composite score = better rank)
ba_jtm_og['jtm_rank'] = ba_jtm_og.groupby('phoenix_msa')['composite_score'].rank(method='min', ascending=True)

# Sort and reorder
ba_jtm_og = ba_jtm_og.sort_values(['phoenix_msa', 'jtm_rank'])
cols = ba_jtm_og.columns.tolist()
cols.remove('jtm_rank')
cols.insert(1, 'jtm_rank')
ba_jtm_og = ba_jtm_og[cols]

print(f"Original BA JTM dataset: {ba_jtm_og.shape}")
print(ba_jtm_og.groupby('phoenix_msa').size())

Original BA JTM dataset: (58, 44)
phoenix_msa
Phoenix–Mesa–Chandler    58
dtype: int64


In [221]:
# Then use the standard identify_jtm function (which has >=30 threshold)
ba_jtm = identify_jtm(ba_jtm_filtered, ba, metrics, occ_cols)

# Additional living wage filter
ba_jtm = ba_jtm[ba_jtm['salary_use'] > ba_jtm['living_wage_metric']]

print(f"BA JTM base dataset: {ba_jtm.shape}")
print(f"BA JTMs by MSA (before ranking):")
print(ba_jtm.groupby('phoenix_msa').size())

BA JTM base dataset: (58, 42)
BA JTMs by MSA (before ranking):
phoenix_msa
Phoenix–Mesa–Chandler    58
dtype: int64


In [222]:
# BA JTM Weighted Version (3x salary weight) 

ba_jtm_weighted = ba_jtm.copy()

# Calculate weighted JTM rank using raw growth variables and two-step approach
# Step 1: Calculate composite score first (3x salary weight)
# Calculate JTM rank using raw growth variables instead of normalized
ba_jtm_weighted ['composite_score'] = (
    ba_jtm_weighted ['POSTING_OBS_NATIONAL_rank'] +
    ba_jtm_weighted ['OBS_GROWTH_NATIONAL_rank'] +        # Use raw national growth rank
    ba_jtm_weighted ['POSTINGS_OBS_LOCAL_rank'] +
    ba_jtm_weighted ['obs_growth_local_rank'] +           # Use raw local growth rank
    ba_jtm_weighted ['ECI_SOC5_ESTIMATED_rank'] +
    3 * ba_jtm_weighted ['salary_use_rank'] +             # 3x weight on salary
    ba_jtm_weighted ['mean_tot_earn_rank']
)

# Step 2: Rank the composite score within each MSA (lower composite score = better rank)
ba_jtm_weighted['jtm_rank'] = ba_jtm_weighted.groupby('phoenix_msa')['composite_score'].rank(method='min', ascending=True)

# Sort by MSA and rank
ba_jtm_weighted = ba_jtm_weighted.sort_values(['phoenix_msa', 'jtm_rank'])

# Reorder columns to put jtm_rank second
cols = ba_jtm_weighted.columns.tolist()
cols.remove('jtm_rank')
cols.insert(1, 'jtm_rank')
ba_jtm_weighted = ba_jtm_weighted[cols]

# BA JTM Original Version (1x salary weight) 

ba_jtm_original = ba_jtm.copy()

# FIXED: Calculate original JTM rank using raw growth variables and two-step approach
# Step 1: Calculate composite score first (1x salary weight)
ba_jtm_original['composite_score'] = (
    ba_jtm_original['POSTING_OBS_NATIONAL_rank'] +  
    ba_jtm_original['OBS_GROWTH_NATIONAL_rank'] +        # Use raw national growth rank
    ba_jtm_original['POSTINGS_OBS_LOCAL_rank'] +
    ba_jtm_original['obs_growth_local_rank'] +           # Use raw local growth rank
    ba_jtm_original['ECI_SOC5_ESTIMATED_rank'] +
    ba_jtm_original['salary_use_rank'] +                 # 1x weight on salary
    ba_jtm_original['mean_tot_earn_rank']
)

# Step 2: Rank the composite score within each MSA (lower composite score = better rank)
ba_jtm_original['jtm_rank'] = ba_jtm_original.groupby('phoenix_msa')['composite_score'].rank(method='min', ascending=True)

# Sort by MSA and rank
ba_jtm_original = ba_jtm_original.sort_values(['phoenix_msa', 'jtm_rank'])

# Reorder columns to put jtm_rank second
cols = ba_jtm_original.columns.tolist()
cols.remove('jtm_rank')
cols.insert(1, 'jtm_rank')
ba_jtm_original = ba_jtm_original[cols]



In [223]:
# Save BA JTM Output Files

# Reorder columns so salary vars come after SOC_5_NAME
def reorder_salary_columns(df, salary_cols):
    cols = df.columns.tolist()
    if 'SOC_5_NAME' in cols:
        idx = cols.index('SOC_5_NAME') + 1
        # Remove salary columns from current position
        for col in salary_cols:
            if col in cols:
                cols.remove(col)
        # Insert salary columns right after SOC_5_NAME
        for i, col in enumerate(salary_cols):
            if col in df.columns:
                cols.insert(idx + i, col)
        return df[cols]
    else:
        return df

# Apply to both BA JTM outputs
ba_jtm_weighted = reorder_salary_columns(ba_jtm_weighted, salary_columns)
ba_jtm_original = reorder_salary_columns(ba_jtm_original, salary_columns)

# Compose filenames with date
weighted_filename = f'phoenix_ba_jtm_weighted_wage_{today_str}.xlsx'
original_filename = f'phoenix_ba_jtm_orig_{today_str}.xlsx'

# Save files
ba_jtm_weighted.to_excel(weighted_filename, index=False)
ba_jtm_original.to_excel(original_filename, index=False)



In [224]:
# Add Tercile Labels and Salary Growth Estimate – BA JTMs


for msa, group in ba_jtm_weighted.groupby('phoenix_msa'):
    idx = group.index
    
    # 1. LOCAL DEMAND terciles (based on POSTINGS_OBS_LOCAL)
    ba_jtm_weighted.loc[idx, 'lp_tercile'] = pd.qcut(group['POSTINGS_OBS_LOCAL'], 3, labels=[1, 2, 3])
    ba_jtm_weighted.loc[idx, 'lp_tercile_label'] = ba_jtm_weighted.loc[idx, 'lp_tercile'].map({
        1: 'Low', 2: 'Medium', 3: 'High'
    })
    
    # 2. NATIONAL DEMAND terciles (based on POSTING_OBS_NATIONAL)
    ba_jtm_weighted.loc[idx, 'np_tercile'] = pd.qcut(group['POSTING_OBS_NATIONAL'], 3, labels=[1, 2, 3])
    ba_jtm_weighted.loc[idx, 'np_tercile_label'] = ba_jtm_weighted.loc[idx, 'np_tercile'].map({
        1: 'Low', 2: 'Medium', 3: 'High'
    })
    
    # 3. NATIONAL GROWTH terciles (based on OBS_GROWTH_NATIONAL)
    ba_jtm_weighted.loc[idx, 'growth_tercile'] = pd.qcut(group['OBS_GROWTH_NATIONAL'], 3, labels=[1, 2, 3])
    ba_jtm_weighted.loc[idx, 'growth_tercile_label'] = ba_jtm_weighted.loc[idx, 'growth_tercile'].map({
        1: 'Low Growth', 2: 'Medium Growth', 3: 'High Growth'
    })
    
    # 4. LOCAL GROWTH terciles (based on OBS_GROWTH_LOCAL)
    ba_jtm_weighted.loc[idx, 'local_growth_tercile'] = pd.qcut(group['obs_growth_local'], 3, labels=[1, 2, 3])
    ba_jtm_weighted.loc[idx, 'local_growth_tercile_label'] = ba_jtm_weighted.loc[idx, 'local_growth_tercile'].map({
        1: 'Low Local Growth', 2: 'Medium Local Growth', 3: 'High Local Growth'
    })

# Salary growth estimate as a formatted string
ba_jtm_weighted['estimate_salary_growth'] = (ba_jtm_weighted['ECI_SOC5_ESTIMATED'] * 100).round(2).astype(str) + " %"

In [225]:
def check_final_dataset_education(df, dataset_name="Final Dataset"):
    """
    Comprehensive education check for final merged dataset
    """
    print(f"=== {dataset_name.upper()} COMPREHENSIVE EDUCATION CHECK ===")
    
    # Define expected mapping
    expected_mapping = {
        0: 'High school or GED',
        1: 'Associate degree', 
        2: 'Bachelor\'s degree',
        3: 'Master\'s degree',
        4: 'Ph.D. or professional degree'
    }
    
    print(f"\n1. EDUCATION LEVEL DISTRIBUTION:")
    print(df['EDUCATION_LEVEL'].value_counts())
    
    print(f"\n2. EDUCATION LEVEL CODE DISTRIBUTION:")
    print(df['EDUCATION_LEVEL_CODE'].value_counts())
    
    print(f"\n3. MISMATCH ANALYSIS:")
    total_mismatches = 0
    
    for code, expected_name in expected_mapping.items():
        # Find mismatched records
        mismatched = df[(df['EDUCATION_LEVEL_CODE'] == code) & 
                       (df['EDUCATION_LEVEL'] != expected_name)]
        
        if len(mismatched) > 0:
            total_mismatches += len(mismatched)
            print(f"\n   Code {code} should be '{expected_name}' but found:")
            mismatch_counts = mismatched['EDUCATION_LEVEL'].value_counts()
            for wrong_name, count in mismatch_counts.items():
                print(f"     '{wrong_name}': {count} occupations")
                
            # Show a few examples
            examples = mismatched[['SOC_5', 'SOC_5_NAME']].head(3)
            print(f"     Examples: {', '.join(examples['SOC_5_NAME'].tolist())}")
    
    print(f"\n4. SUMMARY:")
    print(f"Total education mismatches: {total_mismatches} out of {len(df)} occupations")
    print(f"Mismatch rate: {total_mismatches/len(df)*100:.1f}%")
    
    if total_mismatches == 0:
        print("All education data is consistent!")
    else:
        print("Education mismatches found - recommend applying fix")
        
        # Show surgical technologists specifically if they exist
        surgical_socs = ['29-2055', '29-9093']
        surgical_data = df[df['SOC_5'].isin(surgical_socs)]
        if len(surgical_data) > 0:
            print(f"\n   Surgical occupations check:")
            for _, row in surgical_data.iterrows():
                status = "CORRECT" if row['EDUCATION_LEVEL'] == expected_mapping.get(row['EDUCATION_LEVEL_CODE'], '') else "MISMATCH"
                print(f"     {status}: {row['SOC_5']} {row['SOC_5_NAME']}: {row['EDUCATION_LEVEL']}")
    
    return total_mismatches == 0

# Usage after national posting data:
print("After creating national posting data:")
check_posting_data_education(posting_data_national, "National Posting Data")

# Usage after final merged dataset:
print("\n" + "="*80)
print("After creating final merged dataset:")
is_clean = check_final_dataset_education(phoenix_lmi, "Phoenix LMI Final Dataset")

if not is_clean:
    print("\nTo fix all mismatches, run:")
    print("phoenix_lmi = fix_all_education_mismatches(phoenix_lmi)")

After creating national posting data:
=== NATIONAL POSTING DATA EDUCATION CHECK ===

Education level distribution:
EDUCATION_LEVEL
High school or GED              475
Bachelor's degree               180
Associate degree                 43
Ph.D. or professional degree     38
Master's degree                  30
Name: count, dtype: int64
Education mismatches: 0 out of 766 occupations (0.0%)
All education data is consistent!

After creating final merged dataset:
=== PHOENIX LMI FINAL DATASET COMPREHENSIVE EDUCATION CHECK ===

1. EDUCATION LEVEL DISTRIBUTION:
EDUCATION_LEVEL
High school or GED              414
Bachelor's degree               143
Associate degree                 42
Ph.D. or professional degree     37
Master's degree                  29
Name: count, dtype: int64

2. EDUCATION LEVEL CODE DISTRIBUTION:
EDUCATION_LEVEL_CODE
0    414
2    143
1     42
4     37
3     29
Name: count, dtype: int64

3. MISMATCH ANALYSIS:

4. SUMMARY:
Total education mismatches: 0 out of 665 occupatio

In [226]:
missing_socs = [
    '11-2011', '11-2021', '11-2022', '11-2032', '11-2033',
    '19-5011', '19-5012',
    '45-1011', '45-2011', '45-2021', '45-2041', '45-2091',
    '45-2092', '45-2093', '45-2099', '45-3031',
    '45-4011', '45-4021', '45-4022', '45-4023', '45-4029',
    '55-9999'
]

print("Missing SOCs in bls_lct_soc_cw:")
print(bls_lct_soc_cw[bls_lct_soc_cw['soc_2021_5'].isin(missing_socs)])


Missing SOCs in bls_lct_soc_cw:
    soc_2019_5                                    soc_2019_5_name soc_2021_5  \
1      55-3014                 Artillery and Missile Crew Members    55-9999   
2      55-1016                                  Infantry Officers    55-9999   
3      55-1012              Aircraft Launch and Recovery Officers    55-9999   
4      55-1017                            Special Forces Officers    55-9999   
5      55-3016                                           Infantry    55-9999   
6      55-2012  First-Line Supervisors of Weapons Specialists/...    55-9999   
7      55-1011                                  Air Crew Officers    55-9999   
8      55-1013                   Armored Assault Vehicle Officers    55-9999   
9      55-1014                     Artillery and Missile Officers    55-9999   
10     55-1015                Command and Control Center Officers    55-9999   
11     55-1019  Military Officer Special and Tactical Operatio...    55-9999   
12     5

In [227]:
print(phoenix_lmi.columns)
print(national_wage_growth.columns)


Index(['phoenix_msa', 'SOC_5', 'SOC_5_NAME', 'salary_use', 'salary_use_source',
       'a_median_local', 'MEDIAN_SALARY_LOCAL', 'a_median',
       'MEDIAN_SALARY_NATIONAL', 'POSTINGS_OBS_LOCAL', 'obs_growth_local',
       'local_median_growth', 'POSTING_OBS_NATIONAL', 'EDUCATION_LEVEL',
       'EDUCATION_LEVEL_CODE', 'OBS_GROWTH_NATIONAL', 'h_median_local',
       'tot_emp_local', 'n_observations', 'mean_tot_earn',
       'ECI_SOC5_ESTIMATED', 'tot_emp', 'wage_growth_national', 'MSA',
       'living_wage_metric', 'pct_wage_msa', 'edulevel_name', 'LLM_EXPOSURE',
       'tot_emp_55_plus', 'share_55_plus', 'few_obs_flag', 'Less_than_HS',
       'HS_Some_College', 'BA', 'BA_Plus', 'OBS_GROWTH_NATIONAL_rank',
       'obs_growth_local_rank'],
      dtype='object')
Index(['YEAR', 'SOC_2021_5', 'SOC_2021_5_NAME', 'ECI_SOC5_ESTIMATED'], dtype='object')


In [228]:
# Diagnostic: Which filter is limiting JTMs?

def diagnose_jtm_filters(df, educ_level, msa=None):
    """
    Step-by-step diagnosis of JTM filters to see where occupations get dropped
    """
    print(f"\n=== JTM Filter Diagnosis for {educ_level} ===")
    
    # Start with the base dataset
    step0 = df.copy()
    if msa:
        step0 = step0[step0['phoenix_msa'] == msa]
        print(f"Analyzing MSA: {msa}")
    
    print(f"Starting dataset: {len(step0)} total occupations")
    
    # Step 1: Education filter
    if educ_level == "Bachelor's degree":
        step1 = step0[step0['EDUCATION_LEVEL'].isin([educ_level])]
        filter_name = f"'{educ_level}'"
    else:  # Sub-BA
        sub_ba_levels = ['High school or GED', 'Associate degree', 'No Education Listed']
        step1 = step0[step0['EDUCATION_LEVEL'].isin(sub_ba_levels)]
        filter_name = "Sub-BA (High school, Associate, No Education Listed)"
    
    print(f"After education filter ({filter_name}): {len(step1)} occupations")
    print(f"  → Dropped: {len(step0) - len(step1)} occupations")
    
    # Step 2: Local postings threshold
    threshold = 50 if educ_level == "Bachelor's degree" else 30
    step2 = step1[step1['POSTINGS_OBS_LOCAL'] >= threshold]
    print(f"After local postings >= {threshold}: {len(step2)} occupations")
    print(f"  → Dropped: {len(step1) - len(step2)} occupations")
    
    # Step 3: National postings >=30
    step3 = step2[step2['POSTING_OBS_NATIONAL'] >= 30]
    print(f"After national postings >= 30: {len(step3)} occupations")
    print(f"  → Dropped: {len(step2) - len(step3)} occupations")
    
    # Step 4: National growth > 0 (using raw growth variable)
    step4 = step3[step3['OBS_GROWTH_NATIONAL'] > 0]
    print(f"After OBS_GROWTH_NATIONAL > 0: {len(step4)} occupations")
    print(f"  → Dropped: {len(step3) - len(step4)} occupations")
    
    # Step 5: Local growth > 0 (using raw growth variable)
    step5 = step4[step4['obs_growth_local'] > 0]
    print(f"After obs_growth_local > 0: {len(step5)} occupations")
    print(f"  → Dropped: {len(step4) - len(step5)} occupations")
    
    # Step 6: ECI > 0
    step6 = step5[step5['ECI_SOC5_ESTIMATED'] > 0]
    print(f"After ECI_SOC5_ESTIMATED > 0: {len(step6)} occupations")
    print(f"  → Dropped: {len(step5) - len(step6)} occupations")
    
    # Step 7: Salary >= living wage
    step7 = step6[step6['salary_use'] >= step6['living_wage_metric']]
    print(f"After salary_use >= living_wage_metric: {len(step7)} occupations")
    print(f"  → Dropped: {len(step6) - len(step7)} occupations")
    
    job_type = "BA" if educ_level == "Bachelor's degree" else "Sub-BA"
    print(f"\nFinal {job_type} JTMs: {len(step7)} occupations")
    
    # Show which MSAs have low counts
    if not msa:
        print(f"\n{job_type} JTMs by MSA:")
        msa_counts = step7.groupby('phoenix_msa').size().sort_values()
        for msa_name, count in msa_counts.items():
            print(f"  {msa_name}: {count} JTMs")
            if count < 9:
                print(f"    ^^^ LOW COUNT MSA ^^^")
    
    return step7

# Run diagnosis for BA occupations
print("BACHELOR'S DEGREE ANALYSIS")
print("=" * 80)
ba_diagnostic = diagnose_jtm_filters(phoenix_lmi, "Bachelor's degree")

# Run diagnosis for Sub-BA occupations  
print("\n\nSUB-BACHELOR'S DEGREE ANALYSIS")
print("=" * 80)
sub_ba_diagnostic = diagnose_jtm_filters(phoenix_lmi, "Sub-BA")

# Detailed analysis for low-count MSAs (BA)
print("\n" + "="*60)
print("BA - DETAILED ANALYSIS FOR LOW-COUNT MSAS")
print("="*60)

ba_msa_counts = ba_diagnostic.groupby('phoenix_msa').size().sort_values()
ba_low_count_msas = ba_msa_counts[ba_msa_counts < 9].index.tolist()

for msa in ba_low_count_msas:
    diagnose_jtm_filters(phoenix_lmi, "Bachelor's degree", msa)

# Detailed analysis for low-count MSAs (Sub-BA)
print("\n" + "="*60)
print("SUB-BA - DETAILED ANALYSIS FOR LOW-COUNT MSAS")
print("="*60)

sub_ba_msa_counts = sub_ba_diagnostic.groupby('phoenix_msa').size().sort_values()
sub_ba_low_count_msas = sub_ba_msa_counts[sub_ba_msa_counts < 9].index.tolist()

for msa in sub_ba_low_count_msas:
    diagnose_jtm_filters(phoenix_lmi, "Sub-BA", msa)

# Bottleneck analysis for both
print("\n" + "="*60)
print("BOTTLENECK ANALYSIS - BACHELOR'S DEGREE")
print("="*60)

ba_jobs = phoenix_lmi[phoenix_lmi['EDUCATION_LEVEL'] == "Bachelor's degree"]
print(f"\nLocal posting distribution for Bachelor's degree jobs:")
print(f"  Jobs with <30 local postings: {len(ba_jobs[ba_jobs['POSTINGS_OBS_LOCAL'] < 30])}")
print(f"  Jobs with 30-49 local postings: {len(ba_jobs[(ba_jobs['POSTINGS_OBS_LOCAL'] >= 30) & (ba_jobs['POSTINGS_OBS_LOCAL'] < 50)])}")
print(f"  Jobs with >=50 local postings: {len(ba_jobs[ba_jobs['POSTINGS_OBS_LOCAL'] >= 50])}")

print("\n" + "="*60)
print("BOTTLENECK ANALYSIS - SUB-BACHELOR'S DEGREE")
print("="*60)

sub_ba_levels = ['High school or GED', 'Associate degree', 'No Education Listed']
sub_ba_jobs = phoenix_lmi[phoenix_lmi['EDUCATION_LEVEL'].isin(sub_ba_levels)]
print(f"\nLocal posting distribution for Sub-BA jobs:")
print(f"  Jobs with <20 local postings: {len(sub_ba_jobs[sub_ba_jobs['POSTINGS_OBS_LOCAL'] < 20])}")
print(f"  Jobs with 20-29 local postings: {len(sub_ba_jobs[(sub_ba_jobs['POSTINGS_OBS_LOCAL'] >= 20) & (sub_ba_jobs['POSTINGS_OBS_LOCAL'] < 30)])}")
print(f"  Jobs with >=30 local postings: {len(sub_ba_jobs[sub_ba_jobs['POSTINGS_OBS_LOCAL'] >= 30])}")

# Salary vs living wage issues for both
for job_type, jobs_df in [("BA", ba_jobs), ("Sub-BA", sub_ba_jobs)]:
    threshold = 50 if job_type == "BA" else 30
    filtered_jobs = jobs_df[jobs_df['POSTINGS_OBS_LOCAL'] >= threshold]
    
    if len(filtered_jobs) > 0:
        salary_issues = filtered_jobs[filtered_jobs['salary_use'] < filtered_jobs['living_wage_metric']]
        print(f"\n{job_type} jobs failing salary >= living wage test: {len(salary_issues)}")
        if len(salary_issues) > 0:
            print(f"  Sample {job_type} occupations failing salary test:")
            for _, row in salary_issues.head(5).iterrows():
                print(f"    {row['SOC_5_NAME']}: ${row['salary_use']:,.0f} < ${row['living_wage_metric']:,.0f}")

BACHELOR'S DEGREE ANALYSIS

=== JTM Filter Diagnosis for Bachelor's degree ===
Starting dataset: 665 total occupations
After education filter ('Bachelor's degree'): 143 occupations
  → Dropped: 522 occupations
After local postings >= 50: 113 occupations
  → Dropped: 30 occupations
After national postings >= 30: 113 occupations
  → Dropped: 0 occupations
After OBS_GROWTH_NATIONAL > 0: 92 occupations
  → Dropped: 21 occupations
After obs_growth_local > 0: 90 occupations
  → Dropped: 2 occupations
After ECI_SOC5_ESTIMATED > 0: 89 occupations
  → Dropped: 1 occupations
After salary_use >= living_wage_metric: 54 occupations
  → Dropped: 35 occupations

Final BA JTMs: 54 occupations

BA JTMs by MSA:
  Phoenix–Mesa–Chandler: 54 JTMs


SUB-BACHELOR'S DEGREE ANALYSIS

=== JTM Filter Diagnosis for Sub-BA ===
Starting dataset: 665 total occupations
After education filter (Sub-BA (High school, Associate, No Education Listed)): 456 occupations
  → Dropped: 209 occupations
After local postings >= 30

In [229]:
# Weird occupations
# Check the specific surgical occupations
surgical_socs = ['29-2055', '29-9093']

# 1. Check what's in your final merged dataset
print("=== Final Dataset Check ===")
surgical_check = phoenix_lmi[phoenix_lmi['SOC_5'].isin(surgical_socs)][
    ['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL', 'EDUCATION_LEVEL_CODE']
].drop_duplicates()
print(surgical_check)

# 2. Check the national posting data
print("\n=== National Posting Data Check ===")
national_check = posting_data_national[posting_data_national['SOC_5'].isin(surgical_socs)][
    ['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL', 'EDUCATION_LEVEL_CODE']
]
print(national_check)

# 3. Check the original BLS benchmark
print("\n=== BLS Benchmark Check ===")
if 'bls_benchmark' in locals():
    # Check if these SOCs are in the BLS benchmark
    bls_check = bls_benchmark[bls_benchmark['soc_2021_5'].str.startswith('29-2055') | 
                             bls_benchmark['soc_2021_5'].str.startswith('29-9093')]
    print("BLS benchmark for surgical occupations:")
    print(bls_check)
else:
    print("BLS benchmark not available in current session")

# 4. Check what the original BLS data says about these occupations
print("\n=== Direct BLS Query Check ===")
bls_direct_check_sql = """
SELECT DISTINCT
    occ."soc_2018_6_code",
    occ."soc_2018_6_name", 
    occ."education"
FROM OUTSIDE_DATA.BLS.OCC_DEG_REQUIRE occ
WHERE occ."soc_2018_6_code" LIKE '29-2055%' 
   OR occ."soc_2018_6_code" LIKE '29-9093%'
ORDER BY occ."soc_2018_6_code"
"""
bls_direct_result = query_sf(bls_direct_check_sql)
print("Direct BLS education requirements:")
print(bls_direct_result)

# 5. Check crosswalk mapping
print("\n=== SOC Crosswalk Check ===")
crosswalk_check_sql = """
SELECT DISTINCT
    SOC_2019_5,
    SOC_2021_5,
    SOC_2021_5_NAME
FROM CROSSWALKS.CUSTOM.SOC_2019_SOC_2021_CROSSWALK
WHERE SOC_2019_5 LIKE '29-2055%' 
   OR SOC_2019_5 LIKE '29-9093%'
   OR SOC_2021_5 LIKE '29-2055%'
   OR SOC_2021_5 LIKE '29-9093%'
ORDER BY SOC_2021_5
"""
crosswalk_result = query_sf(crosswalk_check_sql)
print("SOC crosswalk mapping:")
print(crosswalk_result)

=== Final Dataset Check ===
      SOC_5              SOC_5_NAME     EDUCATION_LEVEL EDUCATION_LEVEL_CODE
59  29-9093     Surgical Assistants  High school or GED                    0
88  29-2055  Surgical Technologists  High school or GED                    0

=== National Posting Data Check ===
       SOC_5              SOC_5_NAME     EDUCATION_LEVEL EDUCATION_LEVEL_CODE
316  29-2055  Surgical Technologists  High school or GED                    0
328  29-9093     Surgical Assistants  High school or GED                    0

=== BLS Benchmark Check ===
BLS benchmark for surgical occupations:
    soc_2021_5  edulevel       edulevel_name
325    29-2055       0.0  High school or GED
337    29-9093       0.0  High school or GED

=== Direct BLS Query Check ===
Direct BLS education requirements:
  soc_2018_6_code         soc_2018_6_name                      education
0         29-2055  Surgical Technologists  Postsecondary nondegree award
1         29-9093     Surgical Assistants  Postsecond

In [230]:
# Define JTM identification function 

def identify_jtm(df, educ, metrics, cols):
    """Identify jobs that matter based on criteria"""
    filtered_df = df[
        (df['EDUCATION_LEVEL'].isin(educ)) &  
        (df['POSTINGS_OBS_LOCAL'] >= 30) &
        (df['POSTING_OBS_NATIONAL'] >= 30) &  
        (df['OBS_GROWTH_NATIONAL'] > 0) &          # Use raw growth variable, require positive growth
        (df['obs_growth_local'] > 0) &             # Use raw growth variable, require positive growth
        (df['ECI_SOC5_ESTIMATED'] > 0) &
        (df['salary_use'] >= df['living_wage_metric'])
    ].copy()
    
    # Create rank columns for each metric (global ranking like R)
    for metric in metrics:
        if metric in filtered_df.columns:
            filtered_df[f'{metric}_rank'] = filtered_df[metric].rank(ascending=False, method='min')
    
    return filtered_df

# Define sub-BA education levels
sub_ba = ['High school or GED', 'Associate degree', 'No Education Listed']

# Updated metrics to use raw growth variables instead of normalized
metrics = ['POSTING_OBS_NATIONAL', 'OBS_GROWTH_NATIONAL', 'POSTINGS_OBS_LOCAL',
           'obs_growth_local', 'ECI_SOC5_ESTIMATED', 'salary_use', 'mean_tot_earn']
occ_cols = ['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL']

# Identify sub-BA JTMs
sub_ba_jtm = identify_jtm(phoenix_lmi, sub_ba, metrics, occ_cols)

# # Filter out specific occupations (matching R code exactly)
# sub_ba_jtm = sub_ba_jtm[sub_ba_jtm['SOC_5'] != "29-1071"]  # PA exclusion only

In [231]:
ba_jtm_weighted

,phoenix_msa,jtm_rank,SOC_5,SOC_5_NAME,salary_use,salary_use_source,a_median_local,MEDIAN_SALARY_LOCAL,a_median,MEDIAN_SALARY_NATIONAL,...,composite_score,lp_tercile,lp_tercile_label,np_tercile,np_tercile_label,growth_tercile,growth_tercile_label,local_growth_tercile,local_growth_tercile_label,estimate_salary_growth
89,Phoenix–Mesa–Chandler,1.0,15-1252,Software Developers,110000.0,a_median_local,110000.0,130537.5,133080.0,138084.0,...,77.0,3,High,3,High,3,High Growth,3,High Local Growth,22.46 %
3,Phoenix–Mesa–Chandler,2.0,11-9111,Medical and Health Services Managers,80176.0,a_median_local,80176.0,80000.0,117960.0,90000.0,...,129.0,3,High,3,High,3,High Growth,3,High Local Growth,18.37 %
8,Phoenix–Mesa–Chandler,3.0,15-1212,Information Security Analysts,108000.0,a_median_local,108000.0,110000.0,124910.0,112000.0,...,130.0,2,Medium,2,Medium,3,High Growth,3,High Local Growth,22.73 %
219,Phoenix–Mesa–Chandler,4.0,17-2071,Electrical Engineers,120000.0,a_median_local,120000.0,115000.0,111910.0,114756.0,...,131.0,3,High,3,High,3,High Growth,2,Medium Local Growth,18.17 %
244,Phoenix–Mesa–Chandler,5.0,11-9041,Architectural and Engineering Managers,167544.0,a_median_local,167544.0,142500.0,167740.0,146500.0,...,132.0,3,High,3,High,1,Low Growth,2,Medium Local Growth,19.82 %
45,Phoenix–Mesa–Chandler,6.0,17-2112,Industrial Engineers,100000.0,a_median_local,100000.0,109200.0,101140.0,106472.0,...,138.0,3,High,3,High,3,High Growth,3,High Local Growth,17.72 %
178,Phoenix–Mesa–Chandler,7.0,15-1241,Computer Network Architects,101000.0,a_median_local,101000.0,112500.0,130390.0,118466.0,...,140.0,3,High,3,High,3,High Growth,3,High Local Growth,22.3 %
69,Phoenix–Mesa–Chandler,8.0,11-3021,Computer and Information Systems Managers,120000.0,a_median_local,120000.0,150000.0,171200.0,166400.0,...,149.0,2,Medium,2,Medium,3,High Growth,3,High Local Growth,12.98 %
224,Phoenix–Mesa–Chandler,9.0,17-2072,"Electronics Engineers, Except Computer",120000.0,a_median_local,120000.0,122339.5,127590.0,123500.0,...,154.0,2,Medium,2,Medium,3,High Growth,2,Medium Local Growth,18.17 %
80,Phoenix–Mesa–Chandler,10.0,17-2141,Mechanical Engineers,100000.0,a_median_local,100000.0,110000.0,102320.0,108500.0,...,160.0,3,High,3,High,3,High Growth,3,High Local Growth,17.83 %


In [232]:
# Create Output Dataset – BA JTMs (Weighted Version)

ba_jtm_weighted_output = ba_jtm_weighted[[
    'phoenix_msa', 'jtm_rank', 'SOC_5', 'SOC_5_NAME', 
    'lp_tercile_label', 'EDUCATION_LEVEL', 'salary_use', 
    'np_tercile_label', 'estimate_salary_growth', 'growth_tercile_label',
    'local_growth_tercile_label'
]].copy()

# Rename columns to match R output
ba_jtm_weighted_output.columns = [
    "Phoenix MSA", "JTM Ranking", "Occupation Code", "Occupation", "Local Demand",
    "Education Level Needed", "Estimated Salary", "National Demand", 
    "Salary Growth Estimate", "National Demand Growth", "Local Demand Growth"
]

In [233]:
ba_jtm_weighted_output

,Phoenix MSA,JTM Ranking,Occupation Code,Occupation,Local Demand,Education Level Needed,Estimated Salary,National Demand,Salary Growth Estimate,National Demand Growth,Local Demand Growth
89,Phoenix–Mesa–Chandler,1.0,15-1252,Software Developers,High,Bachelor's degree,110000.0,High,22.46 %,High Growth,High Local Growth
3,Phoenix–Mesa–Chandler,2.0,11-9111,Medical and Health Services Managers,High,Bachelor's degree,80176.0,High,18.37 %,High Growth,High Local Growth
8,Phoenix–Mesa–Chandler,3.0,15-1212,Information Security Analysts,Medium,Bachelor's degree,108000.0,Medium,22.73 %,High Growth,High Local Growth
219,Phoenix–Mesa–Chandler,4.0,17-2071,Electrical Engineers,High,Bachelor's degree,120000.0,High,18.17 %,High Growth,Medium Local Growth
244,Phoenix–Mesa–Chandler,5.0,11-9041,Architectural and Engineering Managers,High,Bachelor's degree,167544.0,High,19.82 %,Low Growth,Medium Local Growth
45,Phoenix–Mesa–Chandler,6.0,17-2112,Industrial Engineers,High,Bachelor's degree,100000.0,High,17.72 %,High Growth,High Local Growth
178,Phoenix–Mesa–Chandler,7.0,15-1241,Computer Network Architects,High,Bachelor's degree,101000.0,High,22.3 %,High Growth,High Local Growth
69,Phoenix–Mesa–Chandler,8.0,11-3021,Computer and Information Systems Managers,Medium,Bachelor's degree,120000.0,Medium,12.98 %,High Growth,High Local Growth
224,Phoenix–Mesa–Chandler,9.0,17-2072,"Electronics Engineers, Except Computer",Medium,Bachelor's degree,120000.0,Medium,18.17 %,High Growth,Medium Local Growth
80,Phoenix–Mesa–Chandler,10.0,17-2141,Mechanical Engineers,High,Bachelor's degree,100000.0,High,17.83 %,High Growth,High Local Growth


In [234]:
def investigate_missing_occupation(soc_code, msa_name, df_full, df_jtm, education_levels, verbose=True):
    """
    Diagnoses why a SOC_5 code might not appear in the final JTM list.
    Checks the row in the full phoenix_lmi data and each filter in the JTM pipeline.
    """
    row = df_full[(df_full['SOC_5'] == soc_code) & (df_full['phoenix_msa'] == msa_name)]
    
    if row.empty:
        print(f"❌ Occupation {soc_code} not found in MSA '{msa_name}' in phoenix_lmi.")
        return
    
    if verbose:
        print(f"\n✅ Found {soc_code} in MSA '{msa_name}' in phoenix_lmi. Summary of filters:")
        print(f"   Occupation: {row['SOC_5_NAME'].values[0]}")
    
    # Apply each filter condition and show result (updated to use raw growth variables and correct thresholds)
    education_ok = row['EDUCATION_LEVEL'].isin(education_levels).values[0]
    postings_local_ok = row['POSTINGS_OBS_LOCAL'].values[0] >= 30
    postings_national_ok = row['POSTING_OBS_NATIONAL'].values[0] >= 30
    growth_ok = row['OBS_GROWTH_NATIONAL'].values[0] > 0          # Updated: use raw growth > 0
    local_growth_ok = row['obs_growth_local'].values[0] > 0       # Updated: use raw growth > 0
    eci_ok = row['ECI_SOC5_ESTIMATED'].values[0] > -5            # Updated: correct threshold
    salary_ok = row['salary_use'].values[0] >= row['living_wage_metric'].values[0]
    
    flags = {
        "EDUCATION_LEVEL": education_ok,
        "POSTINGS_OBS_LOCAL >= 30": postings_local_ok,
        "POSTING_OBS_NATIONAL >= 30": postings_national_ok,
        "OBS_GROWTH_NATIONAL > 0": growth_ok,                    # Updated label
        "obs_growth_local > 0": local_growth_ok,                 # Updated label
        "ECI_SOC5_ESTIMATED > -5": eci_ok,                       # Updated label
        "salary_use >= living_wage_metric": salary_ok
    }
    
    # Show actual values for failed filters
    for k, v in flags.items():
        status = '✅' if v else '❌'
        if k == "EDUCATION_LEVEL":
            actual_value = row['EDUCATION_LEVEL'].values[0]
            print(f"  {k:<40} {status} (Actual: {actual_value})")
        elif k == "POSTINGS_OBS_LOCAL >= 30":
            actual_value = row['POSTINGS_OBS_LOCAL'].values[0]
            print(f"  {k:<40} {status} (Actual: {actual_value})")
        elif k == "POSTING_OBS_NATIONAL >= 30":
            actual_value = row['POSTING_OBS_NATIONAL'].values[0]
            print(f"  {k:<40} {status} (Actual: {actual_value})")
        elif k == "OBS_GROWTH_NATIONAL > 0":                     # Updated
            actual_value = row['OBS_GROWTH_NATIONAL'].values[0]
            print(f"  {k:<40} {status} (Actual: {actual_value:.2f})")
        elif k == "obs_growth_local > 0":                        # Updated
            actual_value = row['obs_growth_local'].values[0]
            print(f"  {k:<40} {status} (Actual: {actual_value:.2f})")
        elif k == "ECI_SOC5_ESTIMATED > -5":
            actual_value = row['ECI_SOC5_ESTIMATED'].values[0]
            print(f"  {k:<40} {status} (Actual: {actual_value:.3f})")
        elif k == "salary_use >= living_wage_metric":
            salary_val = row['salary_use'].values[0]
            living_wage_val = row['living_wage_metric'].values[0]
            print(f"  {k:<40} {status} (Salary: ${salary_val:,.0f}, Living Wage: ${living_wage_val:,.0f})")
    
    # Final check: was it in the JTM dataset?
    in_final = not df_jtm[(df_jtm['SOC_5'] == soc_code) & (df_jtm['phoenix_msa'] == msa_name)].empty
    print(f"\nIn final JTM list? {'✅ Yes' if in_final else '❌ No'}")
    
    # Show which filter(s) failed
    failed_filters = [k for k, v in flags.items() if not v]
    if failed_filters:
        print(f"❌ FAILED FILTERS: {', '.join(failed_filters)}")
    else:
        print("✅ ALL FILTERS PASSED")

def investigate_occupation_list(soc_list, msa_name, df_full, df_jtm, education_levels, education_type="Sub-BA"):
    """
    Investigate a list of occupations and show which filters are failing for each.
    """
    print(f"\n{'='*80}")
    print(f"INVESTIGATING {education_type} OCCUPATIONS IN {msa_name}")
    print(f"{'='*80}")
    
    for soc_code in soc_list:
        investigate_missing_occupation(soc_code, msa_name, df_full, df_jtm, education_levels, verbose=True)
        print("-" * 80)

# Example usage for Phoenix:
# Define education levels
sub_ba_levels = ['High school or GED', 'Associate degree', 'No Education Listed']
ba_levels = ["Bachelor's degree"]

# Separate occupation lists for each education level
sub_ba_occupations_to_check = [
    "53-4041",  # Subway and Streetcar Operators
    "51-8021",  # Stationary Engineers and Boiler Operators
    "53-4099",  # Rail Transportation Workers, All Other
    "53-4022",  # Railroad Brake, Signal, and Switch Operators
    "13-1081",  # Logisticans
    "33-2022",  # Forest Fire Inspectors and Prevention Specialists
    "47-1011",  # First-Line Supervisors of Construction Trades
    "11-3071",  # Transportation, Storages, and Distribution Manager
    "33-1099",  # First-Line Supervisors of Protective Service Workers
    "47-2072",  # Pile Driver Operators
    "53-4011",  # Locomotive Engineers
    "47-2221",  # Structural Iron and Steel Workers
    "53-6051",  # Transportation Inspectors
    "53-1041",  # Aircraft Cargo Handling Supervisors
    "29-2091",  # Orthotists and Prosthetists
]

ba_occupations_to_check = [
    "29-2036",  # Medical Dosimetrists
    "29-1024",  # Prosthodontists
    "25-1071",  # Health Specialties Teachers, Postsecondary
    "25-1032",  # Engineering Teachers, Postsecondary
    "19-5012",  # Occupational Health and Safety Technicians
    "13-1023",  # Purchasing Agents, Except Wholesale, Retial 
    "19-3032",  # Industrial-Organizational Psychologists
    "29-2012",  # Medical and Clinical Laboratory Technicians
]

# Run investigations for Sub-BA occupations
print("INVESTIGATING SUB-BA OCCUPATIONS:")
investigate_occupation_list(
    sub_ba_occupations_to_check, 
    "Phoenix–Mesa–Chandler", 
    phoenix_lmi, 
    sub_ba_jtm, 
    sub_ba_levels, 
    "Sub-BA"
)

# Run investigations for BA occupations
print("\n\nINVESTIGATING BA OCCUPATIONS:")
investigate_occupation_list(
    ba_occupations_to_check, 
    "Phoenix–Mesa–Chandler", 
    phoenix_lmi, 
    ba_jtm_original,  # Use ba_jtm_original
    ba_levels, 
    "BA"
)

INVESTIGATING SUB-BA OCCUPATIONS:

INVESTIGATING Sub-BA OCCUPATIONS IN Phoenix–Mesa–Chandler

✅ Found 53-4041 in MSA 'Phoenix–Mesa–Chandler' in phoenix_lmi. Summary of filters:
   Occupation: Subway and Streetcar Operators
  EDUCATION_LEVEL                          ✅ (Actual: High school or GED)
  POSTINGS_OBS_LOCAL >= 30                 ✅ (Actual: 58)
  POSTING_OBS_NATIONAL >= 30               ✅ (Actual: 3564)
  OBS_GROWTH_NATIONAL > 0                  ✅ (Actual: 4.20)
  obs_growth_local > 0                     ✅ (Actual: 16.78)
  ECI_SOC5_ESTIMATED > -5                  ✅ (Actual: 0.191)
  salary_use >= living_wage_metric         ✅ (Salary: $77,872, Living Wage: $53,435)

In final JTM list? ✅ Yes
✅ ALL FILTERS PASSED
--------------------------------------------------------------------------------

✅ Found 51-8021 in MSA 'Phoenix–Mesa–Chandler' in phoenix_lmi. Summary of filters:
   Occupation: Stationary Engineers and Boiler Operators
  EDUCATION_LEVEL                          ✅ (Act

In [235]:
ba_jtm_weighted_output

,Phoenix MSA,JTM Ranking,Occupation Code,Occupation,Local Demand,Education Level Needed,Estimated Salary,National Demand,Salary Growth Estimate,National Demand Growth,Local Demand Growth
89,Phoenix–Mesa–Chandler,1.0,15-1252,Software Developers,High,Bachelor's degree,110000.0,High,22.46 %,High Growth,High Local Growth
3,Phoenix–Mesa–Chandler,2.0,11-9111,Medical and Health Services Managers,High,Bachelor's degree,80176.0,High,18.37 %,High Growth,High Local Growth
8,Phoenix–Mesa–Chandler,3.0,15-1212,Information Security Analysts,Medium,Bachelor's degree,108000.0,Medium,22.73 %,High Growth,High Local Growth
219,Phoenix–Mesa–Chandler,4.0,17-2071,Electrical Engineers,High,Bachelor's degree,120000.0,High,18.17 %,High Growth,Medium Local Growth
244,Phoenix–Mesa–Chandler,5.0,11-9041,Architectural and Engineering Managers,High,Bachelor's degree,167544.0,High,19.82 %,Low Growth,Medium Local Growth
45,Phoenix–Mesa–Chandler,6.0,17-2112,Industrial Engineers,High,Bachelor's degree,100000.0,High,17.72 %,High Growth,High Local Growth
178,Phoenix–Mesa–Chandler,7.0,15-1241,Computer Network Architects,High,Bachelor's degree,101000.0,High,22.3 %,High Growth,High Local Growth
69,Phoenix–Mesa–Chandler,8.0,11-3021,Computer and Information Systems Managers,Medium,Bachelor's degree,120000.0,Medium,12.98 %,High Growth,High Local Growth
224,Phoenix–Mesa–Chandler,9.0,17-2072,"Electronics Engineers, Except Computer",Medium,Bachelor's degree,120000.0,Medium,18.17 %,High Growth,Medium Local Growth
80,Phoenix–Mesa–Chandler,10.0,17-2141,Mechanical Engineers,High,Bachelor's degree,100000.0,High,17.83 %,High Growth,High Local Growth


In [236]:
sub_ba_jtm_ww_output

,Phoenix MSA,JTM Ranking,Occupation Code,Occupation,Local Demand,Education Level Needed,Estimated Salary,National Demand,Salary Growth Estimate,National Demand Growth
10,Phoenix–Mesa–Chandler,1.0,29-2032,Diagnostic Medical Sonographers,Medium,Associate degree,75000.0,High,17.96 %,High Growth
27,Phoenix–Mesa–Chandler,2.0,29-1126,Respiratory Therapists,Medium,Associate degree,66074.0,High,21.46 %,High Growth
14,Phoenix–Mesa–Chandler,3.0,49-9062,Medical Equipment Repairers,Low,Associate degree,61354.0,Low,20.71 %,High Growth
24,Phoenix–Mesa–Chandler,4.0,29-2035,Magnetic Resonance Imaging Technologists,Medium,Associate degree,93713.0,High,19.89 %,High Growth
23,Phoenix–Mesa–Chandler,5.0,49-9041,Industrial Machinery Mechanics,Medium,High school or GED,60000.0,Medium,20.6 %,High Growth
128,Phoenix–Mesa–Chandler,6.0,11-3071,"Transportation, Storage, and Distribution Mana...",High,High school or GED,59351.0,High,10.79 %,High Growth
87,Phoenix–Mesa–Chandler,7.0,29-2034,Radiologic Technologists and Technicians,High,Associate degree,63714.0,High,17.82 %,Medium Growth
142,Phoenix–Mesa–Chandler,8.0,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",High,High school or GED,56275.0,High,19.64 %,High Growth
92,Phoenix–Mesa–Chandler,9.0,47-1011,First-Line Supervisors of Construction Trades ...,High,High school or GED,58994.0,Medium,10.3 %,High Growth
215,Phoenix–Mesa–Chandler,10.0,19-5012,Occupational Health and Safety Technicians,High,High school or GED,56000.0,High,26.0 %,High Growth


In [237]:
# Enhanced Sub-BA output with all terciles and wage variables
sub_ba_jtm_ww_output = sub_ba_jtm_ww[[
    'phoenix_msa', 'jtm_rank', 'SOC_5', 'SOC_5_NAME', 
    'EDUCATION_LEVEL', 'salary_use', 'salary_use_source',
    # All tercile types including local growth
    'lp_tercile_label', 'np_tercile_label', 
    'growth_tercile_label', 'local_growth_tercile_label',
    'estimate_salary_growth',
    # Additional wage variables
    'a_median_local', 'MEDIAN_SALARY_LOCAL', 'a_median', 'MEDIAN_SALARY_NATIONAL', 'mean_tot_earn',
    # Local employment
    'tot_emp_local',
    # ACS 55+ variables 
    'tot_emp_55_plus', 'share_55_plus', 'few_obs_flag',
    # AI exposure
    'LLM_EXPOSURE'
]].copy()

# Rename columns for clean output
sub_ba_jtm_ww_output.columns = [
    "Phoenix MSA", "JTM Ranking", "Occupation Code", "Occupation", 
    "Education Level Needed", "Estimated Salary", "Salary Source",
    "Local Demand", "National Demand", 
    "National Growth", "Local Growth",
    "Salary Growth Estimate",
    "ACS Local Wage", "Local Posted Wage", "OEWS National Wage", "National Posted Wage", "6-Year Total Earnings",
    "Local Employment", "Employment 55+", "Share 55+", "Few Observations Flag", "LLM Risk"
]

# Enhanced BA output with all terciles
ba_columns = [
    'phoenix_msa', 'jtm_rank', 'SOC_5', 'SOC_5_NAME', 
    'EDUCATION_LEVEL', 'salary_use', 'salary_use_source',
    # All tercile types including local growth
    'lp_tercile_label', 'np_tercile_label',
    'growth_tercile_label', 'local_growth_tercile_label',
    'estimate_salary_growth',
    'a_median_local', 'MEDIAN_SALARY_LOCAL', 'a_median', 'MEDIAN_SALARY_NATIONAL', 'mean_tot_earn',
    'tot_emp_local', 'tot_emp_55_plus', 'share_55_plus', 'few_obs_flag',
    'LLM_EXPOSURE'
]

ba_available_columns = [col for col in ba_columns if col in ba_jtm_weighted.columns]
ba_jtm_weighted_output = ba_jtm_weighted[ba_available_columns].copy()

# Rename BA columns (same structure as Sub-BA)
ba_column_names = [
    "Phoenix MSA", "JTM Ranking", "Occupation Code", "Occupation", 
    "Education Level Needed", "Estimated Salary", "Salary Source",
    "Local Demand", "National Demand", 
    "National Growth", "Local Growth",
    "Salary Growth Estimate",
    "ACS Local Wage", "Local Posted Wage", "OEWS National Wage", "National Posted Wage", "6-Year Total Earnings",
    "Local Employment", "Employment 55+", "Share 55+", "Few Observations Flag", "LLM Risk"
]

ba_readable_columns = [ba_column_names[ba_columns.index(col)] for col in ba_available_columns]
ba_jtm_weighted_output.columns = ba_readable_columns

In [238]:
# Data Dictionary for Enhanced Output (Updated with correct 4 terciles + Few Obs Flag + LLM Risk)
data_dict_df = pd.DataFrame({
    'Column_Name': [
        'Phoenix MSA', 'JTM Ranking', 'Occupation Code', 'Occupation',
        'Education Level Needed', 'Estimated Salary', 'Salary Source',
        'Local Demand', 'National Demand', 
        'National Growth', 'Local Growth',
        'Salary Growth Estimate',
        'ACS Local Wage', 'Local Posted Wage', 'OEWS National Wage', 'National Posted Wage', 
        '6-Year Total Earnings', 'Local Employment', 'Employment 55+', 'Share 55+', 'Few Observations Flag',
        'LLM Risk'
    ],
    'Description': [
        'Metropolitan Statistical Area (Phoenix-Mesa-Chandler)',
        'Jobs That Matter ranking (1 = best)',
        '5-digit Standard Occupational Classification code',
        'Occupation title',
        'Education level required',
        'Final wage used in analysis (see wage hierarchy)', 
        'Source of wage data used',
        'Local job postings tercile (Low/Medium/High)',
        'National job postings tercile (Low/Medium/High)',
        'National BLS 10-year employment growth tercile (Low Growth/Medium Growth/High Growth)',
        'Local employment growth tercile (Low Local Growth/Medium Local Growth/High Local Growth)',
        'Formatted wage growth percentage',
        'Phoenix ACS median annual wage', 'Phoenix job posting median wage',
        'National OEWS median annual wage', 'National job posting median wage',
        'Lightcast ONET mobility 6-year earnings', 'Estimated local employment (ACS)',
        'National employment age 55+ (ACS)', 'Share of workers age 55+ (ACS)',
        'Indicates if ACS employment estimates have few observations',
        'LLM exposure score normalized to mean=0. Positive values indicate above-average AI exposure risk'
    ],
    'Source': [
        'Geographic classification', 'Calculated composite score', 'Bureau of Labor Statistics',
        'Bureau of Labor Statistics', 'Job postings + BLS benchmark',
        'Wage hierarchy', 'Calculated from hierarchy',
        'Job Postings (POSTINGS_OBS_LOCAL)', 'Job Postings (POSTING_OBS_NATIONAL)',
        'Bureau of Labor Statistics (OBS_GROWTH_NATIONAL)', 'Arizona State Growth Data (OBS_GROWTH_LOCAL)',
        'Employment Cost Index',
        'American Community Survey', 'Job Postings (Burning Glass)',
        'OEWS (Occupational Employment & Wage Statistics)', 'Job Postings (Burning Glass)',
        'Lightcast ONET Mobility', 'American Community Survey',
        'American Community Survey', 'American Community Survey', 'American Community Survey',
        'Felten et al. AI Occupation Exposure'
    ]
})

# Simple Wage Hierarchy
wage_hierarchy_df = pd.DataFrame({
    'Priority': [1, 2, 3, 4],
    'Wage_Source': [
        'ACS Local Wage',
        'Local Posted Wage', 
        'OEWS National Wage',
        'National Posted Wage'
    ],
    'Description': [
        'Phoenix ACS median wage',
        'Phoenix job posting median wage',
        'National OEWS median wage',
        'National job posting median wage'
    ]
})

In [239]:
def update_salary_source_names(df):
    """Convert technical salary source names to user-friendly names"""
    
    # Create mapping from technical names to user-friendly names
    salary_source_mapping = {
        'a_median_local': 'ACS Local Wage',
        'MEDIAN_SALARY_LOCAL': 'Local Posted Wage',
        'a_median': 'OEWS National Wage', 
        'MEDIAN_SALARY_NATIONAL': 'National Posted Wage'
    }
    
    # Update the salary source column
    if 'Salary Source' in df.columns:
        df['Salary Source'] = df['Salary Source'].map(salary_source_mapping).fillna(df['Salary Source'])
    
    return df

# Update Sub-BA output
sub_ba_jtm_ww_output = update_salary_source_names(sub_ba_jtm_ww_output)

# Update BA output if it exists
ba_jtm_weighted_output = update_salary_source_names(ba_jtm_weighted_output)




In [240]:
sub_ba_jtm_ww_output

,Phoenix MSA,JTM Ranking,Occupation Code,Occupation,Education Level Needed,Estimated Salary,Salary Source,Local Demand,National Demand,National Growth,...,ACS Local Wage,Local Posted Wage,OEWS National Wage,National Posted Wage,6-Year Total Earnings,Local Employment,Employment 55+,Share 55+,Few Observations Flag,LLM Risk
10,Phoenix–Mesa–Chandler,1.0,29-2032,Diagnostic Medical Sonographers,Associate degree,75000.0,ACS Local Wage,Medium,High,High Growth,...,75000.0,109200.0,89340.0,114400.0,655731.869306,950.0,176.0,0.147280,NaN,-0.274320
27,Phoenix–Mesa–Chandler,2.0,29-1126,Respiratory Therapists,Associate degree,66074.0,ACS Local Wage,Medium,High,High Growth,...,66074.0,81733.0,80450.0,93048.0,605324.383706,1324.0,328.0,0.218230,NaN,-0.199421
14,Phoenix–Mesa–Chandler,3.0,49-9062,Medical Equipment Repairers,Associate degree,61354.0,ACS Local Wage,Low,Low,High Growth,...,61354.0,60000.0,62630.0,62920.0,506821.569483,612.0,325.0,0.494673,NaN,-0.536776
24,Phoenix–Mesa–Chandler,4.0,29-2035,Magnetic Resonance Imaging Technologists,Associate degree,93713.0,ACS Local Wage,Medium,High,High Growth,...,93713.0,122772.0,88180.0,122200.0,731379.626354,816.0,192.0,0.228844,NaN,-0.136628
23,Phoenix–Mesa–Chandler,5.0,49-9041,Industrial Machinery Mechanics,High school or GED,60000.0,ACS Local Wage,Medium,Medium,High Growth,...,60000.0,55120.0,63760.0,58853.5,365459.974597,3006.0,847.0,0.244868,NaN,-1.217021
128,Phoenix–Mesa–Chandler,6.0,11-3071,"Transportation, Storage, and Distribution Mana...",High school or GED,59351.0,ACS Local Wage,High,High,High Growth,...,59351.0,73101.0,102010.0,75000.0,542322.633012,4035.0,1045.0,0.237716,NaN,0.904548
87,Phoenix–Mesa–Chandler,7.0,29-2034,Radiologic Technologists and Technicians,Associate degree,63714.0,ACS Local Wage,High,High,Medium Growth,...,63714.0,103405.0,77660.0,109564.0,382284.000000,2305.0,539.0,0.222911,NaN,-0.404834
142,Phoenix–Mesa–Chandler,8.0,49-3042,"Mobile Heavy Equipment Mechanics, Except Engines",High school or GED,56275.0,ACS Local Wage,High,High,High Growth,...,56275.0,63960.0,63980.0,61290.0,438144.831959,2382.0,921.0,0.322028,NaN,-1.258737
92,Phoenix–Mesa–Chandler,9.0,47-1011,First-Line Supervisors of Construction Trades ...,High school or GED,58994.0,ACS Local Wage,High,Medium,High Growth,...,58994.0,74880.0,78690.0,70200.0,472890.478829,9097.0,2281.0,0.206407,NaN,-0.359874
215,Phoenix–Mesa–Chandler,10.0,19-5012,Occupational Health and Safety Technicians,High school or GED,56000.0,ACS Local Wage,High,High,High Growth,...,56000.0,72800.0,58440.0,67600.0,336000.000000,675.0,142.0,0.165888,NaN,0.328964


In [241]:
# Define target industries
target_naics = {
    'Construction': ['23'],
    'Health': ['62'],  # Just core healthcare
    'Manufacturing': ['31', '32', '33', '34', '35', '36'],  # Core manufacturing
    'Technology': ['51'],  # Just core tech/information
    'Transportation': ['48', '49']
}

# Flatten to list of all target NAICS codes
all_target_naics = [code for codes in target_naics.values() for code in codes]

# Query data
query = """
SELECT 
    SOC_2021_5,
    SOC_2021_5_NAME,
    NAICS_2022_2,
    COUNT(DISTINCT ID) as POSTINGS
FROM EMSI_BURNING_GLASS_INSTITUTE.US.POSTINGS
WHERE POSTED >= '2021-01-01'
    AND POSTED < '2025-01-01'
    AND SOC_2021_5 IS NOT NULL
    AND NAICS_2022_2 IS NOT NULL
    AND ARRAY_CONTAINS('Company'::VARIANT, SOURCE_TYPES)
GROUP BY SOC_2021_5, SOC_2021_5_NAME, NAICS_2022_2
"""

df = pd.read_sql(query, conn)

# Add industry category
def get_industry_category(naics):
    for industry, codes in target_naics.items():
        if naics in codes:
            return industry
    return 'Other'

df['industry'] = df['NAICS_2022_2'].apply(get_industry_category)

# Filter to target industries only
df_target = df[df['industry'] != 'Other'].copy()

# Calculate totals
occupation_totals = df_target.groupby(['SOC_2021_5', 'SOC_2021_5_NAME'])['POSTINGS'].sum().reset_index()
occupation_totals.columns = ['SOC_2021_5', 'SOC_2021_5_NAME', 'total_postings']

# Calculate industry shares for each occupation
df_final = df_target.merge(occupation_totals, on=['SOC_2021_5', 'SOC_2021_5_NAME'])
df_final['industry_share'] = df_final['POSTINGS'] / df_final['total_postings']

# Calculate overall industry distribution
industry_totals = df_target.groupby('industry')['POSTINGS'].sum()
total_all_postings = industry_totals.sum()
industry_overall_share = industry_totals / total_all_postings

# Add overall industry shares to dataframe
df_final = df_final.merge(
    industry_overall_share.reset_index().rename(columns={'POSTINGS': 'industry_overall_share'}),
    on='industry'
)

# Calculate likelihood ratio
df_final['likelihood_ratio'] = df_final['industry_share'] / df_final['industry_overall_share']

# REMOVED: Keep only occupations with high likelihood (2x more likely than average)
# df_concentrated = df_final[df_final['likelihood_ratio'] >= 1.8].copy()

# Select final columns - now includes ALL occupations regardless of likelihood ratio
result_df = df_final[['SOC_2021_5', 'SOC_2021_5_NAME', 'industry', 'likelihood_ratio', 'POSTINGS', 'total_postings']].copy()
result_df = result_df.sort_values('likelihood_ratio', ascending=False)


/tmp/ipykernel_54907/184860788.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [242]:
print("Columns in sub_ba_jtm_ww_output:")
print(sub_ba_jtm_ww_output.columns.tolist())

print("\nColumns in ba_jtm_weighted_output:")
print(ba_jtm_weighted_output.columns.tolist())

Columns in sub_ba_jtm_ww_output:
['Phoenix MSA', 'JTM Ranking', 'Occupation Code', 'Occupation', 'Education Level Needed', 'Estimated Salary', 'Salary Source', 'Local Demand', 'National Demand', 'National Growth', 'Local Growth', 'Salary Growth Estimate', 'ACS Local Wage', 'Local Posted Wage', 'OEWS National Wage', 'National Posted Wage', '6-Year Total Earnings', 'Local Employment', 'Employment 55+', 'Share 55+', 'Few Observations Flag', 'LLM Risk']

Columns in ba_jtm_weighted_output:
['Phoenix MSA', 'JTM Ranking', 'Occupation Code', 'Occupation', 'Education Level Needed', 'Estimated Salary', 'Salary Source', 'Local Demand', 'National Demand', 'National Growth', 'Local Growth', 'Salary Growth Estimate', 'ACS Local Wage', 'Local Posted Wage', 'OEWS National Wage', 'National Posted Wage', '6-Year Total Earnings', 'Local Employment', 'Employment 55+', 'Share 55+', 'Few Observations Flag', 'LLM Risk']


In [243]:
# Create mapping functions (KEEP)
def get_industry_mappings(likelihood_df):
    primary_industry = (
        likelihood_df
        .loc[likelihood_df.groupby('SOC_2021_5')['likelihood_ratio'].idxmax()][['SOC_2021_5', 'industry', 'likelihood_ratio']]
    )
    primary_mapping = dict(zip(primary_industry['SOC_2021_5'], primary_industry['industry']))

    all_industries_mapping = {}
    for _, row in likelihood_df.iterrows():
        occ_code = row['SOC_2021_5']
        if occ_code not in all_industries_mapping:
            all_industries_mapping[occ_code] = []
        all_industries_mapping[occ_code].append({
            'industry': row['industry'],
            'likelihood_ratio': row['likelihood_ratio']
        })

    return primary_mapping, all_industries_mapping

occupation_to_industry, all_industries_mapping = get_industry_mappings(result_df)

# Manual sector overrides dictionary (keep this - already defined early)
manual_sector_overrides = {
    '15-1211': 'Technology',  # Computer Systems Analysts
    '11-9111': 'Healthcare',  # Medical and Health Services Managers
    '49-9062': 'Healthcare',  # Medical Equipment Repairers
    '41-9031': 'Tech-Adjacent', # Sales Engineers
    '13-1161': 'Tech-Adjacent',  # Market Research Analysts and Marketing Specialists
    '41-4011': 'Tech-Adjacent', # Sales Representatives, Wholesale and Manufacturing, Technical and Scientific Products
    '41-3091': 'Tech-Adjacent', # Sales Representatives of Services, Except Advertising, Insurance, Financial Services, and Travel
    '41-1012': 'Tech-Adjacent',  # First-Line Supervisors of Non-Retail Sales Workers
    '29-1299': 'Healthcare',  # Healthcare Diagnosing or Treating Practitioners
    '29-2061': 'Healthcare',  # Licensed Practical and Licensed Vocational Nurses	
    '29-2081': 'Healthcare',  # Opticians, Dispensing
    '29-2099': 'Healthcare',  # Health Technologists and Technicians
    '29-2092': 'Healthcare',  # Hearing Aid Specialists
    '31-2012': 'Healthcare',  # Occupational Therapy Aides
    '31-2011': 'Healthcare',  # Occupational Therapy Assistants
    '29-2072': 'Healthcare',  # Medical Records Specialists
    '31-9095': 'Healthcare',  # Pharmacy Aides
    '21-1094': 'Healthcare',  # Community Health Workers
}

# Assign sectors - manual overrides take priority (KEEP)
def assign_sector_by_likelihood(soc_code):
    if pd.isna(soc_code):
        return 'Other'

    soc_str = str(soc_code)
    
    # Check manual overrides FIRST
    if soc_str in manual_sector_overrides:
        return manual_sector_overrides[soc_str]

    # If not in manual overrides, use likelihood-based assignment
    if soc_str in occupation_to_industry:
        occ_data = result_df[result_df['SOC_2021_5'] == soc_str]
        if occ_data.empty:
            return 'Other'

        max_likelihood_row = occ_data.loc[occ_data['likelihood_ratio'].idxmax()]
        max_likelihood_ratio = max_likelihood_row['likelihood_ratio']
        industry = max_likelihood_row['industry']

        if max_likelihood_ratio >= 1.8:
            industry_to_sector = {
                'Construction': 'Construction',
                'Health': 'Healthcare',
                'Manufacturing': 'Manufacturing',
                'Technology': 'Technology',
                'Transportation': 'Transportation & Distribution'
            }
            return industry_to_sector.get(industry, 'Other')

    return 'Other'

In [244]:
# Create JTMs that meet all criteria EXCEPT living wage threshold
PHOENIX_LIVING_WAGE = 47269.6

def identify_jtm_no_living_wage(df, educ, metrics, cols):
    """Identify jobs that matter based on criteria - EXCLUDING living wage threshold"""
    filtered_df = df[
        (df['EDUCATION_LEVEL'].isin(educ)) &  
        (df['POSTINGS_OBS_LOCAL'] >= 30) &
        (df['POSTING_OBS_NATIONAL'] >= 30) &  
        (df['OBS_GROWTH_NATIONAL'] > 0) &          
        (df['obs_growth_local'] > 0) &             
        (df['ECI_SOC5_ESTIMATED'] > 0)
        # Removed: (df['salary_use'] >= df['living_wage_metric'])
    ].copy()
    
    # Create rank columns for each metric
    for metric in metrics:
        if metric in filtered_df.columns:
            filtered_df[f'{metric}_rank'] = filtered_df[metric].rank(ascending=False, method='min')
    
    return filtered_df

def add_terciles_and_rankings(df):
    """Add tercile calculations and JTM rankings to dataframe"""
    df_with_terciles = df.copy()
    
    # Calculate JTM rank using composite score (same as in your tercile code)
    df_with_terciles['composite_score'] = (
        df_with_terciles['POSTING_OBS_NATIONAL_rank'] +
        df_with_terciles['OBS_GROWTH_NATIONAL_rank'] +
        df_with_terciles['POSTINGS_OBS_LOCAL_rank'] +
        df_with_terciles['obs_growth_local_rank'] +
        3 * df_with_terciles['salary_use_rank'] +       # 3x weight on salary
        df_with_terciles['ECI_SOC5_ESTIMATED_rank'] +
        df_with_terciles['mean_tot_earn_rank']
    )
    
    # Create JTM ranking (lower composite score = better rank)
    df_with_terciles['JTM_Ranking'] = df_with_terciles['composite_score'].rank(method='min', ascending=True)
    
    # Add positive demand growth indicator
    df_with_terciles['positive_demand_growth'] = df_with_terciles['OBS_GROWTH_NATIONAL'] > 0
    
    # Calculate terciles for the entire dataset (no MSA grouping in this case)
    data = df_with_terciles
    
    # 1. LOCAL DEMAND terciles (based on POSTINGS_OBS_LOCAL)
    df_with_terciles['lp_tercile'] = pd.qcut(data['POSTINGS_OBS_LOCAL'], 3, labels=[1, 2, 3])
    df_with_terciles['lp_tercile_label'] = df_with_terciles['lp_tercile'].map({1: 'Low', 2: 'Medium', 3: 'High'})
    
    # 2. NATIONAL DEMAND terciles (based on POSTING_OBS_NATIONAL)
    df_with_terciles['np_tercile'] = pd.qcut(data['POSTING_OBS_NATIONAL'], 3, labels=[1, 2, 3])
    df_with_terciles['np_tercile_label'] = df_with_terciles['np_tercile'].map({1: 'Low', 2: 'Medium', 3: 'High'})
    
    # 3. NATIONAL GROWTH terciles (based on OBS_GROWTH_NATIONAL - ALL occupations)
    df_with_terciles['growth_tercile'] = pd.qcut(data['OBS_GROWTH_NATIONAL'], 3, labels=[1, 2, 3])
    df_with_terciles['growth_tercile_label'] = df_with_terciles['growth_tercile'].map({1: 'Low Growth', 2: 'Medium Growth', 3: 'High Growth'})
    
    # 4. LOCAL GROWTH terciles (based on obs_growth_local)
    df_with_terciles['local_growth_tercile'] = pd.qcut(data['obs_growth_local'], 3, labels=[1, 2, 3])
    df_with_terciles['local_growth_tercile_label'] = df_with_terciles['local_growth_tercile'].map({1: 'Low Local Growth', 2: 'Medium Local Growth', 3: 'High Local Growth'})
    
    # Add salary growth estimate
    df_with_terciles['estimate_salary_growth'] = (df_with_terciles['ECI_SOC5_ESTIMATED'] * 100).round(2).astype(str) + " %"
    
    return df_with_terciles

# Add likelihood information to the dataframes
def add_likelihood_info(df, likelihood_df):
    """Add likelihood ratio and multi-industry info to dataframe"""
    df_with_likelihood = df.copy()
    
    # Add primary likelihood ratio
    primary_likelihood = (likelihood_df
                         .loc[likelihood_df.groupby('SOC_2021_5')['likelihood_ratio'].idxmax()]
                         [['SOC_2021_5', 'likelihood_ratio']])
    
    df_with_likelihood = df_with_likelihood.merge(
        primary_likelihood.rename(columns={'likelihood_ratio': 'Industry_Likelihood_Ratio'}),
        left_on='SOC_5',  # Changed from 'Occupation Code' to match your data
        right_on='SOC_2021_5',
        how='left'
    ).drop('SOC_2021_5', axis=1)
    
    # Add multi-industry flag
    df_with_likelihood['Multi_Industry'] = df_with_likelihood['SOC_5'].apply(
        lambda x: 'Yes' if str(x) in all_industries_mapping and len(all_industries_mapping.get(str(x), [])) > 1 else 'No'
    )
    
    # Add count of industries for multi-industry occupations
    df_with_likelihood['Industry_Count'] = df_with_likelihood['SOC_5'].apply(
        lambda x: len(all_industries_mapping.get(str(x), [])) if str(x) in all_industries_mapping else 0
    )
    
    return df_with_likelihood

# Create mappings for handling multi-industry occupations
def get_industry_mappings(likelihood_df):
    """
    Create mappings for both primary and all industries per occupation
    """
    # Primary industry (highest likelihood ratio)
    primary_industry = (likelihood_df
                       .loc[likelihood_df.groupby('SOC_2021_5')['likelihood_ratio'].idxmax()]
                       [['SOC_2021_5', 'industry', 'likelihood_ratio']])
    
    primary_mapping = dict(zip(primary_industry['SOC_2021_5'], primary_industry['industry']))
    
    # All industries mapping (for multi-industry handling)
    all_industries_mapping = {}
    for _, row in likelihood_df.iterrows():
        occ_code = row['SOC_2021_5']
        if occ_code not in all_industries_mapping:
            all_industries_mapping[occ_code] = []
        all_industries_mapping[occ_code].append({
            'industry': row['industry'],
            'likelihood_ratio': row['likelihood_ratio']
        })
    
    return primary_mapping, all_industries_mapping

def assign_sector_by_likelihood_soc5(soc_code):
    """
    Assign sector based on likelihood analysis with 1.8 threshold
    """
    if pd.isna(soc_code):
        return 'Other'
    
    soc_str = str(soc_code)
    
    # Check if we have likelihood data for this occupation
    if soc_str in occupation_to_industry:
        # Get the highest likelihood ratio for this occupation
        occ_data = result_df[result_df['SOC_2021_5'] == soc_str]
        if occ_data.empty:
            return 'Other'
            
        # Find the industry with highest likelihood ratio
        max_likelihood_row = occ_data.loc[occ_data['likelihood_ratio'].idxmax()]
        max_likelihood_ratio = max_likelihood_row['likelihood_ratio']
        industry = max_likelihood_row['industry']
        
        # Only assign to sector if likelihood ratio >= 1.8
        if max_likelihood_ratio >= 1.8:
            # Map our 5 target industries to sector names
            industry_to_sector = {
                'Construction': 'Construction',
                'Health': 'Healthcare', 
                'Manufacturing': 'Manufacturing',
                'Technology': 'Technology',
                'Transportation': 'Transportation & Distribution'
            }
            
            return industry_to_sector.get(industry, 'Other')
    
    # All others go to "Other" (below threshold or no data)
    return 'Other'

def calculate_living_wage_gap(salary, living_wage_threshold):
    """Calculate how much below living wage an occupation is"""
    if pd.isna(salary):
        return None
    gap = living_wage_threshold - salary
    return max(0, gap)  # Return 0 if already above living wage

# Define education levels and metrics
sub_ba = ['High school or GED', 'Associate degree', 'No Education Listed']
ba = ['Bachelor\'s degree', 'Master\'s degree', 'Doctoral degree', 'Professional degree']

metrics = ['POSTING_OBS_NATIONAL', 'OBS_GROWTH_NATIONAL', 'POSTINGS_OBS_LOCAL',
           'obs_growth_local', 'ECI_SOC5_ESTIMATED', 'salary_use', 'mean_tot_earn']
occ_cols = ['SOC_5', 'SOC_5_NAME', 'EDUCATION_LEVEL']

# Identify JTMs without living wage filter
sub_ba_jtm_no_lw = identify_jtm_no_living_wage(phoenix_lmi, sub_ba, metrics, occ_cols)
ba_jtm_no_lw = identify_jtm_no_living_wage(phoenix_lmi, ba, metrics, occ_cols)

# Add living wage gap calculations
sub_ba_jtm_no_lw['Living_Wage_Gap'] = sub_ba_jtm_no_lw['salary_use'].apply(
    lambda x: calculate_living_wage_gap(x, PHOENIX_LIVING_WAGE)
)

ba_jtm_no_lw['Living_Wage_Gap'] = ba_jtm_no_lw['salary_use'].apply(
    lambda x: calculate_living_wage_gap(x, PHOENIX_LIVING_WAGE)
)

# Filter for only those BELOW living wage
sub_ba_below_lw = sub_ba_jtm_no_lw[sub_ba_jtm_no_lw['Living_Wage_Gap'] > 0].copy()
ba_below_lw = ba_jtm_no_lw[ba_jtm_no_lw['Living_Wage_Gap'] > 0].copy()

if len(sub_ba_below_lw) > 0 or len(ba_below_lw) > 0:
    # Add terciles and rankings to both dataframes
    if len(sub_ba_below_lw) > 0:
        sub_ba_below_lw = add_terciles_and_rankings(sub_ba_below_lw)
    if len(ba_below_lw) > 0:
        ba_below_lw = add_terciles_and_rankings(ba_below_lw)
    
    # Add sector assignments using your likelihood-based function
    if len(sub_ba_below_lw) > 0:
        sub_ba_below_lw['Sector'] = sub_ba_below_lw['SOC_5'].apply(assign_sector_by_likelihood)
        # Add likelihood information
        sub_ba_below_lw = add_likelihood_info(sub_ba_below_lw, result_df)
    
    if len(ba_below_lw) > 0:
        ba_below_lw['Sector'] = ba_below_lw['SOC_5'].apply(assign_sector_by_likelihood_soc5)
        # Add likelihood information  
        ba_below_lw = add_likelihood_info(ba_below_lw, result_df)
    
    # Get all sectors
    all_sectors = []
    if len(sub_ba_below_lw) > 0:
        all_sectors.extend(sub_ba_below_lw['Sector'].unique())
    if len(ba_below_lw) > 0:
        all_sectors.extend(ba_below_lw['Sector'].unique())
    all_sectors = sorted(set(all_sectors))
    
    # Create sector dataframes
    below_lw_sector_dfs = {}
    
    for sector in all_sectors:
        # Get ALL columns for this sector
        sector_sub_ba = sub_ba_below_lw[sub_ba_below_lw['Sector'] == sector].copy() if len(sub_ba_below_lw) > 0 else pd.DataFrame()
        sector_ba = ba_below_lw[ba_below_lw['Sector'] == sector].copy() if len(ba_below_lw) > 0 else pd.DataFrame()
        
        # Add education type
        if not sector_sub_ba.empty:
            sector_sub_ba['Education Type'] = 'Sub-BA'
        if not sector_ba.empty:
            sector_ba['Education Type'] = 'BA'
        sub_ba_below_lw['Sector'] = sub_ba_below_lw['SOC_5'].apply(assign_sector_by_likelihood)
        # Combine
        combined_sector = pd.concat([sector_sub_ba, sector_ba], ignore_index=True)
        
        if not combined_sector.empty:
            # Sort by JTM ranking first, then education type, then smallest living wage gap
            combined_sector = combined_sector.sort_values(['JTM_Ranking', 'Education Type', 'Living_Wage_Gap'], ascending=[True, True, True])
            
            below_lw_sector_dfs[sector] = combined_sector

    # Export to Excel with sector information
    today_str = pd.Timestamp.now().strftime('%Y.%m.%d')
    below_lw_filename = f'JTMs_Below_Living_Wage_with_Sectors_{today_str}.xlsx'
    
    with pd.ExcelWriter(below_lw_filename, engine='openpyxl') as writer:
        for sector, df in below_lw_sector_dfs.items():
            clean_name = sector.replace('&', 'and').replace(' ', '_')[:31]
            df.to_excel(writer, sheet_name=clean_name, index=False)

else:
    pass

In [245]:
# Add BLS education data to phoenix_lmi (same as JTM datasets)
phoenix_lmi = phoenix_lmi.merge(
    bls_benchmark[['soc_2021_5', 'edulevel_name']],
    left_on='SOC_5',
    right_on='soc_2021_5',
    how='left'
).drop('soc_2021_5', axis=1)

# Add likelihood information to the dataframes
def add_likelihood_info(df, likelihood_df):
    df_with_likelihood = df.copy()

    primary_likelihood = (
        likelihood_df
        .loc[likelihood_df.groupby('SOC_2021_5')['likelihood_ratio'].idxmax()][['SOC_2021_5', 'likelihood_ratio']]
    )

    df_with_likelihood = df_with_likelihood.merge(
        primary_likelihood.rename(columns={'likelihood_ratio': 'Industry_Likelihood_Ratio'}),
        left_on='Occupation Code',
        right_on='SOC_2021_5',
        how='left'
    ).drop('SOC_2021_5', axis=1)

    df_with_likelihood['Multi_Industry'] = df_with_likelihood['Occupation Code'].apply(
        lambda x: 'Yes' if str(x) in all_industries_mapping and len(all_industries_mapping.get(str(x), [])) > 1 else 'No'
    )

    df_with_likelihood['Industry_Count'] = df_with_likelihood['Occupation Code'].apply(
        lambda x: len(all_industries_mapping.get(str(x), [])) if str(x) in all_industries_mapping else 0
    )

    return df_with_likelihood

# Merge BLS education data
sub_ba_jtm_ww_output = sub_ba_jtm_ww_output.merge(
    bls_benchmark[['soc_2021_5', 'edulevel_name']],
    left_on='Occupation Code',
    right_on='soc_2021_5',
    how='left'
).drop('soc_2021_5', axis=1)

ba_jtm_weighted_output = ba_jtm_weighted_output.merge(
    bls_benchmark[['soc_2021_5', 'edulevel_name']],
    left_on='Occupation Code',
    right_on='soc_2021_5',
    how='left'
).drop('soc_2021_5', axis=1)

# Add grouped education columns to JTM datasets
grouped_education_cols = ['SOC_5', 'Less_than_HS', 'HS_Some_College', 'BA', 'BA_Plus']
if all(col in phoenix_lmi.columns for col in grouped_education_cols):
    sub_ba_jtm_ww_output = sub_ba_jtm_ww_output.merge(
        phoenix_lmi[grouped_education_cols],
        left_on='Occupation Code',
        right_on='SOC_5',
        how='left'
    ).drop('SOC_5', axis=1)
    
    ba_jtm_weighted_output = ba_jtm_weighted_output.merge(
        phoenix_lmi[grouped_education_cols],
        left_on='Occupation Code',
        right_on='SOC_5',
        how='left'
    ).drop('SOC_5', axis=1)

# Apply sector logic and enrich
sub_ba_jtm_ww_output['Sector'] = sub_ba_jtm_ww_output['Occupation Code'].apply(assign_sector_by_likelihood)
ba_jtm_weighted_output['Sector'] = ba_jtm_weighted_output['Occupation Code'].apply(assign_sector_by_likelihood)

sub_ba_jtm_ww_output = add_likelihood_info(sub_ba_jtm_ww_output, result_df)
ba_jtm_weighted_output = add_likelihood_info(ba_jtm_weighted_output, result_df)

# Build sector_dfs
all_sectors = sorted(set(sub_ba_jtm_ww_output['Sector'].unique()) | set(ba_jtm_weighted_output['Sector'].unique()))
sector_dfs = {}

for sector in all_sectors:
    sub_ba_sector = sub_ba_jtm_ww_output[sub_ba_jtm_ww_output['Sector'] == sector].copy()
    ba_sector = ba_jtm_weighted_output[ba_jtm_weighted_output['Sector'] == sector].copy()

    if not sub_ba_sector.empty:
        sub_ba_sector['Education Type'] = 'Sub-BA'
    if not ba_sector.empty:
        ba_sector['Education Type'] = 'BA'

    combined_sector = pd.concat([sub_ba_sector, ba_sector], ignore_index=True)

    if not combined_sector.empty:
        combined_sector = combined_sector.sort_values(
            ['JTM Ranking', 'Education Type', 'Industry_Likelihood_Ratio'],
            ascending=[True, True, False]
        )

        if 'edulevel_name' in combined_sector.columns:
            combined_sector['BLS Education'] = combined_sector.pop('edulevel_name')

        combined_sector.drop(columns=[
            'Sector', 'Industry_Likelihood_Ratio', 'Multi_Industry', 'Industry_Count'
        ], inplace=True, errors='ignore')
        
        # Ensure grouped education columns are preserved (don't drop them)
        # The grouped education columns should already be in the dataframe from phoenix_lmi processing

        sector_dfs[sector] = combined_sector

# Rescue manual override occupations that aren't already JTMs
existing_occupation_codes = set(sub_ba_jtm_ww_output['Occupation Code']) | set(ba_jtm_weighted_output['Occupation Code'])
manual_missing_codes = [soc for soc in manual_sector_overrides if soc not in existing_occupation_codes]

for soc in manual_missing_codes:
    target_sector = manual_sector_overrides[soc]
    
    # Pull from enriched phoenix_lmi
    try:
        phoenix_match = phoenix_lmi[phoenix_lmi['SOC_5'].astype(str) == str(soc)]
        
        if not phoenix_match.empty:
            row_df = phoenix_match.iloc[[0]].copy()
            
            # Rename columns to match JTM format
            column_renames = {
                'SOC_5': 'Occupation Code',
                'SOC_5_NAME': 'Occupation',
                'EDUCATION_LEVEL': 'Education Level Needed',
                'salary_use': 'Estimated Salary',
                'phoenix_msa': 'Phoenix MSA',
                'tot_emp_55_plus': 'Employment 55+',
                'share_55_plus': 'Share 55+',
                'few_obs_flag': 'Few Observations Flag',
                'LLM_EXPOSURE': 'LLM Risk',
                'edulevel_name': 'BLS Education',
                # Add correct column mappings from phoenix_lmi
                'a_median': 'OEWS National Wage',
                'MEDIAN_SALARY_NATIONAL': 'National Posted Wage',
                'mean_tot_earn': '6-Year Total Earnings',
                # Add employment columns
                'tot_emp_local': 'Local Employment',
                'tot_emp': 'BLS Employment',
                # Add missing columns with correct names
                'local_growth_tercile_label': 'Local Growth',
                'estimate_salary_growth': 'Salary Growth Estimate',
                'a_median_local': 'ACS Local Wage',
                'MEDIAN_SALARY_LOCAL': 'Local Posted Wage',
                # Add grouped education columns
                'Less_than_HS': 'Less_than_HS',
                'HS_Some_College': 'HS_Some_College',
                'BA': 'BA',
                'BA_Plus': 'BA_Plus'
            }
            
            for old_col, new_col in column_renames.items():
                if old_col in row_df.columns:
                    row_df = row_df.rename(columns={old_col: new_col})
            
            # Calculate basic terciles
            try:
                if 'POSTINGS_OBS_LOCAL' in row_df.columns:
                    local_demand_value = row_df['POSTINGS_OBS_LOCAL'].iloc[0] if pd.notna(row_df['POSTINGS_OBS_LOCAL'].iloc[0]) else 0
                    if local_demand_value >= 100:
                        row_df['Local Demand'] = 'High'
                    elif local_demand_value >= 50:
                        row_df['Local Demand'] = 'Medium'
                    else:
                        row_df['Local Demand'] = 'Low'
                
                if 'POSTING_OBS_NATIONAL' in row_df.columns:
                    national_demand_value = row_df['POSTING_OBS_NATIONAL'].iloc[0] if pd.notna(row_df['POSTING_OBS_NATIONAL'].iloc[0]) else 0
                    if national_demand_value >= 1000:
                        row_df['National Demand'] = 'High'
                    elif national_demand_value >= 500:
                        row_df['National Demand'] = 'Medium'
                    else:
                        row_df['National Demand'] = 'Low'
                
                if 'OBS_GROWTH_NATIONAL' in row_df.columns:
                    growth_value = row_df['OBS_GROWTH_NATIONAL'].iloc[0] if pd.notna(row_df['OBS_GROWTH_NATIONAL'].iloc[0]) else 0
                    if growth_value >= 0.15:
                        row_df['National Growth'] = 'High Growth'
                    elif growth_value >= 0.05:
                        row_df['National Growth'] = 'Medium Growth'
                    else:
                        row_df['National Growth'] = 'Low Growth'
                
                # Add Local Growth calculation
                if 'obs_growth_local' in row_df.columns:
                    local_growth_value = row_df['obs_growth_local'].iloc[0] if pd.notna(row_df['obs_growth_local'].iloc[0]) else 0
                    if local_growth_value >= 0.10:  # Adjust thresholds as needed
                        row_df['Local Growth'] = 'High Local Growth'
                    elif local_growth_value >= 0.05:
                        row_df['Local Growth'] = 'Medium Local Growth'
                    else:
                        row_df['Local Growth'] = 'Low Local Growth'
                
                # Add Salary Growth Estimate calculation
                if 'ECI_SOC5_ESTIMATED' in row_df.columns:
                    eci_value = row_df['ECI_SOC5_ESTIMATED'].iloc[0] if pd.notna(row_df['ECI_SOC5_ESTIMATED'].iloc[0]) else 0
                    row_df['Salary Growth Estimate'] = f"{(eci_value * 100):.2f} %"
                    
            except Exception as e:
                # Set default terciles if calculation fails
                row_df['Local Demand'] = 'Low'
                row_df['National Demand'] = 'Low' 
                row_df['National Growth'] = 'Low Growth'
                row_df['Local Growth'] = 'Low Local Growth'
                row_df['Salary Growth Estimate'] = '0.00 %'
            
            # Determine education type
            sub_ba_codes = ['29-2061', '29-2081', '29-2099', '29-2092', '31-2012', '31-2011', '29-2072', '31-9095', '21-1094']
            edu_type = 'Sub-BA' if soc in sub_ba_codes else 'BA'
            row_df['Education Type'] = edu_type
            
            # Add salary source and leave JTM Ranking empty
            row_df['Salary Source'] = 'ACS Local Wage'
            row_df['JTM Ranking'] = None
            
            # Add default Phoenix MSA if missing
            if 'Phoenix MSA' not in row_df.columns or pd.isna(row_df['Phoenix MSA'].iloc[0]):
                row_df['Phoenix MSA'] = 'Phoenix–Mesa–Chandler'
            
            # Apply likelihood info
            try:
                row_df = add_likelihood_info(row_df, result_df)
            except Exception as e:
                # Continue without likelihood info if it fails
                pass
            
            # Add BLS education data manually if not in phoenix_lmi
            if 'BLS Education' not in row_df.columns or pd.isna(row_df['BLS Education'].iloc[0]):
                try:
                    bls_match = bls_benchmark[bls_benchmark['soc_2021_5'].astype(str) == str(soc)]
                    if not bls_match.empty:
                        row_df['BLS Education'] = bls_match.iloc[0]['edulevel_name']
                except Exception as e:
                    pass
            
            # Create sector if it doesn't exist
            if target_sector not in sector_dfs:
                sector_dfs[target_sector] = pd.DataFrame()
            
            # Match column structure
            if not sector_dfs[target_sector].empty:
                row_df = row_df.reindex(columns=sector_dfs[target_sector].columns, fill_value=np.nan)
            
            # Add to sector dataframe
            sector_dfs[target_sector] = pd.concat([sector_dfs[target_sector], row_df], ignore_index=True)
            
    except Exception as e:
        # Skip this occupation if any error occurs
        continue

# Re-sort all sectors
for sector in sector_dfs:
    if not sector_dfs[sector].empty:
        sector_dfs[sector] = sector_dfs[sector].sort_values(
            ['JTM Ranking', 'Education Type'], 
            ascending=[True, True], 
            na_position='last'
        )



/tmp/ipykernel_54907/1133080555.py:226: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sector_dfs[target_sector] = pd.concat([sector_dfs[target_sector], row_df], ignore_index=True)
/tmp/ipykernel_54907/1133080555.py:226: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sector_dfs[target_sector] = pd.concat([sector_dfs[target_sector], row_df], ignore_index=True)
/tmp/ipykernel_54907/1133080555.py:226: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. I

In [246]:
# Create data dictionary for export
data_dictionary = pd.DataFrame({
    'Variable': [
        'Phoenix MSA',
        'JTM Ranking', 
        'Occupation Code',
        'Occupation',
        'Education Level Needed',
        'Estimated Salary',
        'Salary Source',
        'Local Demand',
        'National Demand', 
        'National Growth',
        'Local Growth',
        'Salary Growth Estimate',
        'ACS Local Wage',
        'Local Posted Wage',
        'OEWS National Wage',
        'National Posted Wage',
        '6-Year Total Earnings',
        'Local Employment',
        'Employment 55+',
        'Share 55+',
        'Few Observations Flag',
        'LLM Risk',
        'Education Type',
        'BLS Education',
        'Less_than_HS',
        'HS_Some_College',
        'BA',
        'BA_Plus'
    ],
    'Description': [
        'Metropolitan Statistical Area identifier (Phoenix–Mesa–Chandler)',
        'Composite ranking of Jobs That Matter based on demand, growth, and wage metrics (lower number = better rank). Blank for manual overrides below living wage',
        '5-digit Standard Occupational Classification (SOC) code',
        'Occupation title/name',
        'Detailed education requirement for the occupation',
        'Primary salary estimate used in analysis',
        'Source of the salary estimate (e.g., "ACS Local Wage", "Manual Override")',
        'Local job posting volume tercile (Low/Medium/High)',
        'National job posting volume tercile (Low/Medium/High)',
        '10-year national employment growth tercile (Low/Medium/High Growth)',
        '10-year state/local employment growth tercile',
        'Projected annual wage growth percentage',
        'Local median wage from American Community Survey',
        'Median wage from local job postings',
        'National median wage from Occupational Employment and Wage Statistics',
        'National median wage from job postings',
        'Projected total earnings over 6 years',
        'Total employment in the Phoenix MSA for this occupation',
        'Number of workers age 55 and older in this occupation',
        'Percentage of workers age 55 and older',
        'Data quality flag indicating small sample size',
        'Large Language Model automation exposure score',
        'Simplified education category: Sub-BA (High school, Associate) or BA (Bachelor\'s and above)',
        'BLS benchmark education level distribution',
        'Percentage of workers with less than high school education',
        'Percentage of workers with high school through associate degree education',
        'Percentage of workers with bachelor\'s degree',
        'Percentage of workers with master\'s degree or higher'
    ],
    'Source': [
        'Geographic boundary definition',
        'Calculated composite score',
        'Bureau of Labor Statistics',
        'Bureau of Labor Statistics',
        'Bureau of Labor Statistics',
        'ACS Local Wage (preferred) or other sources',
        'Data processing metadata',
        'Job posting data, tercile calculation',
        'Job posting data, tercile calculation',
        'Bureau of Labor Statistics Employment Projections',
        'Arizona Department of Economic Security projections',
        'Employment Cost Index (ECI)',
        'American Community Survey',
        'Job posting data analysis',
        'Bureau of Labor Statistics OEWS',
        'Job posting data analysis',
        'Calculated projection',
        'American Community Survey',
        'American Community Survey',
        'American Community Survey',
        'American Community Survey processing',
        'Felten et al. AI Occupation Exposure research',
        'Derived from Education Level Needed',
        'Bureau of Labor Statistics National Employment Matrix',
        'BLS National Employment Matrix (grouped)',
        'BLS National Employment Matrix (grouped)',
        'BLS National Employment Matrix (grouped)',
        'BLS National Employment Matrix (grouped)'
    ]
})

# Export with data dictionary sheet
today_str = pd.Timestamp.now().strftime('%Y.%m.%d')

# Sort all sector dataframes before export
for sector_name in sector_dfs:
    if not sector_dfs[sector_name].empty:
        # Sort by: BA first (Education Type), then smallest JTM Ranking, then Sub-BA, then smallest JTM Ranking
        # NA values (manual overrides with no JTM ranking) go to the end within each education type
        sector_dfs[sector_name] = sector_dfs[sector_name].sort_values(
            ['Education Type', 'JTM Ranking'], 
            ascending=[False, True],  # False for Education Type puts 'Sub-BA' after 'BA'
            na_position='last'
        )

# For sector sheets export
sector_filename = f'JTM_Sectors_with_Manual_Overrides_{today_str}.xlsx'
with pd.ExcelWriter(sector_filename, engine='openpyxl') as writer:
    # Export each sector
    for sector, df in sector_dfs.items():
        if not df.empty:
            clean_name = sector.replace('&', 'and').replace(' ', '_')[:31]
            df.to_excel(writer, sheet_name=clean_name, index=False)
    
    # Add data dictionary sheet
    data_dictionary.to_excel(writer, sheet_name='Data_Dictionary', index=False)

# For below living wage export (if you have that)
if 'below_lw_sector_dfs' in globals():
    below_lw_filename = f'JTMs_Below_Living_Wage_with_Sectors_{today_str}.xlsx'
    with pd.ExcelWriter(below_lw_filename, engine='openpyxl') as writer:
        # Export each sector
        for sector, df in below_lw_sector_dfs.items():
            if not df.empty:
                clean_name = sector.replace('&', 'and').replace(' ', '_')[:31]
                df.to_excel(writer, sheet_name=clean_name, index=False)
        
        # Add data dictionary sheet
        data_dictionary.to_excel(writer, sheet_name='Data_Dictionary', index=False)

print(f"Exported with data dictionary: {sector_filename}")
if 'below_lw_sector_dfs' in globals():
    print(f"Exported with data dictionary: {below_lw_filename}")

Exported with data dictionary: JTM_Sectors_with_Manual_Overrides_2025.08.07.xlsx
Exported with data dictionary: JTMs_Below_Living_Wage_with_Sectors_2025.08.07.xlsx


In [248]:
# Write Enhanced Excel Deliverable – sub-BA and BA JTMs (Clean Version)
filename = f'Phoenix_JTM_Enhanced_Deliverable_{today_str}.xlsx'
print(f"Saving enhanced deliverable to: {filename}")

# Function to round numeric columns to 2 decimal places
def round_numeric_columns(df, decimal_places=2):
    df_rounded = df.copy()
    for col in df_rounded.columns:
        if df_rounded[col].dtype in ['float64', 'float32']:
            df_rounded[col] = df_rounded[col].round(decimal_places)
    return df_rounded

with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    
    # Write the data dictionary we created earlier (updated with ACS 55+ variables)
    data_dict_rounded = round_numeric_columns(data_dict_df)
    data_dict_rounded.to_excel(writer, sheet_name='Data Dictionary', index=False)
    
    # Write the wage hierarchy we created earlier
    wage_hierarchy_rounded = round_numeric_columns(wage_hierarchy_df)
    wage_hierarchy_rounded.to_excel(writer, sheet_name='Wage Hierarchy', index=False)
    
    # Sub-BA JTMs – MSA sheets
    for msa in sub_ba_jtm_ww_output['Phoenix MSA'].unique():
        msa_data = sub_ba_jtm_ww_output[sub_ba_jtm_ww_output['Phoenix MSA'] == msa]
        msa_data_rounded = round_numeric_columns(msa_data)
        sheet_name = f"SubBA - {str(msa)[:20]}"  # Shortened for Excel limits
        msa_data_rounded.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # BA JTMs – MSA sheets
    for msa in ba_jtm_weighted_output['Phoenix MSA'].unique():
        msa_data_ba = ba_jtm_weighted_output[ba_jtm_weighted_output['Phoenix MSA'] == msa]
        msa_data_ba_rounded = round_numeric_columns(msa_data_ba)
        sheet_name_ba = f"BA - {str(msa)[:23]}"  # Excel limit considerations
        msa_data_ba_rounded.to_excel(writer, sheet_name=sheet_name_ba, index=False)

print(f"✓ Enhanced deliverable saved with all decimals rounded to 2 places: {filename}")

Saving enhanced deliverable to: Phoenix_JTM_Enhanced_Deliverable_2025.08.07.xlsx
✓ Enhanced deliverable saved with all decimals rounded to 2 places: Phoenix_JTM_Enhanced_Deliverable_2025.08.07.xlsx
